In [46]:
!pip install torch torchvision torchaudio torch-geometric transformers sklearn nltk


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
    

In [47]:
# Install PyTorch Geometric and its dependencies
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.0.1+cu117.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.0.1+cu117.html
!pip install torch-geometric


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu117.html


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu117.html


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [48]:
import torch
import torch.nn.functional as F
import nltk

from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import sent_tokenize, word_tokenize
import networkx as nx
import numpy as np

# Download NLTK data
nltk.download('punkt')

class TextGraphSummarizer:
    def __init__(self, summary_size=0.1):
        self.summary_size = summary_size  # Ratio of sentences to retain

    def build_graph(self, sentences, tfidf_matrix):
        """Create a graph with nodes as sentences and edges based on TF-IDF similarity."""
        graph = nx.Graph()
        for i, sentence in enumerate(sentences):
            graph.add_node(i, sentence=sentence, tfidf=tfidf_matrix[i])

        # Edge creation: Similarity between sentence vectors
        for i in range(len(sentences)):
            for j in range(i+1, len(sentences)):
                weight = np.dot(tfidf_matrix[i], tfidf_matrix[j])
                if weight > 0:  # Connect only if they share common terms
                    graph.add_edge(i, j, weight=weight)
        return graph

    def generate_tfidf_weights(self, sentences):
        """Generate TF-IDF weighted matrix for the sentences."""
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(sentences).toarray()
        return tfidf_matrix

    def prepare_data_for_gnn(self, graph):
        """Prepare node features and edge index for GNN."""
        sentences = [graph.nodes[i]['sentence'] for i in graph.nodes]
        node_features = np.array([graph.nodes[i]['tfidf'] for i in graph.nodes])
        
        # Extracting edges and their weights
        edge_index = []
        edge_weight = []
        for edge in graph.edges(data=True):
            i, j = edge[0], edge[1]
            edge_index.append([i, j])
            edge_index.append([j, i])  # undirected graph
            edge_weight.append(edge[2]['weight'])
            edge_weight.append(edge[2]['weight'])

        # Convert to PyTorch tensors
        x = torch.tensor(node_features, dtype=torch.float)
        edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
        edge_weight = torch.tensor(edge_weight, dtype=torch.float)
        
        return Data(x=x, edge_index=edge_index, edge_weight=edge_weight), sentences

    def summarize(self, text):
        sentences = sent_tokenize(text)
        tfidf_matrix = self.generate_tfidf_weights(sentences)
        graph = self.build_graph(sentences, tfidf_matrix)
        data, sentences = self.prepare_data_for_gnn(graph)

        # GNN model
        class GNNModel(torch.nn.Module):
            def __init__(self, in_channels):
                super().__init__()
                self.conv1 = GCNConv(in_channels, 16)
                self.conv2 = GCNConv(16, 1)  # Single output to represent importance

            def forward(self, data):
                x, edge_index, edge_weight = data.x, data.edge_index, data.edge_weight
                x = F.relu(self.conv1(x, edge_index, edge_weight))
                x = self.conv2(x, edge_index, edge_weight)
                return x.squeeze()

        model = GNNModel(data.num_features)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
        
        # Training loop (basic; adjust for larger texts)
        model.train()
        for epoch in range(100):
            optimizer.zero_grad()
            out = model(data)
            loss = F.mse_loss(out, out)  # Placeholder loss, adjust as per training requirements
            loss.backward()
            optimizer.step()

        # Ranking sentences by importance
        model.eval()
        with torch.no_grad():
            sentence_scores = model(data)
        
        ranked_sentences = sorted(
            ((sentence_scores[i].item(), sentences[i]) for i in range(len(sentences))),
            reverse=True
        )

        # Selecting top sentences
        num_sentences = max(1, int(self.summary_size * len(sentences)))
        summary = " ".join([sentence for _, sentence in ranked_sentences[:num_sentences]])
        return summary

    def show_word_weightage(self, sentences):
        """Display word weights based on TF-IDF."""
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(sentences)
        tfidf_scores = dict(zip(vectorizer.get_feature_names_out(), tfidf_matrix.sum(axis=0).tolist()[0]))
        return tfidf_scores

# Example Usage
text = """Supreme Court of India
The Bharat Bank Ltd., Delhi vs The Employees Of The Bharat Bank Ltd., ... on 1 March, 1950
Equivalent citations: [1950]SUPPSCR317
Bench: H.J. Kania, Meher Chand Mahajan, Saiyad Fazal Ali
JUDGMENT
 

Kania, C.J.
 

1. I have read the judgments prepared by Messrs Fazl Ali, Mahajan and Mukherjea, JJ., in this case. As the views in those judgments in respect of the nature of the duties and functions of the Industrial Tribunal do not show agreement I consider it necessary to add a few words of my own.

2. In my opinion, the functions and duties of the Industrial Tribunal are very much like those of a body discharging judicial functions, although it is not a Court. The rules framed by the Tribunal required evidence to be taken and witnesses to be examined, cross-examined and re-examined. The Act constituting the Tribunal imposes penalties for incorrect statements made before the Tribunal. While the powers of the Industrial Tribunal in some respects are different from those of an ordinary Civil Court and it has jurisdiction and powers to give reliefs which a Civil Court administering the law of the land (for instance, ordering the re-instatement of a workman) does not possess in the discharge of its duties it is essentially working as a judicial body. The fact that it determination has to be followed by an order of the Government which makes the award binding, or that in cases where Government is a party the legislature is permitted to revise the decision, or that the Government is empowered to fix the period of the operation of the award do not, to my mind, alter the nature and character of the functions of the Tribunal. Having considered all the provision of the Act it seems to me clear that the Tribunal is discharging functions very near those of a Court, although it is not a Court in the technical sense of the word.

3. The next question is whether under Art. 136 the Court has jurisdiction to entertain an application for leave to appeal against the decision of such a body. It is not disputed that the Court has power to issue writs of certiorari of prohibition in respect of the work of the Tribunal. The only question is whether there is a right of appeal also. In my opinion the wording of Art. 136 is wide enough to give jurisdiction to the Court to entertain an application for leave to appeal, although it is obvious that having regard to the nature of the functions of the Tribunal, this Court will be very reluctant to entertain such an application.

4. As regards the merits, I do not think this is a case in which I would admit the appeal. The aggrieved parties may apply for redress by adopting other appropriate proceedings. The appeal, therefore should be dismissed with costs.

Fazl Ali, J.

5. The important question to be decided in this case is whether the present appeal lies at all to this Court. The question is not free from difficulty, but on the whole I am inclined to think that the appeal does lie. It is fully recognized that the scope of Art. 136 of the Constitution is very wide, but the significance of the language used in the section can be appreciated only by comparing it with the articles which precede it. Art. 132 deals with the Appellate Jurisdiction of the Supreme Court in cases involving a substantial question of law as to the interpretation in the Constitution, and the words used in that article are : "appeal........ from any judgment, decree or final order." Art. 133 deals with appeal in civil matters and the same words are used here also. Art. 134 deals with appeals in criminal matters, and the words used in it are; "appeal........ from any judgment, final order or sentence." In Art. 136, the words "judgment" and "decree," which are used in Arts. 132 and 133 are retained. Similarly, the words "judgment" and.........."sentence" occurring in Art. 134 are also retained. But the expression "final order" becomes "order," and, instead of the High Court, reference is made to "any Court." Certain other words are also used in the article which seem to me to have a special significance, these being "determination," "cause or matter" and "tribunal." It is obvious that these words greatly widen the scope of Art. 136. They show that an appeal will lie also from a determination or order of "any tribunal" in any cause or matter.

6. Can we then say that an industrial tribunal does not fall within the scope of Art. 136? If we go by a mere label, the answer must be in the affirmative. But we have to look further and see what are the main functions of the tribunal and how it proceeds to discharge those functions. This is necessary because I take it to be implied that before an appeal can lie to this court from a tribunal it must perform some kind of judicial function and partake to some extent of the character of a Court.

7. Now there can be no doubt that the industrial tribunal has, to use a well-known expression, "all the trappings of a Court" and performs functions which cannot but be regarded as judicial. This is evident from the rules by which the proceedings before the tribunal are regulated. It appears that the proceeding before it commences on an application which in many respects is in the nature of a plaint. It has the same powers as are vested in a Civil Court under the Code of Civil Procedure when trying a suit, in respect of discovery, inspection, granting adjournment, reception of evidence taken on affidavit, enforcing the attendance of witnesses, compelling the production of documents, issuing commissions, etc. It is to be deemed to be a Civil Court within the meaning of secs. 480 and 482 of the Criminal Procedure Code, 1898. It may admit and call for evidence at any stage of the proceeding and has the power to administer oaths. The parties appearing before it have the right of examination, cross- examination and re-examination and of addressing it after all evidence has been called. A party may also be represented by a legal practitioner with its permission.

8. The matter does not rest there. The main function of this tribunal is to adjudicate on industrial disputes which implies that there must be two or more parties before it with conflicting cases, and that it has also to arrive at a conclusion as to how the dispute is to be ended. prima facie, therefore, a tribunal like this cannot be excluded from the scope of Art. 136, but before any final conclusion can be expressed on the subject certain contentions which have been put forward on behalf of the Respondents have to be disposed of.

9. The first contention is that the industrial tribunal cannot be said to perform a judicial or quasi-judicial function, since it is not required to be guided by any recognised substantive law in deciding disputes which come before it. On the other hand, in deciding industrial disputes, it has to override contracts and create rights which are opposed to contractual rights. In these circumstances, it is said that the very questions which arose before the Privy Council in Moses v. Parker, Ex parte Moses (1) arise in this case, these questions being :-

(1) How can the propriety of the tribunals' decision be tested on appeal ? and (2) What are the canons by which the Appellate Court is to be guided in deciding the appeal ?
10. Their Lordships of the Privy Council undoubtedly felt that these were serious questions, but they had no hesitation in saying that "if it were clear that appeals ought to be allowed, such difficulties would doubtless be met somehow." This, in my opinion, is a sufficient answer to the difficulty raised. The tribunal has to adjudicate in accordance with the provisions of the Industrial Disputes Act. It may sometimes override contracts, but so can a Court which has to administer law according to the Bengal or Bihar Money-Lenders Act, Encumbered Estates Act, and other similar Acts. The tribunal has to observe the provisions of the special law which it has to administer through that law may be different from the law which an ordinary Court of justice administers. The Appellate Court, therefore, can at least see that the rules according to which it has to act and the provisions which are binding upon it are observed, and its powers are not exercised in an arbitrary or capricious manner.

11. The second contention, which is a more serious one, is that the adjudication of the tribunal has not all the attributes of a judicial decision, because the adjudication cannot bind the parties until it is declared to be binding by the Government under section 15 of the Industrial Disputes Act. It is said that the adjudication is really in the nature of an advice or report which is not effective until made so by the Government. It appears that a similar objection was raised in Rex v. Electricity Commissioners, London Electricity Joint Committee Co., Ex parte (2) for the purpose of deciding whether a writ of certiorari should be issued in the circumstances of the case but was disposed of in these words :-

"It is necessary, however, to deal with what I think was the main objection of the Attorney-General. In this case he said the commissioners come to no decision at all. They act merely as advisers. They recommend an order embodying a scheme to the Minister of Transport, who may confirm it with or without modifications. Similarly, the Minister of Transport comes to know decision. He submits the order to the Houses of Parliament, who may approve it with or without modifications. The Houses of Parliament may put anything into the order they please, whether consistent with the Act of 1919, or not. Until they have approved, nothing is decided, and in truth the whole procedure, draft scheme, inquiry, order, confirmation, approval, is only part of a process by which Parliament is expressing its will, and at no stage is subject to any control by the Courts. It is unnecessary to emphasize the constitutional importance of this contention................ In the provision that the final decision of the Commissioner is not to be operative until it has been approved by the two Houses of Parliament I find nothing inconsistent with the view that in arriving at that decision the Commissioner themselves are to act judicially and within the limits prescribed by Act of Parliament, and that the Courts have power to keep them within those limits. It is to be noted that it is the order of the Commissioner that eventually takes effect; neither the Minister of Transport who confirms, nor the Houses of Parliament who approve, can under the statute make an order which in respect of the matter in question has any operation. I known of no authority which compels me to hold that a proceeding cannot be a judicial proceeding, subject to prohibition or certiorari because it is subject to confirmation or approval, even where the approval has to be that of the Houses of Parliament. The authorities are to the contrary."
12. It is well-known that a writ of certiorari can issue only against an order of a judicial or quasi-judicial tribunal and if it is permissible for the High Court to issue a writ of certiorari against an industrial tribunal, which fact was not seriously disputed before us, I find is difficult to hold that the tribunal does not come within the purview of Art. 136. If a subordinate Court acts in excess of its jurisdiction or assumes a jurisdiction which it does not possess, the Appellate Court can always interfere and do what is contemplated to be done by writ of certiorari.

13. It is to be noted that under section 15 of the Industrial Disputes Act, 1947, in cases where the appropriate Government is not a party to the dispute, all that the Government has to do on receiving the award of the tribunal is to declare it to be binding and to state from what date and for what period it will be binding. Section 15(2) is mandatory and it provides :

"On receipt of such award, the appropriate Government shall by order in writing declare the award to be binding. . . ."
14. Thus the Government cannot alter, or cancel, or add to the award, but the award must be declared to be binding as it is. In substance, therefore, the adjudication of the tribunal amounts to a final determination of the dispute which binds the parties as well as the Government.

Our attention was however drawn to the proviso to section 15(2), which runs as follows :-

"Provided that where the appropriate Government is a party to the dispute and in its opinion it would be inexpedient on public grounds to give effect to the whole or any part of the award, it shall on the first available opportunity lay the award together with the statement of its reasons for not making a declaration as aforesaid before the Legislative Assembly of the Province, or where the appropriate Government is the Central Government, before the Central Legislative Assembly, and shall, as soon as may be, cause to be moved therein a resolution for the consideration of the award, and the Legislative Assembly may by its resolution, confirm, modify, or reject the award."
15. This proviso was relied upon by the Respondents to show that the right to appeal from the award could not have been contemplated in any case. But the Act itself makes a distinction between cases in which the Government is a party and those in which the Government is not a party. The proviso relates to a very special type of case and as at present advised I do not wish to express any opinion as to whether an appeal lies to this Court or not in such a case, but, in my judgment, where the Government has only to declare the award to be binding, an appeal shall lie.

16. It is necessary here to say a few words as to the scope of the appeal. As was pointed out by this Court in Pritam Singh v. The State (3), the power under Art. 136 of the Constitution being a special power is to be exercised only in special cases. The rule so laid down is bound to restrict the scope of the appeal in practice in almost all the cases which fall under Art. 136. But in some cases a limitation will be imposed on the scope of the appeal by the very nature of the case and of the tribunal from which an appeal is sought to be brought, and a case under the Industrial Disputes Act seems to be an example of such a case.

17. Dealing now with the merits of the appeal. I am not prepared to hold that this is a proper case for interference with the adjudication of the tribunal. The power of this Court was invoked by the Appellants on four grounds. These grounds have been elaborately examined by Mahajan, J., and two of them have been pronounced to be wholly inadequate for justifying our interference. May view with regard to these two grounds is identical with that of Mahajan, J. and I do not wish to add to what has already said on the subject. The remaining two grounds also are, in my opinion, wholly insufficient to justify the exercise of our special power under Art. 136. One of these grounds is that the award of the tribunal is based on no evidence whatsoever. I do not, however, find that this ground was urged in this form in the application for special leave to appeal to this Court. All that was intended to be urged was that the Appellants wanted to adduce evidence but were not allowed to do so. From the decision of the tribunal, however, it appears that the evidence that was that our related to one isolated point only and the tribunal might well have been justified in not allowing evidence to be admitted on a point which in its opinion had no direct bearing on the issue before them. After hearing the Respondents on this particular point. I am not disposed to hold that the tribunal has committed such an error as would justify the interference of this Court.

18. The last ground urged is that the award has been signed by only two members of the tribunal though it originally consisted of three persons and though the entire hearing of the dispute had taken place before all the three persons. This objection does not appear to me to be fatal to the jurisdiction of the tribunal, because under section 8 of the Act it is not obligatory on the Government to appoint a new member to fill a vacancy if one of the members ceases to be available at any time during the proceedings. Under that section, if the Chairman ceases to be available, the Government must appoint his successor, whereas if a member ceases to be available the Government may or may not appoint any one to fill his place. In the present case, our attention was drawn to some correspondence which shows that one of the members was called upon to act as a member or another tribunal and the award in question was pronounced after informing the Government of the procedure which the Chairman and the remaining member intended to adopt.

19. In the view I have taken, this appeal must fail, and I would accordingly dismiss it with costs, Mahajan, J.

20. This is an appeal by special leave from the determination of an industrial dispute by the Industrial Tribunal appointed under Ordinance VI of 1949.

21. Bharat Bank, Limited, Delhi, the Appellant, is a company registered under the Indian Companies Act. Its employees made certain demands and as a result of an unfavourable response from the bank it appears that they struck work on the 9th March, 1949. The bank in its turn served notices on them to resume work and proceeded to discharge a number of them between the 19th March and 24th March as they failed to do so. The Central Government constituted a tribunal consisting of three persons for the adjudication of industrial disputes in banking companies under section 7 of the Industrial Disputes Act (XIV of 1947). The disputes mentioned in Schedule II of the notification were referred under section 10 of the Act to this Tribunal Item 18 of this schedule reads as follows :-

"Retrenchment and victimization (Specific cases to be cited by employees)."
22. The dispute under this item between the Bharat Bank and its employees was heard by the tribunal at Delhi and its award was made on the 19th January, 1950. It was published in the Government of India Gazette dated 4th February, 1950 and was declared to be binding for a period of one year. The award of the tribunal was signed by two out of its three members.

23. A preliminary objection was raised on behalf of the Central Government as well as on behalf of the Respondents that this Court had not jurisdiction to grant special leave to appeal against the determination of an industrial tribunal inasmuch as it did exercise the judicial powers of the State and that its determination was not in the nature of a judgment, decree or order of a Court so as to be appealable. This being the first case in which special leave was granted from the determination of an industrial tribunal, it is necessary to examine the provisions of the Constitution dealing with this matter and if possible, to define the limits of the jurisdiction of this Court under Art. 136. This article is in these terms :-

"(1) Notwithstanding anything in this Chapter, the Supreme Court may, in its discretion, grant special leave to appeal from any judgment, decree, determination, sentence or order in any cause or matter passed or made by any Court or tribunal in the territory of India.
(2) Nothing in clause (1) shall apply to any judgment, determination, sentence or order passed or made by any Court or tribunal constituted by or under any law relating to the Armed Forces."
24. The article occurs in Chapter IV of Part V of the Constitution "The Union Judiciary." Art. 124 deals with the establishment and constitution of the Supreme Court. Art. 131 confers original jurisdiction on this Court in certain disputes arising between the Government of India and the States, etc. Arts. 132 and 133 deal with the appellate jurisdiction of the Court in appeals from High Courts within the territory of India in civil matters. By Art. 134 limited right of appeal in criminal cases has been allowed. The Judicial Committee of the Privy Council which was the highest Court of Appeal for India prior to 10th October, 1949, was not a Court of criminal appeal in the sense in which this Court has been made a Court of criminal appeal under Art. 134. It could only entertain appeals on the criminal side in exercise of the prerogative of the Kings. Art. 135 empowers this Court to hear all appeals which under existing laws could heard by the Federal Court of India. By the Abolition of the Privy Council Jurisdiction Act, 1949, which came into force on the 10th October, 1949, all the powers that were possessed by the Judicial Committee of the Privy Council in regard to cases or matters arising in India became exercisable by the Federal Court of India whether those powers were exercisable by reason of statutory authority or under the prerogative of the King. The powers of the Judicial Committee were conferred upon it by the Judicial Committee Act, 1844 (7 & 8 Vict. C. 69). Appeals lay to His Majesty in Council from judgments, sentences, decrees or orders of any Court of justice within any British colony or possession abroad. Closely following Art. 135 which confers all the powers of the judicial Committee on the Supreme Court comes Art. 136. The language employed in this article is very wide and is of a comprehensive character. Power given are of an overriding nature. The article commences with the words "Notwithstanding anything in this Chapter." These words indicate that the intention of the Constitution was to disregard in extraordinary cases the limitations contained in the previous article on this Court's power to entertain appeals. These articles dealt with the right of appeal against final decisions of High Courts within the territory of India Art. 136, however, overrides that qualification and empowers this Court to grant special leave even in case where the judgment has not been given by a High Court but has been given by any Court in the territory of India : in other words, it contemplates grant of special leave in cases where a Court subordinate to a High Court has passed or made any order and the situation demands that the order should be quashed or reversed even without having recourse to the usual procedure provided by law in the nature of an appeal, etc. The word "order" in Art. 136 has not been qualified by the word "final." It is clear, therefore, that the power to grant special leave under this article against an order of a Court could be exercised with respect to interlocutory orders also. Another new feature introduced in Art. 136 is the power given to grant special leave against orders and determinations, etc., of any tribunal in the territory of India. This word did not find place in the Judicial Committee's Act, where the phrase used was "a Court of justice." It is the introduction of this new expression in Art. 136 that has led to considerable argument as to its scope. Another expression that did not find place in the Judicial Committee Act but has been introduced in Art. 136 is the word "determination." A question has been raised as to the meaning to be given to these words in the article. On the one hand, it was contended that the words "determination" and "tribunal" were introduced in the article in order to bring within the scope of the appellate jurisdiction of this Court all orders of tribunals of different varieties and descriptions. On the other hand, it was said that the words "determination" and "tribunal" were added in the article by way of abundant caution and the intention was that if a tribunal exercised the judicial powers of the State and the decisions passed in the exercise of that power, this Court as the highest judicial Court in the Republic would have power if it considered necessary in the ends of justice to grant special leave. Clause (2) of Art. 136 excludes the jurisdiction of this Court in respect of military Courts or tribunal. It is interesting to observe that in Art. 138, 139 and 140 the Constitution has conferred powers on Parliament for further enlargement of the powers of this Court.

25. Two points arise for determination in this case, (1) whether the word "Tribunal" in this article has been used in the same sense as "Court," or whether it has been used in a wider sense, and (2) whether the word "determination" in the article includes within its scope the determinations made by industrial tribunals or other similarly constituted bodies or whether it has reference only to determinations of a Court or a tribunal of a purely judicial character. It was conceded by the learned Counsel appearing for the Central Government, Mr. Alladi Krishnaswami Aiyar, that if any tribunal, whether administrative, domestic or quasi-judicial, acts in excess of its jurisdiction, then it can be controlled by the High Court under the powers conferred on them by Art. 226 by the issue of a writ of certiorari. It was said that if the industrial tribunal in this case could be proved to have trespassed beyond the limits of its statutory jurisdiction, then the remedy lies elsewhere and not be a petition of leave under Art. 136. Mr. Alladi's contentions may be briefly summarized as follows : (1) The expression "tribunal" means seat of a Judge, or a Court of justice. Its necessary attribute is that it can give a final judgment between two parties which carries legal sanction by its own force. That the word "tribunal" in juxtaposition to the word "Court" could only mean a tribunal which exercised judicial functions of the State and did not include within its ambit a tribunal which had quasi-tribunal or administrative powers. (2) The kind of orders against which special leave to appeal could be given under Art. 136 have to be of the same nature as passed by a Court : in other words, it was said that unless there was a judicial determination of a controversy between two parties, the order would not be appealable. That in the case of an industrial tribunal what gives binding force to the award is the declaration of the government, that the spark of life to it is given by that declaration and without that the award of the tribunal is lifeless and has no enforceability and hence cannot be held to be of an appealable nature. It was further said that in cases between the Government and its employees by the procedure prescribed in the Act the award could also be rejected and that being so, by its own determination a tribunal could not impose a liability or affect rights. Dr. Bakshi Tek Chand, appearing for the bank, on the other hand, argued that whenever a tribunal, whether exercising judicial or quasi-judicial functions, determined a matter in a judicial manner, then such a determination is within Art. 136. It was said that an industrial tribunal has no administrative or executive functions, that its duty is to adjudicate on an industrial dispute, i.e. to act as a Judge on certain kinds of disputes between employers and employees and that its functions are of a judicial nature, though the ambit of the powers conferred is larger than that of an ordinary Court of law inasmuch as it can grant reliefs which no Court of law could give but that is because of the powers conferred on it by law. It was argued that the plain words of the article should not be given a narrow meaning when the intention of the Constitution was to confer the widest power on this Court. It was further contended that as between private employers and employees and even in certain cases between Government and its employees the decision of the tribunal was binding on the Government and Government had no power either to affirm, modify or reject it. All that it was authorised to do was to announce it and by its declaration give it enforceability; that fact, however, could not affect the question of appealability of the determination under Art. 136. It was finally argued that powers should be exercised by this Court wherever there is a miscarriage of justice by a determination of any tribunal and that if the intention of the Constitution by use of the word "tribunal" was in the same sense as "Court," then it was not necessary to import it in Art. 136, because all tribunals that exercise judicial functions fall within the definition of the word "Court" though they may not have been so described.

26. After considerable thought I have reached the conclusion that the preliminary objection should be overruled. I see no cogent reasons to limit the plain words of the statute and to place a narrow interpretation on words of widest amplitude used therein. In construing the articles of the Constitution it has always to be remembered that India has been constituted into a sovereign democratic republic in order to ensure justice to all its citizens. In other words, the foundations of this republic have been laid on the bedrock of justice. To safeguard these foundations so that they may not be undermined by injustice occurring anywhere, this Court has been constituted. By Art. 3 of the Constitution the Court is empowered to see that the fundamental rights conferred on the citizens by the Constitution are not in any way affected. By Art. 136 it has been given overriding power to grant special leave to appeal against orders of Court and tribunals which go against the principles of natural justice and lead to grave miscarriage of justice. The exercise of these powers could only have been contemplated in cases which affect the rights of people living within the territory of India in respect of their person, property or status. The question, therefore, for consideration is whether the jurisdiction conferred by use of unambiguous phraseology and by words which have a plain grammatical meaning and are of the widest amplitude should be limited and restricted on considerations suggested by Mr. Alladi. The construction suggested by the learned Counsel, if accepted, would in the first instance make the use of certain words in the article unnecessary and redundant and would run counter to the spirit of the Constitution. It must be presumed that the draftsmen of the Constitution knew well the fact that there were a number of tribunals constituted in this country previous to the coming into force of the Constitution which were performing certain administrative, quasi-judicial or domestic functions, that some of them had even the trappings of a Court but in spite of those trappings could not be given that description. If must also be presumed that the Constitution-makers were aware of the fact that the highest Courts in this country had held that all tribunals that discharged judicial functions fell within the definition of the expression "Court." If by the use of the word "tribunal" in Article 136 the intention was to give it the same meaning as "Court," then it was redundant and unnecessary to import it in the article because, by whatever name described, such a tribunal would fall within the definition of the word "Court." The word "Court" has a well-known meaning in legislative history and practice.

27. As pointed out in Halsbury's Laws of England, the word "Court" originally meant the King's Palace but subsequently acquired the meaning of (1) a place where justice was administered, and (2) the person or persons who administer it. In the Indian Evidence Act it is defined as including all Judges and Magistrates and all persons except arbitrators legally authorised to take evidence. This definition is by no means exhaustive and has been framed only for the purpose of the Act. There can be no doubt that to be a Court, the person or persons who constitute it must be entrusted with judicial functions, that is, of deciding questions according to law. However, by agreement between parties arbitrators may be called upon to exercise judicial powers and to decide a dispute according to law but that would not make the arbitrators a Court. It appears to me that before a person or persons can be said to constitute a Court it must be held that they derive their powers from the State and are exercising the judicial powers of the State. In R. v. London County Council, Saville, L. J., gave the following meaning to the word "Court" or "judicial authority" :-

"It is not necessary that it should be a Court in the sense that this Court is a Court : it is enough if it is exercising, after hearing evidence, judicial functions in the sense that it has to decide on evidence between a proposal and an opposition; and that it is not necessary to be strictly a Court; if it is a tribunal which has to decide rightly after hearing evidence and opposition."
28. As pointed out in picturesque language by Lord Sankey, L. C., in Shell Co. of Australia v. Federal Commissioner of Taxation, there are tribunals with many of the trappings of a Court which, nevertheless, are not Courts in the strict sense of exercising judicial power. It seems to me that such tribunals though they are not full-fledged Courts, yet exercise quasi-judicial functions are within the ambit of the word "tribunal" in Art. 136 of the Constitution. It was pointed out in the above case that a tribunal is not necessarily a Court in this strict sense because it gives a final decision, nor because it hears witnesses on oath, nor because two or more contending parties appear before it between whom it has to decide, nor because it gives decisions which affect the rights of subjects nor because there is an appeal to a Court, nor because it is a body to which a matter is referred by another body. The intention of the Constitution by the use of the word "tribunal" in the article seems to have been to include within the scope of Art. 136 tribunals adorned with similar trappings a Court but strictly not coming within that definition. Various definitions of the phrase "judicial power" have been given from time to time. The best definition of it on high authority is the one given by Griffith, C.J., in Huddart, Parker & Co. v. Moorehead, wherein it is defined as follows :-

"The words 'judicial power' as used in section 71 of the Constitution mean the power which every sovereign authority must of necessity have to decide controversies between its subjects, or between itself and its subjects, whether the rights relate to life, liberty or property. The exercise of this power does not begin until some tribunal which has power to give a binding and authoritative decision (whether subject to appeal or not) is called upon to take action."
29. It was conceded that a tribunal constituted under the Industrial Disputes Act, 1947, exercises quasi-judicial powers. That phrase implies that a certain content of the judicial power of the State is vested in it and it is called upon to exercise it. An attempt was made to define the words "judicial" and "quasi-judicial" in the case of Cooper v. Wilson. The relevant quotation reads thus :-

"A true judicial decision presupposes an existing dispute between two or more parties, and then involves four requisites :- (1) The presentation (not necessarily orally) of their case by the parties to the dispute; (2) if the dispute between them us a question of fact, the ascertainment of the fact by means of evidence adduced by the parties to the dispute and often with the assistance of argument by or on behalf of the parties on the evidence; (3) if the dispute between them is a question of law, the submission of legal argument by the parties, and (4) a decision which disposes of the whole matter by a finding upon the facts in dispute and application of the law of the land to the facts so found, including where required a ruling upon any disputed question of law. A quasi-judicial decision equally presupposes an existing dispute between two or more parties and involves (1) and (2), but does not necessarily involve (3) and never involves (4). The place of (4) is in fact taken by administrative action, the character of which is determined by the Minister's free choice."
The extent of judicial power exercise by an Industrial Tribunal will be considered hereinafter in the light of the observations cited above.

30. Reference was made to certain passengers from Prof. Allen's book on Law and Order, Ch. IV, page 69, where mention is made of the kinds of administrative tribunals functioning in various countries to-day. Porter on Administrative Law, 1929 Edn., page 194, was also relied upon. There can be no doubt that varieties of administrative tribunals and domestic tribunals are known to exist in this country as well as in other countries of the world but the real question to decide in each case is as to the extent of judicial power of the State exercised by them. Tribunals which do not derive authority from the sovereign power cannot fall within the ambit of Art. 136. The condition precedent for bringing a tribunal within the ambit of Art. 136 is that it should be constituted by the State. Again a tribunal would be outside the ambit of Art. 136 if it is not invested with any part of the judicial functions of the State but discharges purely administrative or executive duties. Tribunals, however, which are found invested with certain functions of a Court of justice and have some of its trappings also would fall within the ambit of Art. 136 and would be subject to the Appellate control of this Court whenever it is found necessary to exercise that control in the interests of justice.

31. It is now convenient to consider whether a tribunal constituted under the Industrial Disputes Act, 1947, exercises all or any of the functions of a Court of justice and whether it discharges them according to law or whether it can act as it likes in its deliberations and is guided by its own notions of right and wrong. The phrase "industrial dispute" has been defined in section 2 clause (k) of the Act as follows :-

"any dispute or difference between employers and employees, or between employers and workmen, or between workmen and workmen, which is connected with employment or non-employment or the terms of employment or with the conditions of labour, of any person."
32. Such a dispute concerns the rights of employers and employees. Its decision affects the terms of a contract of service or the conditions of employment. Not only may the pecuniary liability of an employer be considerably affected by the adjudication of such dispute but it may even result in the imposition of punishments on him. It may adversely affect the employees as well. Adjudication of such a dispute affects valuable rights. The dispute and its result can always be translated in terms of money. The point for decision in the dispute usually is how much money has to pass out of the pocket of the employer to the pocket of the employee in one form or another and as to what extent the right of freedom of contract stands modified to bring about industrial peace. Power to adjudicate on such a dispute is given by section 7 of the statute to an Industrial Tribunal and a duty is cast on it to adjudicate it in accordance with the provisions of the Act. The words in italics clearly imply that the dispute has to be adjudicated according to law and not in any other manner. When the dispute has to be adjudicated in accordance with the provisions of the Act, it follows that the tribunal has to adhere to law, though that law may be different from the law that an ordinary Court of justice administers. It is noteworthy that the tribunal is to consist of experienced judicial officers and its award is defined as a determination of the dispute. The expression "adjudication" implies that the tribunal is to act as a Judge of the dispute; in other words, it sits as a Court of justice and does not occupy the chair of an administrator. It is pertinent to point out that the tribunal is not given any executive or administrative powers. In Section 38 of the Act power is given to make rules for the purpose of giving effect to the provisions of the Act. Such rules can provide in respect of matters which concern the powers and procedure of tribunals including rules as to the summoning of witnesses, the production of documents relevant to the subject-matter and as to appearance of legal practitioners in proceedings under this Act. Rule 3 of these rules provides that any application for the reference of an industrial dispute to a tribunal shall be made in form. (A) and shall be accompanied by a statement setting forth, inter alia, the names of the parties to the dispute and the specific matters of dispute. It is in a sense in the nature of a plaint in a suit. In Rule 13 power is given to administer oaths. Rule 14 provides as follows :-

"A tribunal may accept, admit or call for evidence at any stage of the proceedings before it and in such manner as it may think fit."
33. Rule 17 provides that at its first sitting the tribunal is to call upon the parties to state their case. In rule 19 provision has been made for proceedings ex parte. Rue 21 provides that in addition to the powers conferred by sub-section (3) of section 11 of the Act, a tribunal shall have the same powers as are vested in a Civil Court under the Code of Civil Procedure when trying a suit, in respect of the following matters, namely, (a) discovery and inspection; (b) granting of adjournment; (c) reception of evidence taken on affidavit; and that the tribunal may summon and examine suo motu any person whose evidence appears to it to be material. It further says that the tribunal shall be deemed to be a Civil Court within the meaning of secs. 480 and 482 of the Code of Criminal Procedure, 1898, Rule 21 says that the representation of the parties, appearing before a tribunal, shall have the right of examination, cross-examination and re-examination and of addressing the court or tribunal when all evidence has been called. In rule 30 it is provided that a party to a reference may be represented by a legal practitioner with the permission of the tribunal and subject to such conditions as the tribunal may impose. In section 11(3) it is said that a tribunal shall have the same powers as are vested in a Civil Court under the Code of Civil Procedure when trying a suit, in respect of the following matters, namely, (a) enforcing the attendance of any person and examining him on oath; (b) compelling the production of documents and material objects; (c) issuing commissions for the examination of witnesses; (d) in respect of such other matters as may be prescribed; and every inquiry or investigation by a tribunal shall be deemed to be a judicial proceeding within the meaning of secs. 193 and 228 of the Indian Penal Code. It is difficult to conceive in view of these provisions that the Industrial Tribunal performs any functions other than that of a judicial nature. The tribunal has certainly the first three requisites a considerable element of the fourth also inasmuch as the tribunal cannot take any administrative action, the character of which is determined by its own choice. It has to make the adjudication in accordance with the provisions of the Act as laid down in section 7. It consists of persons who are qualified to be or have been Judges. It is its duty to adjudicate on a serious dispute between employers and employees as affecting their right of freedom of contract and it can impose liabilities of a pecuniary nature and disobedience of its award is made punishable. The powers exercisable by a tribunal of the nature were considered in a judgment of the Federal Court of India in Western India Automobile Association v. Industrial Tribunal, Bombay, and it was observed that such a tribunal can do what no Court can, namely, add to or alter the terms or conditions of the contract of service. The tribunal having been entrusted with the duty of adjudicating a dispute of a peculiar character, it is for this reason that it is armed with extraordinary powers. These powers, however, are derived from the statute. These are the rules of the game and it has to decide according to these rules. The powers conferred have the sanction of law behind it and are not exercisable by reason of any discretion vested in the members of the tribunal. The adjudication of the dispute has to be in accordance with evidence legally adduced and the parties have a right to be heard and being represented by a legal practitioner. Right to examine and cross- examine witnesses has been given to the parties and finally they can address the tribunal when evidence is closed. The whole procedure adopted by the Act and the rules is modelled on the Code of Civil Procedure. In my opinion, therefore, the Industrial Tribunal has all the necessary attributes of a Court of justice. It has no other function except that of adjudicating on a dispute. It is no doubt true that by reason of the nature of the dispute that they have to adjudicate the law gives them wider powers than are possessed by ordinary Courts of law, but powers of such a nature do not affect the question that they are exercising judicial power. Statutes like the Relief of Indebtedness Act, or the Encumbered Estates Act have conferred powers on Courts which are not ordinarily known to law and which affect contractual rights. That circumstances does not make them anything else but tribunals exercising judicial power of the State, though in a degree different from the ordinary Courts and to an extent which is also different from that enjoyed by an ordinary Court of law. They may rightly as quasi-judicial bodies because they are out of the hierarchy of the ordinary judicial system but that circumstances cannot affect the question of their being within the ambit of Art. 136.

34. It may also be observed that the tribunal is deemed to be a Civil Court for certain purposes as laid down in rule 21 of the rules above cited and in section 11(3) of the Act. As a Civil Court if it exercises any of the powers contemplated by this section its decisions would become subject to appeal to a District Judge and a fortiori this Court's power under Art. 136 would at once be attracted in any case in respect of these matters. Again in Chapter VI of the Act breach of the terms of an award has been made punishable by section 29 of the Act. The result, therefore, is that disobedience of the terms of an award is punishable under the Act. That being so, a determination of the tribunal not only affects the freedom of contract and imposes pecuniary liability on the employer or confers pecuniary benefits on the employees, but it also involves serious consequences as failure to observe those terms makes a persons liable to the penalties laid down in Chapter VI. An award which has these serious consequences can hardly be said to have been given by a tribunal which does not exercise some of the important judicial functions of the State.

Considerable stress was laid by Mr. Alladi on the provisions of secs. 15 of the Act. Section 15 enacts as follows :-

"(1) Where an industrial dispute has been referred to a Tribunal for adjudication, it shall hold its proceedings expeditiously and shall, as soon as practicable on the conclusion thereof, submit its award to the appropriate Government.
(2) On receipt of such award, the appropriate Government shall by order in writing declare the award to be binding.
Provided that where the appropriate Government is a party to the dispute and in its opinion it would be inexpedient on public grounds to give effect to the whole or any part of the award, it shall on the first available opportunity lay the award together with the statement of its reasons for not making a declaration as aforesaid before the Legislative Assembly of the province, or where the appropriate Government is the Central Government, before the Central Legislature, and shall, as soon as may be, cause to be moved therein a resolution for the consideration of the award; and the Legislative Assembly or as the case may be, the Central Legislature, may by its resolution confirm, modify, or reject the award.

(3) On the passing of a resolution under the proviso to sub-section (2), unless the award is rejected thereby, the appropriate Government shall by order in writing declare the award as confirmed or modified by the resolution, as the case may, to be binding.

(4) Save as provided in the proviso to sub-section (3) of section 19, an award declared to be binding under this section shall not be called in question in any manner."

35. As regards clause (4), it was conceded rightly that a law dealing with industrial disputes and enacted in the year 1947 could not in any way affect the provisions of the Constitution laid down in Art. 136. It was, however, strenuously urged that the award of the Tribunal had no binding force by itself and unless the appropriate Government made a declaration in writing under clause (2) of section 15, this award was a lifeless document and had no sanction behind it and therefore it could not have been contemplated that it would be appealable even by special leave. In my opinion, this contention is unsound. The provisions of clause (2) of section 15 leave no discretion in the Government either to affirm, modify or reject the award. It is bound to declare it binding. It has no option in the matter. In such a situation it is the determination by the Tribunal that matters. Without that determination Government cannot function. It does not possess the power either to adjudicate the dispute or to alter it in any manner whatsoever. That power vests in the Tribunal alone. The rights of the parties are really affected by the adjudication contained in the award, not by the Government's declaration which is automatic. It is no doubt true that announcement of the award by the Government gives it binding force but that does not affect the question of the appealability of the determination under Art. 136 of the Constitution. The apposite answer to this contention may be given in the language of the decision in Rex v. Electricity Commissioners. The relevant passage runs thus :-

"It is necessary, however, to deal with what I think was the main objection of the Attorney-General. Is this case he said the Commissioners come to no decision at all. They act merely as advisers. They recommend an order embodying a scheme to the Minister of Transport who may confirm it with or without modifications. Similarly the Minister of Transport comes to no decision. He submits the order to the Houses of Parliament, who may approve it with or without modifications. The Houses of Parliament may put anything into the order they please, whether consistent with the Act of 1919, or not. Until they have approved, nothing is decided, and in truth the whole procedure, draft scheme, inquiry, order, confirmation, approval, is only part of a process by which Parliament is expressing its will, and at no stage is subject to any control by the Courts. It is unnecessary to emphasize the constitutional importance of the contention. Given its full effect, it means that the checks and safeguards which have been imposed by Act of Parliament, including the freedom from compulsory taking, can be removed, and new and onerous and inconsistent obligations imposed without an Act of Parliament, and by simple resolution of both Houses of Parliament. I do not find it necessary to determine whether, on the proper construction of the statute, resolutions of the two Houses of Parliament could have the effect claimed. In the provisions that the final decision of the Commissioners is not to be operative until it has been approved by the two Houses of Parliament. I find nothing inconsistent with the view that they act judicially and within the limits prescribed by the Act of Parliament, and that the Courts have power to keep them within those limits. It is to be noted that it is the order of the Commissioner that eventually takes effect, neither the Minister of Transport who confirms, nor the Houses of Parliament who approve, can under the statute make an order which in respect of the matters in question has any operation. I know of no authority which compels me to hold that a proceeding cannot be a judicial proceeding subject to confirmation or approval, even where the approval has to be that of the Houses of Parliament. The authorities are to the contrary."
36. The observations, though they relate to a case which concerns the issue of a writ of prohibition and certiorari have application to the present case. Here no discretion whatsoever has been left in the Government in ordinary cases to either modify or to reject the determination of the tribunal. The fact that the Government has to make a declaration after the final decision of the tribunal is not in any was inconsistent with the view that the tribunal acts judicially. It may also be pointed out that within the statute itself a clue has been provided which shows that the circumstances that the award has to be declared by an order of Government to be binding does not affect the question of its applicability. In Art. 136, clause (2) express provision has been made for excepting from the ambit of Art. 136 the decisions of military Courts and Tribunals. It follows that but for the exception it was considered that these would be within Art. 136 clause (1). It is quite clear from the various provisions of the Army Act that the decisions of military tribunals or Courts are subject to confirmation either by the Commander-in-Chief or various other military authorities. It is only after such confirmation that can operate. It has never been considered that that fact in any way affects the question of their applicability.

37. Rex v. Minister of Health, also supports this view. There by the Housing Act, 1925, by section 40, a local authority which had prepared an improvement scheme was required to present a petition to the Minister praying that an order should be made confirming such scheme. Sub-section (3) provided that the Minister after considering the petition may cause a local inquiry to be made and may by order confirm the scheme with or without conditions or modifications. In sub-section (5), it was stated that the order of the Minister when made shall have effect as if enacted in this Act. It was held by the Court of Appeal that as the order made by the Minister was made without the statutory conditions having been complied with, it was ultra vires and, therefore, a writ of certiorari should issue for the purpose of quashing it. Reliance was placed by Scrutton, L. J., on Rex v. Electricity Commissioners. The same view was expressed in Minister of Health v. The King. It was observed that judicial review by prohibition or a writ of certiorari was permissible if the Minister of Health in confirming the order exceeded his statutory powers. It is clear, therefore, that simply because an order has to be confirmed by a Minister or by the Government it in any way affects the power of judicial review. Reference may also be made to the observations in Smith v. The Queen. At page 623 is was observed that it is a common principle in every case which was in itself the character of a judicial proceeding that the party against whom a judgment is to operate shall have an opportunity of being heard. In this sense it can hardly be disputed that the proceeding before an industrial tribunal is a judicial proceedings. In my judgment, therefore, the contention raised by Mr. Alladi that this Court cannot exercise its powers under Art. 136 because the decision of the tribunal has no force till a declaration is made by the Government cannot be sustained.

38. As regards section 19, it was contended that an award declared by the appropriate Government under section 15 to be binding can only come into operation on such date as may be specified by the appropriate Government and can only remain in operation for such period not exceeding one year, as may be fixed by that Government and it was said that herein the Government had the power to state the period from which the award was to commence and the time for which it was to remain in force. This section does not, in my opinion, affect the question of the appealability of the determination of the tribunal. Government has certain functions to perform in its own sphere after the award is made. In certain cases it is itself a party to the dispute, it has certain overriding powers and these overriding powers are that if it considers that the award is not in public interests it may refer it to the legislature. The legislature, however, has the power to modify, accept or reject the award. These overriding powers presuppose the existence of a valid determination by a tribunal. If that determination is in excess of jurisdiction or otherwise proceeds in a manner that offends against the rules of natural justice and is set aside by exercise of power under Art. 136, then no occasion arises for exercise of governmental power under the Act. Given a valid award, it could not be denied that the Government could exercise its powers in any manner it considered best and the exercise of that power is outside the Constitution of this Court. In this connection reference was made to Moses v. Parker. The passage on which emphasis was laid reads as follows :-

"The Court has been substituted for the commissioners to report to the governor. The difference is that their report is to be binding on him. Probably it was thought that the status and training of the judges made them the most proper depositaries of that power. But that does not make their action a judicial action in the sense that it can be tested and altered by appeal. It is no more judicial than was the action of the commissioners and the governor. The Court is to be guided by equity and good conscience and the best evidence. So were the commissioners. So every public officer ought to be. But they are expressly exonerated from all rules of law and equity, and all legal forms. How then can the propriety of their decision be tested on appeal ? What are the cannons by which this Board is to be guided in advising. Her Majesty whether the Supreme Court is right or wrong ? It seems almost impossible that decisions can be varied except by reference to some rule; whereas the Court making them is free from rules. If appeals were allowed, the certain result would be to establish some system of rules, and that is the very thing from which the Tesmanian Legislature has desired to leave the Supreme Court free and unfettered in each case. If it were clear that appeals ought to be allowed such difficulties would doubtless be met somehow. But there are strong arguments to show that the matter is not of an appreciable nature."
39. One would have expected that after this opinion the decision would have been that the Judicial Committee had no jurisdiction to entertain the appeal but their Lordships proceeded to base their decision not on this ground but on the ground that this was not a fit case for the exercise of the prerogative of the King. In my opinion, the observations made in that case have no apposite application to the provisions of the statute with which we are concerned. I do not see any difficulty in this case in testing the propriety of the determination of the tribunal. This Court is not to substitute its decision for the determination of the tribunal when granting relief under Art. 136. When it chooses to interfere in the exercise of these extraordinary powers, it does so because the tribunal has either exceeded its jurisdiction or has approached the questions referred to it in a manner which is likely to result in injustice or has adopted a procedure which runs counter to the well-established rules of natural justice. In other words, if it has denied a hearing to a party or has refused to record his evidence or has acted in any other manner, in an arbitrary or despotic fashion - in such circumstances no question arises of this Court constituting itself into a tribunal and assuming powers of settling a dispute. All that the Court when it entertains an appeal would do is to quash the award and direct the tribunal to proceed within the powers conferred on it and to approach the adjudication of the dispute according to principles of natural justice. This Court under Art. 136 would not constitute itself into a mere Court of error. Extraordinary powers have to be exercised in rare and exceptional cases and on well-known principles. Considered in the light of these principles, there is no insuperable difficulty in the present case of the nature pointed out in the passage cited above. It was conceded that the High Court could exercise powers under section 226 and could quash an award but it was said that under Art. 136 this power should not be exercised in an appeal. I do not see why, particularly when after the High Court has passed any decision on an application made to it in exercise of the powers under section 226, that decision could be brought to this Court in appeal. In the matter of an industrial dispute where expedition is the crux of the matter, it is essential that any abuse of powers by such tribunals is corrected as soon as possible and with expedition.

40. It may be mentioned that it is no novel practice for a Court empowered to grant special leave to exercise its powers even though there may be intermediate rights of appeal or other remedies available, if it is considered to do so in extraordinary situations. Vide Bentwick's Privy Council Practice, 3rd Edition, page 125. Therein it is stated as follows :-

"In several cases from Jamaica, the Privy Council granted leave to appeal to the Queen in Council directly from the Supreme Court, without an intermediate appeal (which would have been attended with much expense and delay) to the Court of Error in the island, there being in each of those cases manifestly some point of law raised which deserved discussion."
41. The cases were In re Barnett, Harrison v. Scott and Attorney-General of Jamaica v. Manderson. The phraseology employed in Art. 136 itself justifies this course. The article empowers this Court to grant special leave against sentences or orders made by this Court. In all other articles of the Constitution right of appeal is conferred against final decisions of the highest Court of Appeal in the country but under this article prayer is given to this Court to circumvent that procedure if it is considered necessary to do. I am, therefore, of the opinion that the mere circumstances that a remedy in the nature of a writ of certiorari is open to the petitioners does not necessarily lead to the conclusion that the power of this Court under Art. 136 is circumscribed by that circumstances. Whenever judicial review is permissible in one form or another, this Court as the highest Court in the land can exercise its special powers and circumvent ordinary procedure by granting special leave. What it has to ultimately decide, it can decide earlier.

42. I now proceed to examine some of the cases to which reference was made by Mr. Alladi.

43. Three Australian cases were cited which concern the construction of secs. 51, 71 and 72 of the Australian Constitution (63 and 64 Vict. c. 12), Section 72 requires that every justice of the High Court and every justice of any other Court created by the Parliament of the Commonwealth shall, subject to the power of removal contained in the section, be appointed for life. Section 71 confers the whole judicial power of the Commonwealth upon the Courts therein mentioned and no other tribunal or body can exercise that power. Every Court referred to in section 71 has to be constituted in the manner provided by section 72. The question in these cases was as to the meaning of the phrase "judicial power of the Commonwealth." Similar phraseology has not been used in any part of the Constitution of India and these circumstances it is difficult to derive any assistance from these decisions in solving the problem before us. The Constitution of India is not modelled on the Constitution of Australia and that being so, any observation made in decisions given under that Constitution cannot be held to be a safe guide in the interpretation of languages employed in a Constitution differently drafted.

44. The first of these cases is Waterside Worker's Federation of Australia v. J. W. Alexander Ltd. Therein it was held that the power conferred by the Commonwealth Conciliation and Arbitration Act, 1904-1915, upon the Commonwealth Court of Conciliation and Arbitration to enforce awards made by it is part of "the judicial power of the Commonwealth" within the meaning of section 71 of the Constitution, and can only be vested in the Courts mentioned in that section. Mr. Alladi placed reliance on a passage at page 467 in the judgment of Isaacs and Rich, JJ., which reads as follows :-

"The arbitral part of the Act, therefore, is quite within the power of pl. xxxv., and it not intended by the Act to be exercised by an ordinary Court of Justice, which, it is suggested, Parliament by some strange perversity proceeded to destroy at birth. It is true that enforcement provisions are found......... But all this was in imitation of the State Acts of Arbitration, and not in reliance on the Judicature Chapter of the Federal Constitution. The arbitral portion of the Act is, in our opinion, perfectly good, subject to its severability from any other portion which may be bad."
45. It was argued that the Industrial Tribunal here was an arbitration of the same kind as in Australia and exercises similar functions. It is, however, pertinent to observe that the phraseology employed in section 15 of the Indian Act is different from that used in the Australian statute. The Indian statute has constituted different bodies for different purposes. An Industrial Tribunal has been constituted only to discharge one function of adjudication. It is not described as an arbitral Tribunals. The Act has avoided the use of the word "arbitration" either in is preamble or in any of its relevant provisions though the determination has been named as an award. In these circumstances it is unsafe to seek any guidance from observations made in this case.

46. The next case to which reference was made is Rola Co. (Australia) Proprietary, Ltd. v. The Commonwealth. The question here was whether the Women's Employment Board constituted under the Women's Employment Act, 1942, did not exercise the judicial power of the Commonwealth. It was held that the Board exercised functions which were arbitral in character. Emphasis was laid on a passage occurring in page 198 of the report which reads as follows :-

"An industrial award lays down rules of conduct for the future. It does not purport to ascertain and enforce existing rights, it is directed to the creation of new rights. It is urged on behalf of the Plaintiff that a determination of the Committee does not create a rule of conduct binding the parties for the future, but that it authoritatively determines a possibly controverted question of fact and that the making of such an authoritative determination is necessarily an exercise of judicial power. Reference is made to the frequently quoted statement of Griffith, C.J., in Huddart Parker & Co. Pty., Ltd. v. Moorehead, approved by the Privy Council in Shell Co. of Australia, Ltd. v. Federal Commissioner of Taxation -
"I am of opinion that the words 'judicial power' as used in section 71 of the Constitution mean the power which every sovereign authority must of necessity have to decide controversies between its subjects, or between itself and its subjects, whether the rights relate to life, liberty or property. The exercise of this power does not begin until some tribunal which has power to give a binding and authoritative decision (whether subject to appeal or not) is called upon to take action."
47. Reg. 5C gives Committees power to decide controversies between subjects relating to their rights and the regulation purports to make those decisions binding and authoritative.

48. I am not satisfied that the words of Griffith, C.J., are properly interpreted when it is said that they mean that a power to make binding and authoritative decisions as to facts is necessarily judicial power. I direct attention to the concluding words - "is called upon to take action." In any opinion these words are directed to action to be taken by a Tribunal which has power to give a binding and authoritative decision. The mere giving of the decision is not the action to which the learned Chief Justice referred. If a body which has power to give a binding and authoritative decision is able to take action so as to enforce that decision, then, but only then, according to the definition quoted, all the attributes of judicial power are plainly present. I refer to what I say more in detail hereafter, that the Privy Council, in the Shell case in which approval was given to the definition quoted, expressly held that a Tribunal was not necessarily a Court because it gave decisions (even final decisions) between contending parties which affected their rights.

49. In Huddart Parker's case, Isaacs, J., referred to the statement of Palles, C. B., in R. v. Local Government Board for Ireland "to erect a Tribunal into a "Court" or "jurisdiction," so as to make its determinations judicial, the essential element is that it should have powers by its determination within jurisdiction, to impose liability or affects right."By this," said the learned Chief Baron, "I mean that the liability is imposed, or the right affected by the determination only, and not by the fact determined, and so that the liability will exist, or the right will be affected, although the determination be wrong in law or in fact. It is otherwise of a ministerial power. If the existence of such a power depends upon a contingency, although it may be necessary for the officer to determine whether the contingency has happened, in order to know whether he shall exercise the power, his determination does not bind. The happening of the contingency may be questioned in an action brought to try the legality of the act done under the alleged exercise of the power. But where the determination binds, although it is based on an erroneous view of facts or law, then the power authorizing it is judicial. There we get a modern use of the term "judicial power." This statement of the characteristics of judicial power looks to what, in Waterside Worker's Federation of Australia v. Gilchrist, Watt & Sanderson, Ltd., Isaacs and Rich, JJ., referred to as the creation of instant liability in specified persons as distinct from laying down a rule or standard of conduct for the future.

50. The decision of an ordinary Court that B is bound to pay money to A applies a pre-existing standard of rights and duties not created by the Court itself, with the result that there is an immediately enforceable liability of B to pay to A the sum of money in question. The decision of the Women's Employment Board does not create any such liability, nor does the determination of a Committee of Reference create any such liability, for example, to pay wages to a particular female, it would be necessary for the female or some person on her behalf (see Reg. 9A) to sue in a Court of competent jurisdiction. If such a proceeding succeeded there would then be a liability created by the determination of the Court. In such a proceeding the determination of the Committee of Reference would be evidence of the facts to which it related, but that determination would not in itself create liability." The concluding words of the passage quoted above at once distinguish the present case from the Australian case. The award given by the Industrial Tribunal in respect either of bonus or higher wages, etc. is enforceable by its own force and by the coercive machinery of the Act and it is not merely a declaration of a character that furnishes a cause of action to the employee to bring a suit on its foot to recover the wages. An arbitral Tribunal's decision cannot be enforced unless it has the sanction of the Court of justice behind it but the award of the tribunal is enforceable under the Act itself by the coercive machinery provided therein. It is the terms of the award that are enforceable and not the terms of the order made by the Government. It is the breach of the terms of the award that is punishable and not any breach of Government's order. The Government itself is bound to declare the award binding and it has no option whatsoever in the matter. It is no doubt true that the tribunal has not only to decide the existing rights and liabilities of the parties and it can lay down rules of conduct for the future but it does so because by law it is authorised to do so. Its decision carries the sanction with it. The Government is bound to give effect to it and the statute enforces it by coercive machinery. In may view, therefore, this decision again has no relevancy to the present case.

51. The third case to which reference was made is Shell Co. of Australia v. Federal Commissioner of Taxation. That was an Income-tax matter and the decision has been considered in an earlier part of this judgment. Reference was also made to Mohammad Ahmed v. Governal-General in Council in which it was held that an improvement trust was not a Civil Court subordinate to the High Court under section 115 of the Code of Civil Procedure. That has no bearing on the matter in issue here. Similar point was discussed in Hari v. Secretary of State for India. Labour Relations Board v. John East Iron Works Ltd. is a Canadian case and the decision proceeded on the same lines as in the Australian cases.

52. Mr. Sen appearing for the respondents placed reliance on O'Connor v. Waldron. The relevant passage occurs at page 81 which runs thus :-

53. Mr. Sen appearing for the respondents placed reliance on O'Conloped. Originally it was intended for the protection of Judges sitting in recognised Courts of justice established as such. The object no doubt was that Judge might exercise their functions free from any danger that they might be called to account for any words spoken as Judges. The doctrine has been extended to Tribunals exercising functions equivalent to those of an established Court of justice. In their Lordships' opinion the law on the subject was accurately stated by Lord Esher in Royal Aquarium, etc., Ltd. v. Parkinson, where he says that the privilege 'applies wherever there is an authorised inquiry which, though not before a Court of Justice, is before a Tribunal which has similar attributes...... This doctrine has never been extended further than to Courts of Justice and Tribunals acting in a manner similar to that in which such Courts act.'"

54. The learned Counsel contended that the word "Tribunal" in Art. 136 could only have reference to those Tribunals which exercise functions equivalent to that of a Court of Justice. I have no hesitation in holding that the Industrial Tribunal has similar attributes as that of a Court of Justice in view of the various provisions to which I have made reference. Reference was also made to certain passage occurring in pages 422 and 428 of Toronto Corporation v. York Corporation. That was a case of the Municipal Board of Ontario. It was held there that the Board was merely an administrative Tribunal. Next reliance was placed on R. v. National Arbitration Tribunal, Ex parte Horatio Crowther & Co., Ltd. That dealt with the powers of the National Arbitration Tribunal. In my opinion this citation also is not of much assistance.

55. It was again urged by Mr. Alladi that the word "Tribunal" was introduced in the article to provide for cases of Tribunals like the Board of Revenue. The suggestion does not appear to be sound because a Revenue Board has all the attributes of a Court of Justice and falls within the definition of the ward "Court" in matters where it adjudicates on rights of parties.

56. The word "Tribunal" has been used in previous legislation in a number of statutes and it is difficult to think that the Constitution when it introduced this word in Art. 136 intended to limit its meaning to only those Tribunals which though not described as Courts strictly speaking, were discharging the same or analogous functions as were being discharged by Courts.

57. For the reasons given above I am of the opinion that the word "tribunal" in Art. 136 has to be construed liberally and not in any narrow sense and an industrial inasmuch as it discharges functions of a judicial nature in accordance with law comes within the ambit of the article and from its determination an application for special leave is competent.

58. The question now to determine is whether the exercise of overriding powers of this Court can be justified on any ground whatsoever in the present. As I have already said, exceptional and extra-ordinary powers of this character can only be justifiably used where there has been a grave miscarriage of justice or where the procedure adopted by the Tribunal is such that it offends against all notions of legal procedure.

59. Dr. Bakshi Tek Chand for the petitioner-bank urged four grounds justifying exercise of true special jurisdiction of this Court. Firstly, he contended that the word "victimization" used in clause of the reference has been interpreted in such a manner by the Tribunal that it has usurped jurisdiction to decide disputes which were never referred to it. In my view this is not a matter which can justify the exercise of the powers under Art. 136. This Court is not a mere Court of error. The word "victimization" has not been defined in the statute and is not in any sense a term of law or a term of art. It is an ordinary English word which means that a certain person has become a victim, in other words, that he has been unjustly dealt with. It was argued that the word has acquired a special meaning in regard to industrial disputes and connotes a person who becomes a victim of the employer's wrath by reason of his trade union activities and that the word cannot relate to a person who has been merely unjustly dismissed. Be that as it may. The determination of the Tribunal has not been materially affected by this interpretation of the word to any large extent and that being so, it does not call for the exercise of the special power.

60. The second ground urged was that the tribunal has erred in ordering reinstatement of persons who were guilty of an illegal strike. It was contended that section 23(b) of the Act has been wrongly construed by it and as a result of this misconstruction persons who were guilty of a wrong and who could not have been reinstated have been reinstated. In brief, the argument was that under section 23(b) when a matter has been referred to a tribunal in respect of an earlier strike, any strike during the pendency of that dispute is an illegal strike and that was the situation here. The employees of the bank had struck work in December, 1948. That dispute had been referred to an industrial tribunal. It has during the pendency of that dispute that another strike took place which led to the dismissal of the employees who have now been reinstated by the present award. The Calcutta High Court has held that a strike during the pendency of the period of truce and during the pendency of an earlier dispute before a tribunal is illegal even if it is brought about as a result of fresh and new demands which are not covered by the earlier dispute. One of the members of the tribunal thought that the decision laid down the law correctly on the point, but the other member thought that the decision was erroneous. Both of them, however, agreed that whether the strike was legal or illegal that point did not in any way affect the question that they had to decide under issue 18. The consequences of an illegal strike are laid down in the Act and certain penalties are provided therein. The Act nowhere states that persons guilty of illegal strike cannot be reinstated. Be that as it may. The reference to the tribunal was made by the Government in respect of an illegal strike and the Tribunal was bound to give its decision on the reference. Item 18 of Schedule 2 clearly empowers the tribunal to deal with cases of victimization as a result of the third strike which the petitioner described as illegal. The tribunal may be right or may be wrong in the view they have taken but it seems to me this is again not a question of that vital character which would justify the grant of special leave under Article 136.

61. The next question raised by the learned Counsel was that the award of the tribunal is based on no evidence whatsoever. This contention requires serious consideration. I have examined the proceedings of the tribunal and it appears that all it did was that as required by rule 17 at the first sitting it called upon the parties to state their cases. Mr. Parwana on behalf of the employees stated their respective cases and Mr. Ved Vyas who represented the bank stated the bank's case and after the cases have been stated the proceedings terminated and both parties addressed arguments and the tribunal proceeded to give its award. Whether the charge of victimization in individual cases was proved or not depended on proof of certain facts which has to be established by evidence. The onus or proving victimization clearly rested on the employees. No evidence whatsoever was led on their behalf. The statement of the case by Mr. Parwana was not on oath. There was no examination or cross-examination of Mr. Parwana. No affidavit supporting the facts stated by Mr. Parwana was filed by him or by any employee. Mr. Parwana produced an abstract of the correspondence but the original correspondence was not produced. The bank disputed the facts stated by Mr. Parwana by means of a lengthy affidavit. It seems no reference was made ever to this affidavit by the tribunal. No counter affidavit was filed in reply to the facts stated in this affidavit. The bank wanted to call some evidence. Particular reference was made in respect of a scurrilous letter issued by one Bhattacharya on behalf of the employees and distributed by them, which, it is alleged, considerably shook the credit of the bank. This opportunity was denied to it. It was contended before us that the bank wanted to lead evidence on certain matters and that the opportunity to lead it was denied. There is nothing on the record to support this contention. The result, therefore, is that the facts on the basis of which allegations of victimization have been made are neither supported by an affidavit nor by any evidence and the award is based on no evidence whatsoever. The Act as well as the rules framed under it contemplate a proper hearing, discovery and inspection of documents and production of evidence, etc. None of this procedure was followed by the tribunal. It is difficult to set on what material the tribunal has given its award as there is none existing on the present record and the Respondents' Counsel could not point out to any such material. At one time during the argument I was inclined to think that possibly both parties by agreement consented to treat the statement of case as evidence in the case and did not wish to produce any other evidence, but the affidavit filed on behalf of the bank disputes all the facts stated by Mr. Parwana. The only evidence on the record is the bank's affidavit and if the facts contained in the affidavit are accepted, then the determination made by the tribunal cannot stand. It seems to me, therefore, that the procedure adopted by the tribunal was against all principles of natural justice and the award is thereby vitiated and should be set aside. It happens that when the safeguard of an appeal is not provided by law the tendency sometimes is to act in an arbitrary manner like a benevolent despot. Benevolent despotism, however, is foreign to a democratic Constitution. The members of the tribunal seems to have thought that having heard the statement of the cases of the parties they could proceed to a judgment on their own view of its right or wrong unaided by any material. That kind of procedure to my mind is unwarranted by the statute and is foreign to a democratic Constitution. In these circumstances it is the compelling duty of this Court to exercise its extraordinary powers and to quash such an award.

62. The last contention raised by Bakshi Tek Chand was that though a tribunal consisting of three persons was appointed to adjudicate on the dispute, the award has only been signed by two of them. Reference in this connection was made to section 16 of the Act which says that the award of a tribunal shall be in writing and shall be signed by all the members of the tribunal and that nothing in the section shall be deemed to prevent any member of the tribunal from recording a minute of dissent. The provisions of the section are mandatory and have not been complied with. It is common ground that the case was stated by the parties at a sitting when all the members of the tribunal were present and the arguments were heard by all of them. No sitting took place subsequent to this which would have necessitated the carrying on of proceedings by two members of the tribunal by a quorum. When the matter has been heard by all the three members, the award should have been given by all of them. Therefore the award given by two of them is not the award of the tribunal constituted by the Government. It is, therefore, vitiated and has to be quashed. Reference in this connection was made to section 8 of the Act which reads as follows :-

"If the services of the Chairman of the Board or of the Chairman or other member of a Court or Tribunal cease to be available at any time, the appropriate Government shall, in the case of a Chairman, and may in the case of any other member, appoint another independent person to fill the vacancy, and the proceedings shall be continued before the Board, Court or Tribunal so reconstituted."
63. The tribunal was never reconstituted by the Government by any notification. Under section 7 a tribunal has to be constituted in accordance with the provisions of the Act by the Government. The Government having constituted a tribunal of three persons it had power under section 8 to reconstitute it but it did not exercise that power. The result, therefore, is that the tribunal as originally constituted was not the tribunal which gave the award in this reference. Only two members have given the award. It was said the one of the members ceased to be available and the Government was not bound to fill up that vacancy. There is no material on the record to prove whether any member became unavailable and if so, when. But even if a member becomes unavailable and the Government does not choose to fill up the vacancy, still the Government has to reconstitute the tribunal by saying that two members will now constitute the tribunal. An affidavit with two telegrams annexed was filed before us on behalf of the Respondents which disclosed that Mr. Chandrasekhara Aiyar who was one of the members of the tribunal, in November, 1949, was appointed a member of the Boundary commission in Bengal and that the other two members sent a telegram to the Labour Ministry asking it to fill up the vacancy or to reconstitute the tribunal. The advice given by the Ministry was that they could proceed as they were and the Government would later on, if necessary, fill up the vacancy. We are not concerned whether the advice given was right or wrong. But the fact remains that the tribunal was never reconstituted and it was not denied that Mr. Chandrasekhara Aiyar is now sitting in the same tribunal without being again nominated to it and the tribunal is hearing the same reference under the other issues referred to it. Moreover, I do not see why after having heard the reference he could not give the award even if he was in Calcutta on sign the award given by the other two members. The idea of three persons hearing a case and two of them deciding it is repugnant to all notions of fairness. It may well have been that the opinion of the third may have influenced the other two or the decision arrived at may have been quite different. It so happened in this case that two members of the tribunal differed on an important question of law but somehow adjusted their difference and gave a unanimous award. The presence of the third is such a situation may have very vitally affected the result. After a good deal of thought I feel that it would be most dangerous for this Court to condone proceedings of this character. If exceptional powers are not exercised even when a body legally constituted under the statute does not function according to the statute, then they defeat the very purpose of the Constitution.

64. Reference in this connection may be made to the decision of their Lordships of the Privy Council in Fakira v. King-Emperor. In that case section 377 of the Code of Criminal Procedure as modified and as applicable to Hyderabad stood as follows :-

"In every case so submitted, the confirmation of the sentence or order passed by the Court of the Resident at Hyderabad shall, when such court consists of two or more Judges, be made, passed and signed by at least two of them."
65. In Fakira's case the order of confirmation was only made, passed and signed by one of them, though the Court of the Resident consisted of two Judges. Their Lordships held that the peremptory provisions of section 377 had not been complied with and that the sentence passed had not been validly confirmed. The appeal was allowed and the case was remitted to the Court of the Resident. The provisions of section 18 of the Industrial Disputes Act are also of the peremptory nature. Reference may also be made to a case arising under the Bar Councils Act reported in In re : an Advocate, Madras, where one member of the tribunal under that Act had died and had not signed the report. It was held that the tribunal ceased to be properly constituted and that the report could not be considered.

66. For the reasons given above I would quash this award and direct that the tribunal which is still functioning should readjudge item 18 of the reference and then submit its award on this point to Government. The employees cannot be held responsible for the method of procedure adopted by two members of the tribunal. Each party will have to bear their own costs in this Court. The appeal is allowed to the extent indicated above.

Mukherjea, J.

67. This appeal, which has come up before us on special leave, is directed against an award made by the All-India Industrial Tribunal, dated the 19th of January, 1950. The Tribunal was constituted by the Central Government under section 7 of the Industrial Dispute Act and a large number of dispute between several Banking companies and their employees were referred to it for adjudication. amongst these Banking companies were the Bharat Bank Limited, the Appellants before us, and the disputed between them and their employees, who are Respondents in this appeal, related inter alia to a number of cases of retrenchment and victimization which the latter alleged against the former. The tribunal held its enquiry in Delhi in respect to the cases which were connected with the Delhi Branch of the Appellants and as a result of the same, made their award on 19th January, 1950, holding that 26 persons, who were employees under the Appellants, were improperly dismissed by the latter and should be reinstated. Further directions were given in the award regarding the salaries and allowances that were to be paid to the dismissed employees. This award was declared to be binding in terms of the provisions of secs. 15 and 19 of the Industrial Disputes Act by the Central Government on 30th of January, 1950, and it was directed to remain in operation for a period of one year. It is against this award that the present appeal has been preferred.

68. On behalf of the Indian Union which appeared as an intervener in this appeal, as also on behalf of the Respondents, a preliminary objection was taken challenging the competency of the appeal. The contention put forward by Sir Alladi Krishnaswami Aiyar, who appeared for the intervener, in substance, is that Art. 136 of the Indian Constitution, under which special leave was prayed for and obtained by the Appellants in this case, does not contemplate or include within its scope an appeal against an award of an Industrial Tribunal which is not vested with, and cannot exercise, judicial powers, and the decision of which cannot, therefore, rank as a judicial determination. The Industrial Tribunal, it is said, is an administrative body exercising quasi-judicial functions and this Court cannot be called upon to exercise the powers of an Appellate Court in respect to the decision of a Tribunal which is really a part of the administrative machinery of the Government.

69. In reply to this objection, it has been urged by Bakshi Sir Tek Chand that the Tribunal constituted under the Industrial Disputes Act is really and in substance, a Court or judicial tribunal which is invested with the power and authority to exercise judicial functions; and in any event, the language of Art. 136 of the Constitution is wide enough to include an appeal from the award or determination of any Tribunal, be it judicial or not.

70. There are two questions which require consideration on this preliminary point. The first is, whether the award or decision of an Industrial Tribunal constituted under the Industrial Disputes Act is a judicial decision in the proper sense of the expression or is the pronouncement of an administrative or quasi judicial body which may exercise some of the functions of a Court of law but is really not so ? The other question turns upon the construction to be put upon Art. 136 of the Constitution particularly on the meaning to be given to the words "tribunal" and "determination" occurring therein; and the question is whether the language is wide enough to include an adjudication or award of an Industrial Tribunal.

71. As regards the first question, it is to be noticed that owing to the intricate and complex system of Government that exists in a modern State and the vast expansion of social legislation of all sorts that have taken place in England and in other countries including our own, within the last few decades, the so-called administrative and quasi judicial Tribunals have come to be a permanent feature of out social and political system. They function as adjudicating bodies in disputed concerning a large number of economic and social affairs. In a sense they are governmental bodies appertaining to the executive and not to the judicial branch of the State, though in various matters they are armed with judicial powers analogous to those normally carried out by Courts of law. The question is, what are the tests or distinguishing features, if any, which distinguish an administrative Tribunal from a Court of law. Once we are able to formulate these tests, we would be in a position to determine whether a Tribunal functioning under the Industrial Disputes Act is or is not a judicial Tribunal properly so called.

72. Whether a particular function or activity is judicial or not is often a difficult question to decide. The point was elaborately dealt with by Lord Sankey who delivered the judgment of the Privy Council in Shell Co. of Australia v. Federal Commissioner of Taxation. The question raised in that case was whether the Board of Review, which was set up in 1925 under the Commonwealth Income Tax legislation, was a Court exercising Judicial powers by a majority that it was an administrative and not a judicial Tribunal and this majority judgment was affirmed in course of this judgment that "the decided cases show that there are Tribunals which possess many of the trappings of a court but which, nevertheless, are not courts in the strict sense of exercising judicial power. Mere externals do not make a direction by an ad hoc tribunal to an administrative officer, an exercise by a Court of judicial power."

73. The actual decision in the case rested on the ground that the Board of Review could not be a judicial tribunal, as its orders were not conclusive for any purpose whatsoever. The decision, is seems, has only a negative value. The Lord Chancellor enumerated a series of negative propositions which stated inter alia that a tribunal is not necessarily a Court because two or more contending parties appear before it, nor because it hears witness, or gives a final decision which affects the right of the parties. What the real or positive test is, the Privy Council did not care to formulate, though the judgment quoted, with approval, certain observations of Griffith, C. J., given in another Australian case, namely, Huddart Parker and Co. v. Moorehead which to some extent neutralised the effect of the negative tests enumerated in the judgment. The observations of Griffith, C.J., are as follows :-

"I am of opinion that the words 'judicial power'.... mean to power which every sovereign authority must have to necessity to decide controversies between its subjects, or between itself and its subjects, whether the rights relate to life, liberty or property. The exercise of this power does not begin until some tribunal which has power to give a binding and authoritative decision (whether subject to appeal or not) is called upon to take action."
74. It may be stated that the authority to hear and decide on evidence between a proposal and an opposition though it is one of the most essential of judicial powers, may be present in an administrative tribunal also. In the majority of cases, administrative bodies are also armed with the powers of a Court of justice in summoning witnesses, administering oaths and punishing disobedience to its order made for the purpose of effecting its enquiries. Vide W. F. O'Conner v. Waldron. As a matter of fact, it is usual to find that those features which were at one time attached exclusively to activities carries on in a Court of law are being extended to committees, commissions or boards conducting enquiries under directions or supervision of the Government. The presence or absence of these features, therefore, does not furnish any conclusive test to determine whether a particular body is a judicial body or not. In the observations of Griffith, C.J., quoted above, the learned Chief Justice laid stress on the power to make a binding and authoritative decision as the essential element in the exercise of judicial power. The exact meaning and implication of these expressions were the subject-matter of discussion in later Australian cases and it was held by the majority of the Judges in Rola Co. (Australia) Pty. Limited v. The Commonwealth that they do not simply mean that if an authority is given power to decide controverted questions of fact and its determination is made binding on the parties to the controversy, if would be sufficient to show that judicial power was entrusted to such authority. A determination, it was pointed out, may be binding on the parties in the same sense as contract is binding on them. What is necessary is that the determination by its own force and without the aid or instrumentality of any other authority or power must affect the rights and obligations of the parties; or in other words, the decision itself irrespective of the facts decided, must create rights and impose obligations; and it should be enforceable as such under the ordinary law of the land. This undoubtedly is one of the fundamental tests which distinguishes a judicial body from one which exercises administrative or quasi judicial functions. Sometimes the decision or report of the administrative tribunal becomes operative after it is accepted by the head of the department under which the tribunal conducted its enquiries and it is then enforced by some sort of administrative process; or it might create rights between the parties which have to be sued upon in the ordinary way in a Court of Law and it is only on the basis of a judgment or decree that is obtained in such action that relief could be had by the party. The essence of judicial determination is that nothing further remains to be done except the enforcement of the judgment, a step which is compelled automatically by the law of the land.

75. The other fundamental test which distinguishes a judicial from a quasi judicial or administrative body is that the former decides controversies according to law, while the latter is not bound strictly to follow the law for its decision. The investigation of facts on evidence adduced by the parties may be a common feature in both judicial and quasi judicial tribunals, but the difference between the two lies in the fact that in a judicial proceeding the judge has got to apply to the facts found, the law of the land which is fixed and uniform. The quasi judicial tribunal on the other hand gives its decision on the differences between the parties not in accordance with fixed rules of law but on principles or administrative policy or convenience or what appears to be just and proper in the circumstances of a particular case. In other words, the process employed by an administrative tribunal in coming to its decision is not what is known as "judicial process." (See Robson's Justice and Administrative Law, p. 74.) Sir Maurice Gwyer in his deposition before the Committee on Minister's Powers appointed by the English Parliament in 1929 stated that "a clear distinction is to be drawn between judicial and quasi judicial powers." The "judicial power" was defined by the witness as a power to decide a question of legal right in a dispute between parties involving either a finding of fact or the application of a fixed rule or principle of law or involving both."The quasi judicial power," he defined as meaning "the power of giving decisions on questions of differences of an administrative and not justiciable character which cannot be determined by reference to any fixed law or principle of law but are matters of administrative discretion and judgment." (Vide Committee of Minister's Powers, Minutes of Evd. Vol. II, pages 15-16 and also Robson's Justice and Administrative Law, p. 319). In Cooper v. Wilson, Scott, L. J., quoted with approval and adopted as the basis of his judgment the following passage from the report of the above committee :

"A true judicial decision presupposes an existing dispute between two or more parties, and then involves four requisites :- (1) The presentation (not necessarily orally) of their case by the parties to the dispute; (2) if the dispute between them is a question of fact, the ascertainment of the fact by means of evidence of adduced by the parties to the dispute and often with the assistance of argument by or on behalf of the parties on the evidence; (3) if the dispute between them is a question of law, the submission of legal argument by the parties to the dispute and often with the assistance of argument by a finding upon the facts in dispute and an application of the law of the land to the facts so found, including where required a ruling upon any disputed question of law. A quasi judicial decision equally presupposes an existing dispute between two or more parties and involves and but does not necessarily involve and never involves. The place of is in fact taken by administrative action, the character of which is determined by the Minister's free choice."
76. In our opinion these statement correctly bring out the distinction between a judicial tribunal and an administrative body which exercises quasi judicial functions. These being the essential features which distinguish the two classes of tribunals, we would have to ascertain with reference to the provisions of the Industrial Disputes Act, which class or category of tribunals an Industrial Tribunal comes under.

77. The object of the Industrial Disputes Act, as set out in the preamble, is "to make provisions for investigation and settlement of industrial disputes and for certain other purposes hereinafter appearing." The word "settlement" suggests the idea of establishing compromise between the interest of disputing parties.

78. There are three classes of authorities provided for by the Act who are entrusted with the powers and duties of investigation and settlement of industrial disputes. First of all, there are conciliation officer of Boards of Conciliation, whose duties mainly are to induce parties to come to a fair and amicable settlement amongst themselves. Secondly, there are Courts of Enquiry and though they are described as Courts, their duties end with investigation into the matters referred to them and submitting reports thereupon to the appropriate Government. Lastly, there are Industrial Tribunals, composed of independent persons who either are or had been Judges of a High Court or District Judges or are qualified for appointment as High Court Judges.

79. It will be seen from the description given above that the Board of Conciliation or Court of Enquiry Disputes Act could, on no account, be regarded as judicial tribunals. To enable them to investigate facts they are, however, armed with certain powers of compelling attendance of witnesses and production of documents, etc. These provisions are to be found in section 11 of the Act. The significant thing to note is, that there is no distinction made in this respect between Conciliation Boards and Courts of Enquiry on the one hand and Industrial Tribunals on the other. The came power are conferred on the three classes of authorities without any distinction whatsoever and sub-section (3) of the Section 11 further lays down that any enquiry or investigation by a Board, Court of Enquiry or Tribunal shall be deemed to be judicial proceeding within the meaning of secs. 193 and 228 of the Indian Penal Code. This means that proceedings before an Industrial Tribunal or for the matter of that before the other two bodies could be deemed to be judicial proceedings only for certain specified purposes. The express provision making the proceedings judicial proceedings for those purposes only emphasises that they are not judicial proceedings otherwise.

80. Under section 15(1), the Industrial Tribunal has got to submit its award to the appropriate Government and sub-section (2) lays down that on receipt of such an award, the appropriate Government shall by order in writing declare the award to be binding. A different provision has been made in regard to cases where the Government itself figures as a party to the dispute. In such case, if the Government considers it inexpedient on public grounds to give effect to the award either in whose or in part, it may, at the earliest opportunity, lay the award for consideration before the Provincial or Central Legislative Assembly as the case may be and the Legislative Assembly may by its resolution confirm, modify or reject the award. After the resolution is passed, the Government is to declare the award so confirmed or modified to be binding [see sub-section (3)]. Sub-section (4) of section 15 expressly lays down that an award declared to be binding under any two of the previous sub-sections shall not be called into question in any manner whatsoever. The Government is not merely to declare the award binding but under section 19(3), if has got to specify the date when the award would come into force and also to fix the period during which it would remain binding, and this period shall not exceed one year.

81. It will be seen, therefore, that there is nothing in the Industrial Disputes Act from which it could be inferred that the Industrial Tribunal really functions as Court exercising judicial functions. Regarding the trappings or the external indicia of a Court, its position is almost the same as that of the Board of Conciliation or court of Enquiry and Bakshi Sir Tek Chand concedes that the latter are not Judicial Tribunals at all. The powers of an Industrial Tribunal are certainly wider than those of the other bodies, but it has no power to make a final pronouncement which would proprio vigore be binding on, and create rights and obligations between the parties. It is for the appropriate Government to declare the award to be binding and the part which the Government plays in such matters is not a mechanical part merely, for the award can really become operative only when the date of its commencement and the period of its duration are fixed, and it is for the Government and Government alone to fix the same. With regard to other class of cases, where the Government itself is one of the parties to the dispute, the position is still worse. An award in such cases is always subject to the contingency of being rejected or modified by the legislature before whom it could be placed for consideration at the option of the Government. Where a contingency like this is attached to an award, it can never be regarded as a final or binding decision which is of the essence of a judicial proceeding. The fact that in cases of disputes between private employers and their workmen, the Government has to accept the award as it is, make no difference in principle. Possibly, this rule was made in consideration of the status and training of the people who constitute the Tribunal, but nevertheless the determination cannot acquire any authority or force, so long as the appropriate Government does not make the declaration and fix the time of its operation as mentioned above. In regard to the other class of awards, where the Government is one of the disputing parties, the award on the face of it is neither the final nor the authoritative pronouncement on the matter in dispute, and it is always in the powers of one of the disputing parties to subject it to further scrutiny at the hands of the legislature who can reject the whole award or effect such change in it as to considers proper. This shows the real nature of the Tribunal and it is not and could not be suggested that the Industrial Tribunal which exercises judicial functions when the dispute is only between private employers and their workmen, and it ceases to be such when the employer is the Government itself.

82. We would now examine the process by which an Industrial Tribunal comes to its decisions and I have no hesitation in holding that the process employed is not judicial process at all. In setting the disputes between the employers and the workmen, the function of the Tribunal is not confined to administration of justice in accordance with law. It can confer rights and privileges on either party which it considers reasonable and proper, through they may not be within the terms of any existing agreement. It is not merely to interpret or give effect to the contractual rights and obligations of the parties. It can create new rights and obligations between them which it considers essential for keeping industrial peace. An industrial dispute as has been said on many occasions is nothing but a trial of strength between the employers on the one hand and the workmen's organisation on the other and the Industrial Tribunal has got to arrive at some equitable arrangement for averting strikes and lock outs which impede production of goods and the industrial development of the country. The Tribunal is not bound by the rigid rules of law. The process it employs is rather an extended form of the process of collective bargaining and is more akin to administrative than to judicial function.

83. In describing the true position of an Industrial Tribunal in dealing with Labour disputes, this Court in Western India Automobile Association v. Industrial Tribunal, Bombay and others quoted with approval, a passage from Ludwig Teller's well known work on the subject, where the learned author observes that "industrial arbitration may involve the extension of an existing agreement or the making of new one or in general the creation of new obligation or modification of old ones, while commercial arbitration generally concerns itself with interpretation of existing obligations and disputes relating to existing agreement." The views expressed in these observations were adopted in its entirety by this Court. Out conclusion, therefore, is that an Industrial Tribunal formed under the Industrial Disputes Act is not a judicial tribunal and its determination is not a judicial determination in the proper sense of these expressions.

84. We now come to the other question as to whether an appeal could be taken to this Court against an award of an Industrial Tribunal by special leave under Article 136 of the Constitution. Article 136 is a part of Chapter IV of the Constitution which deals with the Union Judiciary. The different jurisdictions of the Supreme Court have been prescribed in a series of articles commencing from Article 131. Art. 131 defines the original jurisdiction of the Supreme Court. Art. 132 deals with its appellate powers in cases where substantial question of law as to the interpretation of the Constitution are involved. Article 133 contains the provision relating to appeals in civil cases from judgments, decrees and orders of the High Courts; and Article 134 makes provisions relating to criminal appeals. Article 135 lays down that the Supreme Court shall have jurisdiction and powers with respect to any matter not covered by Articles 133 and 134, if such jurisdiction and power could have been exercised by the Federal Court prior to the coming into force of the present Constitution. Then comes Article 136 which runs as follows :-

"(1) Notwithstanding anything in this chapter the Supreme Court may, in its discretion, grant special leave to appeal from any judgment, decree, determination, sentence or order in any cause or matter passed or made by any Court or tribunal in the territory of India."
85. The article is worded in the widest terms possible. It vests in the Supreme Court a plenary jurisdiction in the matter of entertaining and hearing appeals by granting of special leave against any kind of judgment, decree or order made by any Court or tribunal in any cause or matter and the powers could be exercised in spite of and overriding the specific provisions for appeal contained in the previous Articles. The controversy so far as the present case is concerned mainly centers round the interpretation to be put upon two words, namely, "determination" and "tribunal" used in the Article. Does the word "tribunal" mean a judicial tribunal only and is the expression "determination" restricted to what is known as "judicial determination"?

86. Sir Alladi's contention is that in interpreting these words we should follows the principle of ejusdem generis."Determination", he says, "must be taken to be judicial determination which is of the same nature as decree, judgment, order or sentence; and "tribunal" associated with the word "Court" could not but mean "judicial tribunal".

87. Bakshi Sir Tek Chand on the other hand lays stress on the fact that the word "determination" was not in the original draft Constitution, and it was subsequently added presumably with a view to widen the scope of Article 136 and include within it, the decisions of administrative and quasi judicial tribunals also. He points out that according to the definition given section 2(b) of the Industrial Disputes Act, "award" means a determination, either interim or final, of an industrial dispute by an Industrial Tribunal.

88. There is undoubtedly something to be said in favour of both these views. The difficulty, in our opinion, arises from the fact that neither of these terms "determination" or "tribunal" has a fixed or definite connotation in ordinary language. The word "determination" means and signifies the ending of a controversy or litigation by the decision of a Judge or Arbitrator. It cannot be said that it is restricted exclusively to proceedings in Court. Likewise, the dictionary meaning of the word "tribunal" is "Court of justice" or "seat of a Judge". By "Judge" we mean some authority by which contested matters are decided between rival parties. Here again, it is not possible to say that the expression is applicable only to a regular Court of law. If the tribunal is a full-fledged judicial tribunal, it is not disputed that its decisions would be proper subject-matter or appeal under Article 136 of the Constitution. The question is whether this Article includes within its scope the determinations of quasi judicial tribunal as well.

89. Our view is that ordinarily we should not put any restricted interpretation upon the plain words of an Article in the Constitution and thereby limit out powers of granting special leave for appeals, which the Constitution for best of reason did not choose to fetter or circumscribe in any way. At the same time, we must admit that some sort of restricted interpretation may be unavoidable in view of the context in which particular words appear; and certain restrictions may be implicit in the very purpose for which Article 136 has been framed. Article 136 empowers us in our discretion to hear appeals from pronouncements of all inferior Courts and tribunals. With regard to law Courts, no difficulty arises. As regards tribunals which are not Courts in the proper sense of the expression, it may not be proper, in our opinion, to lay down a hard and fast rule that no appeals could, on any account, be allowed against determinations of such tribunals. There are numerous varieties of these adjudicating bodies, whose structures vary greatly in character and composition and so do the powers and functions which they exercise. The best thing to do would be to examine each type of cases as it arises and if we find that with regard to determinations emanating from certain tribunals it is not possible for us to exercise fully and effectively the powers of an Appellate Court, such determinations must be held to lie outside the purview of Article 136 of the Constitution.

90. This disability in the matter of exercising our powers as an Appellate Court might arise from the fact that the rules and principles by which we ordinarily judge the soundness or otherwise of judicial decisions are not capable of being applied to the determinations of certain administrative tribunals. It might also arise from the fact that the law under which the tribunal functions prevents us from making any effective order which would be binding and operative of its own force without the intervention of some other power or authority; or there may be some kind of contingency attached to it. In our opinion, these difficulties do confront us in the entertaining or hearing of an appeal against the decision of an Industrial Tribunal. In the first place, as we have said above the determination of an Industrial Tribunal does not become complete and binding unless and until it is declared to be so by the appropriate Government. Till the Government makes such declaration, neither of the parties to the dispute can have any real reason for filing an appeal. An, appeal, if it lies, could be filed after the determination has been declared binding. But in such cases, is it the determination of the tribunal merely which is challenged by way of appeal or is it the determination by the tribunal to which has been super-added a declaration by the Government. The decision in the appeal would undoubtedly affect not merely the decision of the tribunal merely which is challenged by way of appeal of is it the a tribunal within the meaning of Article 136. Assuming again that the award is set aside and we substitute out own determination in place of the award given by the tribunal, will our award be enforceable by itself or will it require a declaration by the Government to make it binding ? If Government is itself a party to the dispute, will it be open to Government to place out decision for consideration by the Legislative Assembly ? And will the Legislative Assembly be competent to reject or modify our award ? These problems arise because under section 15 the award under the Act becomes binding only when the Government declares it to be so and if our judgment takes the place of the award of the tribunal, all the infirmities that attach to the award must necessarily attach to out judgment also.

91. The other difficulty is no less formidable. As said above, the tribunal is not bound to decide the disputes by application of the ordinary law of the land. A good deal depends upon questions of policy and public convenience. It is not possible for us to judge the propriety of the decision by the reference to some standard or fixed rules and we think that the very policy of the law prevents us from interfering with the discretion exercised by the tribunal.

92. Where the discretion is committed to any body or a tribunal exercising quasi judicial functions which are not fettered by ordinary rules of law, the tribunal should in the absence of any provision to the contrary be deemed to have the final authority in the exercise of that discretion. We cannot sit in appeal over their decision and substitute our own discretion for theirs. Questions, however, may and so arise where such quasi judicial body attempts to usurp jurisdiction which it does not possess. It may assume jurisdiction under a mistaken view of law or refuse to exercise jurisdiction properly by adoption of extraneous or irrelevant considerations; or there may be cases where in its proceedings the tribunal violates the principles of natural justice. In all such cases the most proper and adequate remedy would be by writs of certiorari or prohibition and the Court having authority may direct that the decision of the body or tribunal might be brought up to the quashed for lack of jurisdiction or for mistake apparent on the face of it; and if the proceedings had not terminated at that time, a writ prohibition may also be issued for preventing the tribunal from exceeding its jurisdiction. The issuing of such writs would not be an exercise of appellant powers which means the rehearing of the case and passing of such judgment which of the opinion of the Appellate Court the original tribunal should have made. The object of these writs is simply to keep the exercise of powers by these quasi judicial tribunal within the limits of jurisdiction assigned to them by law and to restrain them from acting in excess of their authority. These principles are well settled and require no elucidation, Rex. v. Electricity Commissioner and Board of Education v. Rice. Our conclusion, therefore, is that Article 136 of the Constitution does not contemplate a determination given by the Industrial Tribunal.

93. Even assuming for argument's sake that we have got jurisdiction under Article 136, the exercise of which would depend upon the circumstances of each case, in view of the reasons which we have set out above, this is not an appeal which, in our opinion, should be admitted even if we have the power to do so.

94. The result is that the preliminary objection succeeds and the appeal fails and is dismissed with costs.

Patanjali Sastri, J.

95. I entirely agree and have nothing to add.

96. Agents : Ganpat Rai for the Appellant; R. R. Biswas for the Respondents; P. A. Mehta for the Intervener (The Union of India)."""
summarizer = TextGraphSummarizer(summary_size=0.1)
summary = summarizer.summarize(text)
word_weights = summarizer.show_word_weightage(sent_tokenize(text))

print("Summary:\n", summary)
print("\nTF-IDF Word Weightage:\n", word_weights)


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Summary:
 75. Vol. 70. 43. 27. 57. 32. 96. 46. Reg. 90. 68. 88. 48. 78. Mukherjea, J. 10. 76. 84. 91. 40. 26. 42. 86. 89. 49. 24. 66. Patanjali Sastri, J. 64. 92. 65. 51. 77. 73. 37. 29. 45. 62. 79. 20. 22. 55. 59. 33. 81. 136? 136. 136. 136. 136. 136. 136. 136. 136. 136. 136. 136. 136. 136. 136. 16. 25. Fazl Ali, J. 58. 63. Arts. 19. 82. Art. Art. Art. Art. Art. Art. Art. Art. 136 of the Constitution. 136 of the Constitution. 136 clause (1). 34. By Art. By Art. By Art. 21. In Art. In Art. 136 is the word "determination."

TF-IDF Word Weightage:
 {'10': 1.2253501875137975, '10th': 0.37532763380745926, '11': 2.1615446238615412, '115': 0.2421752506600045, '12': 1.2262153020441149, '124': 0.49099753223668013, '125': 0.37035697069920376, '13': 1.4913471170356551, '131': 1.1184530216665391, '132': 1.3938147405203893, '133': 1.574864149270826, '134': 2.665166893744964, '135': 0

In [49]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, AttentionalAggregation
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import spacy
from transformers import BertModel, BertTokenizer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

class TextGraphSummarizer:
    def __init__(self, summary_size=0.3):
        print(f"Initializing TextGraphSummarizer on device: {device}")
        self.nlp = spacy.load("en_core_web_sm")
        self.summary_size = summary_size
        self.important_legal_terms = {"judgment": 10.0, "plaintiff": 8.0, "defendant": 8.0, "petition": 6.0}
        self.model = BertModel.from_pretrained("bert-base-uncased").to(device)
        self.tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
        self.pos_tags = ["ADJ", "NOUN", "VERB"]

    def preprocess_text(self, text):
        print("Running preprocess_text...")
        doc = self.nlp(text.lower())
        words = [token.text for token in doc if not token.is_stop]
        print(f"Preprocessed words (count): {len(words)}")
        return words, doc

    def get_embeddings(self, words):
        print("Running get_embeddings...")
        inputs = self.tokenizer(words, padding=True, truncation=True, return_tensors="pt").to(device)
        with torch.no_grad():
            embeddings = self.model(**inputs).last_hidden_state.mean(dim=1)
        return embeddings

    def build_graph(self, words, doc, embeddings):
        print("Running build_graph...")
        G = nx.Graph()

        # Calculate TF-IDF scores
        tfidf_vectorizer = TfidfVectorizer()
        tfidf_scores = tfidf_vectorizer.fit_transform([' '.join(words)])
        feature_names = tfidf_vectorizer.get_feature_names_out()
        tfidf_dict = {word: tfidf_scores[0, idx] for word, idx in zip(feature_names, range(tfidf_scores.shape[1]))}
        tfidf_dict_gpu = {word: torch.tensor(tfidf_score, dtype=torch.float32).to(device) for word, tfidf_score in tfidf_dict.items()}

        feature_size = 782  # Adjust based on your requirements
        for i, token in enumerate(doc):
            if not token.is_alpha or i >= len(embeddings):
                continue
            
            pos_tag_vector = torch.zeros(len(self.pos_tags), dtype=torch.float32).to(device)
            if token.pos_ in self.pos_tags:
                pos_tag_vector[self.pos_tags.index(token.pos_)] = 1.0

            entity_type = torch.zeros(10, dtype=torch.float32).to(device)
            if token.ent_type_:
                entity_type[hash(token.ent_type_) % 10] = 1.0

            tfidf_weight = tfidf_dict_gpu.get(token.text.lower(), torch.tensor(1.0, dtype=torch.float32).to(device))
            law_weight = torch.tensor(self.important_legal_terms.get(token.text.lower(), 1.0), dtype=torch.float32).to(device)
            combined_weight = tfidf_weight * law_weight

            node_features = torch.cat([embeddings[i], pos_tag_vector, entity_type, combined_weight.unsqueeze(0)])
            G.add_node(i, x=node_features)

        print(f"Total nodes created: {len(G.nodes)}")

        # Add edges
        window_size = 5
        print("Adding edges to graph...")
        for i in range(len(doc) - window_size):
            for j in range(i + 1, i + window_size):
                if i < len(doc) and j < len(doc):
                    G.add_edge(i, j, weight=torch.tensor(1.0, dtype=torch.float32).to(device))
        print(f"Total edges created: {len(G.edges)}")

        return G

    def build_pyg_data(self, G):
        num_nodes = len(G.nodes)
        node_features = torch.zeros(num_nodes, 782).to(device)  # Adjust based on your requirements

        for node in G.nodes:
            if 'x' in G.nodes[node]:
                node_features[node] = G.nodes[node]['x']
            else:
                node_features[node] = torch.zeros(782).to(device)

        edge_index = torch.tensor(list(G.edges)).t().contiguous()
        edge_attr = torch.ones(edge_index.size(1)).to(device)  # Placeholder for edge features
        
        return Data(x=node_features, edge_index=edge_index, edge_attr=edge_attr)

class GNNTextSummarizer(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNNTextSummarizer, self).__init__()
        
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.attention = AttentionalAggregation(nn.Linear(hidden_dim, 1))  # Use the new aggregator
        self.output_layer = nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr

        # Debugging information: Print shapes of the input features and edge index
        print("Input feature shape:", x.shape)
        print("Edge index shape:", edge_index.shape)

        # Check if all node features have the same dimension
        if x.shape[1] != self.input_dim:
            raise ValueError(f"Node features must have {self.input_dim} dimensions, but got {x.shape[1]}")

        # Graph Convolution layers
        x = self.conv1(x, edge_index)
        x = F.relu(x)

        x = self.conv2(x, edge_index)
        x = F.relu(x)

        # Global attention pooling (using AttentionalAggregation now)
        x = self.attention(x)
        
        # Final output layer
        x = self.output_layer(x)
        
        return x

# Standalone summarize_text function
def summarize_text(report):
    summarizer = TextGraphSummarizer(summary_size=0.8)
    print(f"Summary size set to: {summarizer.summary_size}")

    words, doc = summarizer.preprocess_text(report)
    embeddings = summarizer.get_embeddings(words)
    G = summarizer.build_graph(words, doc, embeddings)
    data = summarizer.build_pyg_data(G)
    data = data.to(device)

    input_dim = 782  # Ensure this matches the feature dimension
    model = GNNTextSummarizer(input_dim=input_dim, hidden_dim=64, output_dim=32).to(device)

    model.eval()
    with torch.no_grad():
        scores = model(data).cpu().numpy()

    # Debug print scores
    print("Scores:", scores.flatten())

    # Select indices based on the adjusted summary size
    num_selected = int(len(words) * summarizer.summary_size)
    selected_indices = np.argsort(scores.flatten())[-num_selected:]  # Get top indices

    # Create a more comprehensive summary
    selected_indices = sorted(selected_indices)  # Sort for preserving order
    summary = ' '.join([words[i] for i in selected_indices])  # Create summary from selected indices

    print("Summarization complete.")
    return summary

# Sample input
report = """Supreme Court of India
The Bharat Bank Ltd., Delhi vs The Employees Of The Bharat Bank Ltd., ... on 1 March, 1950
Equivalent citations: [1950]SUPPSCR317
Bench: H.J. Kania, Meher Chand Mahajan, Saiyad Fazal Ali
JUDGMENT
 

Kania, C.J.
 

1. I have read the judgments prepared by Messrs Fazl Ali, Mahajan and Mukherjea, JJ., in this case. As the views in those judgments in respect of the nature of the duties and functions of the Industrial Tribunal do not show agreement I consider it necessary to add a few words of my own.

2. In my opinion, the functions and duties of the Industrial Tribunal are very much like those of a body discharging judicial functions, although it is not a Court. The rules framed by the Tribunal required evidence to be taken and witnesses to be examined, cross-examined and re-examined. The Act constituting the Tribunal imposes penalties for incorrect statements made before the Tribunal. While the powers of the Industrial Tribunal in some respects are different from those of an ordinary Civil Court and it has jurisdiction and powers to give reliefs which a Civil Court administering the law of the land (for instance, ordering the re-instatement of a workman) does not possess in the discharge of its duties it is essentially working as a judicial body. The fact that it determination has to be followed by an order of the Government which makes the award binding, or that in cases where Government is a party the legislature is permitted to revise the decision, or that the Government is empowered to fix the period of the operation of the award do not, to my mind, alter the nature and character of the functions of the Tribunal. Having considered all the provision of the Act it seems to me clear that the Tribunal is discharging functions very near those of a Court, although it is not a Court in the technical sense of the word.

3. The next question is whether under Art. 136 the Court has jurisdiction to entertain an application for leave to appeal against the decision of such a body. It is not disputed that the Court has power to issue writs of certiorari of prohibition in respect of the work of the Tribunal. The only question is whether there is a right of appeal also. In my opinion the wording of Art. 136 is wide enough to give jurisdiction to the Court to entertain an application for leave to appeal, although it is obvious that having regard to the nature of the functions of the Tribunal, this Court will be very reluctant to entertain such an application.

4. As regards the merits, I do not think this is a case in which I would admit the appeal. The aggrieved parties may apply for redress by adopting other appropriate proceedings. The appeal, therefore should be dismissed with costs.

Fazl Ali, J.

5. The important question to be decided in this case is whether the present appeal lies at all to this Court. The question is not free from difficulty, but on the whole I am inclined to think that the appeal does lie. It is fully recognized that the scope of Art. 136 of the Constitution is very wide, but the significance of the language used in the section can be appreciated only by comparing it with the articles which precede it. Art. 132 deals with the Appellate Jurisdiction of the Supreme Court in cases involving a substantial question of law as to the interpretation in the Constitution, and the words used in that article are : "appeal........ from any judgment, decree or final order." Art. 133 deals with appeal in civil matters and the same words are used here also. Art. 134 deals with appeals in criminal matters, and the words used in it are; "appeal........ from any judgment, final order or sentence." In Art. 136, the words "judgment" and "decree," which are used in Arts. 132 and 133 are retained. Similarly, the words "judgment" and.........."sentence" occurring in Art. 134 are also retained. But the expression "final order" becomes "order," and, instead of the High Court, reference is made to "any Court." Certain other words are also used in the article which seem to me to have a special significance, these being "determination," "cause or matter" and "tribunal." It is obvious that these words greatly widen the scope of Art. 136. They show that an appeal will lie also from a determination or order of "any tribunal" in any cause or matter.

6. Can we then say that an industrial tribunal does not fall within the scope of Art. 136? If we go by a mere label, the answer must be in the affirmative. But we have to look further and see what are the main functions of the tribunal and how it proceeds to discharge those functions. This is necessary because I take it to be implied that before an appeal can lie to this court from a tribunal it must perform some kind of judicial function and partake to some extent of the character of a Court.

7. Now there can be no doubt that the industrial tribunal has, to use a well-known expression, "all the trappings of a Court" and performs functions which cannot but be regarded as judicial. This is evident from the rules by which the proceedings before the tribunal are regulated. It appears that the proceeding before it commences on an application which in many respects is in the nature of a plaint. It has the same powers as are vested in a Civil Court under the Code of Civil Procedure when trying a suit, in respect of discovery, inspection, granting adjournment, reception of evidence taken on affidavit, enforcing the attendance of witnesses, compelling the production of documents, issuing commissions, etc. It is to be deemed to be a Civil Court within the meaning of secs. 480 and 482 of the Criminal Procedure Code, 1898. It may admit and call for evidence at any stage of the proceeding and has the power to administer oaths. The parties appearing before it have the right of examination, cross- examination and re-examination and of addressing it after all evidence has been called. A party may also be represented by a legal practitioner with its permission.

8. The matter does not rest there. The main function of this tribunal is to adjudicate on industrial disputes which implies that there must be two or more parties before it with conflicting cases, and that it has also to arrive at a conclusion as to how the dispute is to be ended. prima facie, therefore, a tribunal like this cannot be excluded from the scope of Art. 136, but before any final conclusion can be expressed on the subject certain contentions which have been put forward on behalf of the Respondents have to be disposed of.

9. The first contention is that the industrial tribunal cannot be said to perform a judicial or quasi-judicial function, since it is not required to be guided by any recognised substantive law in deciding disputes which come before it. On the other hand, in deciding industrial disputes, it has to override contracts and create rights which are opposed to contractual rights. In these circumstances, it is said that the very questions which arose before the Privy Council in Moses v. Parker, Ex parte Moses (1) arise in this case, these questions being :-

(1) How can the propriety of the tribunals' decision be tested on appeal ? and (2) What are the canons by which the Appellate Court is to be guided in deciding the appeal ?
10. Their Lordships of the Privy Council undoubtedly felt that these were serious questions, but they had no hesitation in saying that "if it were clear that appeals ought to be allowed, such difficulties would doubtless be met somehow." This, in my opinion, is a sufficient answer to the difficulty raised. The tribunal has to adjudicate in accordance with the provisions of the Industrial Disputes Act. It may sometimes override contracts, but so can a Court which has to administer law according to the Bengal or Bihar Money-Lenders Act, Encumbered Estates Act, and other similar Acts. The tribunal has to observe the provisions of the special law which it has to administer through that law may be different from the law which an ordinary Court of justice administers. The Appellate Court, therefore, can at least see that the rules according to which it has to act and the provisions which are binding upon it are observed, and its powers are not exercised in an arbitrary or capricious manner.

11. The second contention, which is a more serious one, is that the adjudication of the tribunal has not all the attributes of a judicial decision, because the adjudication cannot bind the parties until it is declared to be binding by the Government under section 15 of the Industrial Disputes Act. It is said that the adjudication is really in the nature of an advice or report which is not effective until made so by the Government. It appears that a similar objection was raised in Rex v. Electricity Commissioners, London Electricity Joint Committee Co., Ex parte (2) for the purpose of deciding whether a writ of certiorari should be issued in the circumstances of the case but was disposed of in these words :-

"It is necessary, however, to deal with what I think was the main objection of the Attorney-General. In this case he said the commissioners come to no decision at all. They act merely as advisers. They recommend an order embodying a scheme to the Minister of Transport, who may confirm it with or without modifications. Similarly, the Minister of Transport comes to know decision. He submits the order to the Houses of Parliament, who may approve it with or without modifications. The Houses of Parliament may put anything into the order they please, whether consistent with the Act of 1919, or not. Until they have approved, nothing is decided, and in truth the whole procedure, draft scheme, inquiry, order, confirmation, approval, is only part of a process by which Parliament is expressing its will, and at no stage is subject to any control by the Courts. It is unnecessary to emphasize the constitutional importance of this contention................ In the provision that the final decision of the Commissioner is not to be operative until it has been approved by the two Houses of Parliament I find nothing inconsistent with the view that in arriving at that decision the Commissioner themselves are to act judicially and within the limits prescribed by Act of Parliament, and that the Courts have power to keep them within those limits. It is to be noted that it is the order of the Commissioner that eventually takes effect; neither the Minister of Transport who confirms, nor the Houses of Parliament who approve, can under the statute make an order which in respect of the matter in question has any operation. I known of no authority which compels me to hold that a proceeding cannot be a judicial proceeding, subject to prohibition or certiorari because it is subject to confirmation or approval, even where the approval has to be that of the Houses of Parliament. The authorities are to the contrary."
12. It is well-known that a writ of certiorari can issue only against an order of a judicial or quasi-judicial tribunal and if it is permissible for the High Court to issue a writ of certiorari against an industrial tribunal, which fact was not seriously disputed before us, I find is difficult to hold that the tribunal does not come within the purview of Art. 136. If a subordinate Court acts in excess of its jurisdiction or assumes a jurisdiction which it does not possess, the Appellate Court can always interfere and do what is contemplated to be done by writ of certiorari.

13. It is to be noted that under section 15 of the Industrial Disputes Act, 1947, in cases where the appropriate Government is not a party to the dispute, all that the Government has to do on receiving the award of the tribunal is to declare it to be binding and to state from what date and for what period it will be binding. Section 15(2) is mandatory and it provides :

"On receipt of such award, the appropriate Government shall by order in writing declare the award to be binding. . . ."
14. Thus the Government cannot alter, or cancel, or add to the award, but the award must be declared to be binding as it is. In substance, therefore, the adjudication of the tribunal amounts to a final determination of the dispute which binds the parties as well as the Government.

Our attention was however drawn to the proviso to section 15(2), which runs as follows :-

"Provided that where the appropriate Government is a party to the dispute and in its opinion it would be inexpedient on public grounds to give effect to the whole or any part of the award, it shall on the first available opportunity lay the award together with the statement of its reasons for not making a declaration as aforesaid before the Legislative Assembly of the Province, or where the appropriate Government is the Central Government, before the Central Legislative Assembly, and shall, as soon as may be, cause to be moved therein a resolution for the consideration of the award, and the Legislative Assembly may by its resolution, confirm, modify, or reject the award."
15. This proviso was relied upon by the Respondents to show that the right to appeal from the award could not have been contemplated in any case. But the Act itself makes a distinction between cases in which the Government is a party and those in which the Government is not a party. The proviso relates to a very special type of case and as at present advised I do not wish to express any opinion as to whether an appeal lies to this Court or not in such a case, but, in my judgment, where the Government has only to declare the award to be binding, an appeal shall lie.

16. It is necessary here to say a few words as to the scope of the appeal. As was pointed out by this Court in Pritam Singh v. The State (3), the power under Art. 136 of the Constitution being a special power is to be exercised only in special cases. The rule so laid down is bound to restrict the scope of the appeal in practice in almost all the cases which fall under Art. 136. But in some cases a limitation will be imposed on the scope of the appeal by the very nature of the case and of the tribunal from which an appeal is sought to be brought, and a case under the Industrial Disputes Act seems to be an example of such a case.

17. Dealing now with the merits of the appeal. I am not prepared to hold that this is a proper case for interference with the adjudication of the tribunal. The power of this Court was invoked by the Appellants on four grounds. These grounds have been elaborately examined by Mahajan, J., and two of them have been pronounced to be wholly inadequate for justifying our interference. May view with regard to these two grounds is identical with that of Mahajan, J. and I do not wish to add to what has already said on the subject. The remaining two grounds also are, in my opinion, wholly insufficient to justify the exercise of our special power under Art. 136. One of these grounds is that the award of the tribunal is based on no evidence whatsoever. I do not, however, find that this ground was urged in this form in the application for special leave to appeal to this Court. All that was intended to be urged was that the Appellants wanted to adduce evidence but were not allowed to do so. From the decision of the tribunal, however, it appears that the evidence that was that our related to one isolated point only and the tribunal might well have been justified in not allowing evidence to be admitted on a point which in its opinion had no direct bearing on the issue before them. After hearing the Respondents on this particular point. I am not disposed to hold that the tribunal has committed such an error as would justify the interference of this Court.

18. The last ground urged is that the award has been signed by only two members of the tribunal though it originally consisted of three persons and though the entire hearing of the dispute had taken place before all the three persons. This objection does not appear to me to be fatal to the jurisdiction of the tribunal, because under section 8 of the Act it is not obligatory on the Government to appoint a new member to fill a vacancy if one of the members ceases to be available at any time during the proceedings. Under that section, if the Chairman ceases to be available, the Government must appoint his successor, whereas if a member ceases to be available the Government may or may not appoint any one to fill his place. In the present case, our attention was drawn to some correspondence which shows that one of the members was called upon to act as a member or another tribunal and the award in question was pronounced after informing the Government of the procedure which the Chairman and the remaining member intended to adopt.

19. In the view I have taken, this appeal must fail, and I would accordingly dismiss it with costs, Mahajan, J.

20. This is an appeal by special leave from the determination of an industrial dispute by the Industrial Tribunal appointed under Ordinance VI of 1949.

21. Bharat Bank, Limited, Delhi, the Appellant, is a company registered under the Indian Companies Act. Its employees made certain demands and as a result of an unfavourable response from the bank it appears that they struck work on the 9th March, 1949. The bank in its turn served notices on them to resume work and proceeded to discharge a number of them between the 19th March and 24th March as they failed to do so. The Central Government constituted a tribunal consisting of three persons for the adjudication of industrial disputes in banking companies under section 7 of the Industrial Disputes Act (XIV of 1947). The disputes mentioned in Schedule II of the notification were referred under section 10 of the Act to this Tribunal Item 18 of this schedule reads as follows :-

"Retrenchment and victimization (Specific cases to be cited by employees)."
22. The dispute under this item between the Bharat Bank and its employees was heard by the tribunal at Delhi and its award was made on the 19th January, 1950. It was published in the Government of India Gazette dated 4th February, 1950 and was declared to be binding for a period of one year. The award of the tribunal was signed by two out of its three members.

23. A preliminary objection was raised on behalf of the Central Government as well as on behalf of the Respondents that this Court had not jurisdiction to grant special leave to appeal against the determination of an industrial tribunal inasmuch as it did exercise the judicial powers of the State and that its determination was not in the nature of a judgment, decree or order of a Court so as to be appealable. This being the first case in which special leave was granted from the determination of an industrial tribunal, it is necessary to examine the provisions of the Constitution dealing with this matter and if possible, to define the limits of the jurisdiction of this Court under Art. 136. This article is in these terms :-

"(1) Notwithstanding anything in this Chapter, the Supreme Court may, in its discretion, grant special leave to appeal from any judgment, decree, determination, sentence or order in any cause or matter passed or made by any Court or tribunal in the territory of India.
(2) Nothing in clause (1) shall apply to any judgment, determination, sentence or order passed or made by any Court or tribunal constituted by or under any law relating to the Armed Forces."
24. The article occurs in Chapter IV of Part V of the Constitution "The Union Judiciary." Art. 124 deals with the establishment and constitution of the Supreme Court. Art. 131 confers original jurisdiction on this Court in certain disputes arising between the Government of India and the States, etc. Arts. 132 and 133 deal with the appellate jurisdiction of the Court in appeals from High Courts within the territory of India in civil matters. By Art. 134 limited right of appeal in criminal cases has been allowed. The Judicial Committee of the Privy Council which was the highest Court of Appeal for India prior to 10th October, 1949, was not a Court of criminal appeal in the sense in which this Court has been made a Court of criminal appeal under Art. 134. It could only entertain appeals on the criminal side in exercise of the prerogative of the Kings. Art. 135 empowers this Court to hear all appeals which under existing laws could heard by the Federal Court of India. By the Abolition of the Privy Council Jurisdiction Act, 1949, which came into force on the 10th October, 1949, all the powers that were possessed by the Judicial Committee of the Privy Council in regard to cases or matters arising in India became exercisable by the Federal Court of India whether those powers were exercisable by reason of statutory authority or under the prerogative of the King. The powers of the Judicial Committee were conferred upon it by the Judicial Committee Act, 1844 (7 & 8 Vict. C. 69). Appeals lay to His Majesty in Council from judgments, sentences, decrees or orders of any Court of justice within any British colony or possession abroad. Closely following Art. 135 which confers all the powers of the judicial Committee on the Supreme Court comes Art. 136. The language employed in this article is very wide and is of a comprehensive character. Power given are of an overriding nature. The article commences with the words "Notwithstanding anything in this Chapter." These words indicate that the intention of the Constitution was to disregard in extraordinary cases the limitations contained in the previous article on this Court's power to entertain appeals. These articles dealt with the right of appeal against final decisions of High Courts within the territory of India Art. 136, however, overrides that qualification and empowers this Court to grant special leave even in case where the judgment has not been given by a High Court but has been given by any Court in the territory of India : in other words, it contemplates grant of special leave in cases where a Court subordinate to a High Court has passed or made any order and the situation demands that the order should be quashed or reversed even without having recourse to the usual procedure provided by law in the nature of an appeal, etc. The word "order" in Art. 136 has not been qualified by the word "final." It is clear, therefore, that the power to grant special leave under this article against an order of a Court could be exercised with respect to interlocutory orders also. Another new feature introduced in Art. 136 is the power given to grant special leave against orders and determinations, etc., of any tribunal in the territory of India. This word did not find place in the Judicial Committee's Act, where the phrase used was "a Court of justice." It is the introduction of this new expression in Art. 136 that has led to considerable argument as to its scope. Another expression that did not find place in the Judicial Committee Act but has been introduced in Art. 136 is the word "determination." A question has been raised as to the meaning to be given to these words in the article. On the one hand, it was contended that the words "determination" and "tribunal" were introduced in the article in order to bring within the scope of the appellate jurisdiction of this Court all orders of tribunals of different varieties and descriptions. On the other hand, it was said that the words "determination" and "tribunal" were added in the article by way of abundant caution and the intention was that if a tribunal exercised the judicial powers of the State and the decisions passed in the exercise of that power, this Court as the highest judicial Court in the Republic would have power if it considered necessary in the ends of justice to grant special leave. Clause (2) of Art. 136 excludes the jurisdiction of this Court in respect of military Courts or tribunal. It is interesting to observe that in Art. 138, 139 and 140 the Constitution has conferred powers on Parliament for further enlargement of the powers of this Court.

25. Two points arise for determination in this case, (1) whether the word "Tribunal" in this article has been used in the same sense as "Court," or whether it has been used in a wider sense, and (2) whether the word "determination" in the article includes within its scope the determinations made by industrial tribunals or other similarly constituted bodies or whether it has reference only to determinations of a Court or a tribunal of a purely judicial character. It was conceded by the learned Counsel appearing for the Central Government, Mr. Alladi Krishnaswami Aiyar, that if any tribunal, whether administrative, domestic or quasi-judicial, acts in excess of its jurisdiction, then it can be controlled by the High Court under the powers conferred on them by Art. 226 by the issue of a writ of certiorari. It was said that if the industrial tribunal in this case could be proved to have trespassed beyond the limits of its statutory jurisdiction, then the remedy lies elsewhere and not be a petition of leave under Art. 136. Mr. Alladi's contentions may be briefly summarized as follows : (1) The expression "tribunal" means seat of a Judge, or a Court of justice. Its necessary attribute is that it can give a final judgment between two parties which carries legal sanction by its own force. That the word "tribunal" in juxtaposition to the word "Court" could only mean a tribunal which exercised judicial functions of the State and did not include within its ambit a tribunal which had quasi-tribunal or administrative powers. (2) The kind of orders against which special leave to appeal could be given under Art. 136 have to be of the same nature as passed by a Court : in other words, it was said that unless there was a judicial determination of a controversy between two parties, the order would not be appealable. That in the case of an industrial tribunal what gives binding force to the award is the declaration of the government, that the spark of life to it is given by that declaration and without that the award of the tribunal is lifeless and has no enforceability and hence cannot be held to be of an appealable nature. It was further said that in cases between the Government and its employees by the procedure prescribed in the Act the award could also be rejected and that being so, by its own determination a tribunal could not impose a liability or affect rights. Dr. Bakshi Tek Chand, appearing for the bank, on the other hand, argued that whenever a tribunal, whether exercising judicial or quasi-judicial functions, determined a matter in a judicial manner, then such a determination is within Art. 136. It was said that an industrial tribunal has no administrative or executive functions, that its duty is to adjudicate on an industrial dispute, i.e. to act as a Judge on certain kinds of disputes between employers and employees and that its functions are of a judicial nature, though the ambit of the powers conferred is larger than that of an ordinary Court of law inasmuch as it can grant reliefs which no Court of law could give but that is because of the powers conferred on it by law. It was argued that the plain words of the article should not be given a narrow meaning when the intention of the Constitution was to confer the widest power on this Court. It was further contended that as between private employers and employees and even in certain cases between Government and its employees the decision of the tribunal was binding on the Government and Government had no power either to affirm, modify or reject it. All that it was authorised to do was to announce it and by its declaration give it enforceability; that fact, however, could not affect the question of appealability of the determination under Art. 136. It was finally argued that powers should be exercised by this Court wherever there is a miscarriage of justice by a determination of any tribunal and that if the intention of the Constitution by use of the word "tribunal" was in the same sense as "Court," then it was not necessary to import it in Art. 136, because all tribunals that exercise judicial functions fall within the definition of the word "Court" though they may not have been so described.

26. After considerable thought I have reached the conclusion that the preliminary objection should be overruled. I see no cogent reasons to limit the plain words of the statute and to place a narrow interpretation on words of widest amplitude used therein. In construing the articles of the Constitution it has always to be remembered that India has been constituted into a sovereign democratic republic in order to ensure justice to all its citizens. In other words, the foundations of this republic have been laid on the bedrock of justice. To safeguard these foundations so that they may not be undermined by injustice occurring anywhere, this Court has been constituted. By Art. 3 of the Constitution the Court is empowered to see that the fundamental rights conferred on the citizens by the Constitution are not in any way affected. By Art. 136 it has been given overriding power to grant special leave to appeal against orders of Court and tribunals which go against the principles of natural justice and lead to grave miscarriage of justice. The exercise of these powers could only have been contemplated in cases which affect the rights of people living within the territory of India in respect of their person, property or status. The question, therefore, for consideration is whether the jurisdiction conferred by use of unambiguous phraseology and by words which have a plain grammatical meaning and are of the widest amplitude should be limited and restricted on considerations suggested by Mr. Alladi. The construction suggested by the learned Counsel, if accepted, would in the first instance make the use of certain words in the article unnecessary and redundant and would run counter to the spirit of the Constitution. It must be presumed that the draftsmen of the Constitution knew well the fact that there were a number of tribunals constituted in this country previous to the coming into force of the Constitution which were performing certain administrative, quasi-judicial or domestic functions, that some of them had even the trappings of a Court but in spite of those trappings could not be given that description. If must also be presumed that the Constitution-makers were aware of the fact that the highest Courts in this country had held that all tribunals that discharged judicial functions fell within the definition of the expression "Court." If by the use of the word "tribunal" in Article 136 the intention was to give it the same meaning as "Court," then it was redundant and unnecessary to import it in the article because, by whatever name described, such a tribunal would fall within the definition of the word "Court." The word "Court" has a well-known meaning in legislative history and practice.

27. As pointed out in Halsbury's Laws of England, the word "Court" originally meant the King's Palace but subsequently acquired the meaning of (1) a place where justice was administered, and (2) the person or persons who administer it. In the Indian Evidence Act it is defined as including all Judges and Magistrates and all persons except arbitrators legally authorised to take evidence. This definition is by no means exhaustive and has been framed only for the purpose of the Act. There can be no doubt that to be a Court, the person or persons who constitute it must be entrusted with judicial functions, that is, of deciding questions according to law. However, by agreement between parties arbitrators may be called upon to exercise judicial powers and to decide a dispute according to law but that would not make the arbitrators a Court. It appears to me that before a person or persons can be said to constitute a Court it must be held that they derive their powers from the State and are exercising the judicial powers of the State. In R. v. London County Council, Saville, L. J., gave the following meaning to the word "Court" or "judicial authority" :-

"It is not necessary that it should be a Court in the sense that this Court is a Court : it is enough if it is exercising, after hearing evidence, judicial functions in the sense that it has to decide on evidence between a proposal and an opposition; and that it is not necessary to be strictly a Court; if it is a tribunal which has to decide rightly after hearing evidence and opposition."
28. As pointed out in picturesque language by Lord Sankey, L. C., in Shell Co. of Australia v. Federal Commissioner of Taxation, there are tribunals with many of the trappings of a Court which, nevertheless, are not Courts in the strict sense of exercising judicial power. It seems to me that such tribunals though they are not full-fledged Courts, yet exercise quasi-judicial functions are within the ambit of the word "tribunal" in Art. 136 of the Constitution. It was pointed out in the above case that a tribunal is not necessarily a Court in this strict sense because it gives a final decision, nor because it hears witnesses on oath, nor because two or more contending parties appear before it between whom it has to decide, nor because it gives decisions which affect the rights of subjects nor because there is an appeal to a Court, nor because it is a body to which a matter is referred by another body. The intention of the Constitution by the use of the word "tribunal" in the article seems to have been to include within the scope of Art. 136 tribunals adorned with similar trappings a Court but strictly not coming within that definition. Various definitions of the phrase "judicial power" have been given from time to time. The best definition of it on high authority is the one given by Griffith, C.J., in Huddart, Parker & Co. v. Moorehead, wherein it is defined as follows :-

"The words 'judicial power' as used in section 71 of the Constitution mean the power which every sovereign authority must of necessity have to decide controversies between its subjects, or between itself and its subjects, whether the rights relate to life, liberty or property. The exercise of this power does not begin until some tribunal which has power to give a binding and authoritative decision (whether subject to appeal or not) is called upon to take action."
29. It was conceded that a tribunal constituted under the Industrial Disputes Act, 1947, exercises quasi-judicial powers. That phrase implies that a certain content of the judicial power of the State is vested in it and it is called upon to exercise it. An attempt was made to define the words "judicial" and "quasi-judicial" in the case of Cooper v. Wilson. The relevant quotation reads thus :-

"A true judicial decision presupposes an existing dispute between two or more parties, and then involves four requisites :- (1) The presentation (not necessarily orally) of their case by the parties to the dispute; (2) if the dispute between them us a question of fact, the ascertainment of the fact by means of evidence adduced by the parties to the dispute and often with the assistance of argument by or on behalf of the parties on the evidence; (3) if the dispute between them is a question of law, the submission of legal argument by the parties, and (4) a decision which disposes of the whole matter by a finding upon the facts in dispute and application of the law of the land to the facts so found, including where required a ruling upon any disputed question of law. A quasi-judicial decision equally presupposes an existing dispute between two or more parties and involves (1) and (2), but does not necessarily involve (3) and never involves (4). The place of (4) is in fact taken by administrative action, the character of which is determined by the Minister's free choice."
The extent of judicial power exercise by an Industrial Tribunal will be considered hereinafter in the light of the observations cited above.

30. Reference was made to certain passengers from Prof. Allen's book on Law and Order, Ch. IV, page 69, where mention is made of the kinds of administrative tribunals functioning in various countries to-day. Porter on Administrative Law, 1929 Edn., page 194, was also relied upon. There can be no doubt that varieties of administrative tribunals and domestic tribunals are known to exist in this country as well as in other countries of the world but the real question to decide in each case is as to the extent of judicial power of the State exercised by them. Tribunals which do not derive authority from the sovereign power cannot fall within the ambit of Art. 136. The condition precedent for bringing a tribunal within the ambit of Art. 136 is that it should be constituted by the State. Again a tribunal would be outside the ambit of Art. 136 if it is not invested with any part of the judicial functions of the State but discharges purely administrative or executive duties. Tribunals, however, which are found invested with certain functions of a Court of justice and have some of its trappings also would fall within the ambit of Art. 136 and would be subject to the Appellate control of this Court whenever it is found necessary to exercise that control in the interests of justice.

31. It is now convenient to consider whether a tribunal constituted under the Industrial Disputes Act, 1947, exercises all or any of the functions of a Court of justice and whether it discharges them according to law or whether it can act as it likes in its deliberations and is guided by its own notions of right and wrong. The phrase "industrial dispute" has been defined in section 2 clause (k) of the Act as follows :-

"any dispute or difference between employers and employees, or between employers and workmen, or between workmen and workmen, which is connected with employment or non-employment or the terms of employment or with the conditions of labour, of any person."
32. Such a dispute concerns the rights of employers and employees. Its decision affects the terms of a contract of service or the conditions of employment. Not only may the pecuniary liability of an employer be considerably affected by the adjudication of such dispute but it may even result in the imposition of punishments on him. It may adversely affect the employees as well. Adjudication of such a dispute affects valuable rights. The dispute and its result can always be translated in terms of money. The point for decision in the dispute usually is how much money has to pass out of the pocket of the employer to the pocket of the employee in one form or another and as to what extent the right of freedom of contract stands modified to bring about industrial peace. Power to adjudicate on such a dispute is given by section 7 of the statute to an Industrial Tribunal and a duty is cast on it to adjudicate it in accordance with the provisions of the Act. The words in italics clearly imply that the dispute has to be adjudicated according to law and not in any other manner. When the dispute has to be adjudicated in accordance with the provisions of the Act, it follows that the tribunal has to adhere to law, though that law may be different from the law that an ordinary Court of justice administers. It is noteworthy that the tribunal is to consist of experienced judicial officers and its award is defined as a determination of the dispute. The expression "adjudication" implies that the tribunal is to act as a Judge of the dispute; in other words, it sits as a Court of justice and does not occupy the chair of an administrator. It is pertinent to point out that the tribunal is not given any executive or administrative powers. In Section 38 of the Act power is given to make rules for the purpose of giving effect to the provisions of the Act. Such rules can provide in respect of matters which concern the powers and procedure of tribunals including rules as to the summoning of witnesses, the production of documents relevant to the subject-matter and as to appearance of legal practitioners in proceedings under this Act. Rule 3 of these rules provides that any application for the reference of an industrial dispute to a tribunal shall be made in form. (A) and shall be accompanied by a statement setting forth, inter alia, the names of the parties to the dispute and the specific matters of dispute. It is in a sense in the nature of a plaint in a suit. In Rule 13 power is given to administer oaths. Rule 14 provides as follows :-

"A tribunal may accept, admit or call for evidence at any stage of the proceedings before it and in such manner as it may think fit."
33. Rule 17 provides that at its first sitting the tribunal is to call upon the parties to state their case. In rule 19 provision has been made for proceedings ex parte. Rue 21 provides that in addition to the powers conferred by sub-section (3) of section 11 of the Act, a tribunal shall have the same powers as are vested in a Civil Court under the Code of Civil Procedure when trying a suit, in respect of the following matters, namely, (a) discovery and inspection; (b) granting of adjournment; (c) reception of evidence taken on affidavit; and that the tribunal may summon and examine suo motu any person whose evidence appears to it to be material. It further says that the tribunal shall be deemed to be a Civil Court within the meaning of secs. 480 and 482 of the Code of Criminal Procedure, 1898, Rule 21 says that the representation of the parties, appearing before a tribunal, shall have the right of examination, cross-examination and re-examination and of addressing the court or tribunal when all evidence has been called. In rule 30 it is provided that a party to a reference may be represented by a legal practitioner with the permission of the tribunal and subject to such conditions as the tribunal may impose. In section 11(3) it is said that a tribunal shall have the same powers as are vested in a Civil Court under the Code of Civil Procedure when trying a suit, in respect of the following matters, namely, (a) enforcing the attendance of any person and examining him on oath; (b) compelling the production of documents and material objects; (c) issuing commissions for the examination of witnesses; (d) in respect of such other matters as may be prescribed; and every inquiry or investigation by a tribunal shall be deemed to be a judicial proceeding within the meaning of secs. 193 and 228 of the Indian Penal Code. It is difficult to conceive in view of these provisions that the Industrial Tribunal performs any functions other than that of a judicial nature. The tribunal has certainly the first three requisites a considerable element of the fourth also inasmuch as the tribunal cannot take any administrative action, the character of which is determined by its own choice. It has to make the adjudication in accordance with the provisions of the Act as laid down in section 7. It consists of persons who are qualified to be or have been Judges. It is its duty to adjudicate on a serious dispute between employers and employees as affecting their right of freedom of contract and it can impose liabilities of a pecuniary nature and disobedience of its award is made punishable. The powers exercisable by a tribunal of the nature were considered in a judgment of the Federal Court of India in Western India Automobile Association v. Industrial Tribunal, Bombay, and it was observed that such a tribunal can do what no Court can, namely, add to or alter the terms or conditions of the contract of service. The tribunal having been entrusted with the duty of adjudicating a dispute of a peculiar character, it is for this reason that it is armed with extraordinary powers. These powers, however, are derived from the statute. These are the rules of the game and it has to decide according to these rules. The powers conferred have the sanction of law behind it and are not exercisable by reason of any discretion vested in the members of the tribunal. The adjudication of the dispute has to be in accordance with evidence legally adduced and the parties have a right to be heard and being represented by a legal practitioner. Right to examine and cross- examine witnesses has been given to the parties and finally they can address the tribunal when evidence is closed. The whole procedure adopted by the Act and the rules is modelled on the Code of Civil Procedure. In my opinion, therefore, the Industrial Tribunal has all the necessary attributes of a Court of justice. It has no other function except that of adjudicating on a dispute. It is no doubt true that by reason of the nature of the dispute that they have to adjudicate the law gives them wider powers than are possessed by ordinary Courts of law, but powers of such a nature do not affect the question that they are exercising judicial power. Statutes like the Relief of Indebtedness Act, or the Encumbered Estates Act have conferred powers on Courts which are not ordinarily known to law and which affect contractual rights. That circumstances does not make them anything else but tribunals exercising judicial power of the State, though in a degree different from the ordinary Courts and to an extent which is also different from that enjoyed by an ordinary Court of law. They may rightly as quasi-judicial bodies because they are out of the hierarchy of the ordinary judicial system but that circumstances cannot affect the question of their being within the ambit of Art. 136.

34. It may also be observed that the tribunal is deemed to be a Civil Court for certain purposes as laid down in rule 21 of the rules above cited and in section 11(3) of the Act. As a Civil Court if it exercises any of the powers contemplated by this section its decisions would become subject to appeal to a District Judge and a fortiori this Court's power under Art. 136 would at once be attracted in any case in respect of these matters. Again in Chapter VI of the Act breach of the terms of an award has been made punishable by section 29 of the Act. The result, therefore, is that disobedience of the terms of an award is punishable under the Act. That being so, a determination of the tribunal not only affects the freedom of contract and imposes pecuniary liability on the employer or confers pecuniary benefits on the employees, but it also involves serious consequences as failure to observe those terms makes a persons liable to the penalties laid down in Chapter VI. An award which has these serious consequences can hardly be said to have been given by a tribunal which does not exercise some of the important judicial functions of the State.

Considerable stress was laid by Mr. Alladi on the provisions of secs. 15 of the Act. Section 15 enacts as follows :-

"(1) Where an industrial dispute has been referred to a Tribunal for adjudication, it shall hold its proceedings expeditiously and shall, as soon as practicable on the conclusion thereof, submit its award to the appropriate Government.
(2) On receipt of such award, the appropriate Government shall by order in writing declare the award to be binding.
Provided that where the appropriate Government is a party to the dispute and in its opinion it would be inexpedient on public grounds to give effect to the whole or any part of the award, it shall on the first available opportunity lay the award together with the statement of its reasons for not making a declaration as aforesaid before the Legislative Assembly of the province, or where the appropriate Government is the Central Government, before the Central Legislature, and shall, as soon as may be, cause to be moved therein a resolution for the consideration of the award; and the Legislative Assembly or as the case may be, the Central Legislature, may by its resolution confirm, modify, or reject the award.

(3) On the passing of a resolution under the proviso to sub-section (2), unless the award is rejected thereby, the appropriate Government shall by order in writing declare the award as confirmed or modified by the resolution, as the case may, to be binding.

(4) Save as provided in the proviso to sub-section (3) of section 19, an award declared to be binding under this section shall not be called in question in any manner."

35. As regards clause (4), it was conceded rightly that a law dealing with industrial disputes and enacted in the year 1947 could not in any way affect the provisions of the Constitution laid down in Art. 136. It was, however, strenuously urged that the award of the Tribunal had no binding force by itself and unless the appropriate Government made a declaration in writing under clause (2) of section 15, this award was a lifeless document and had no sanction behind it and therefore it could not have been contemplated that it would be appealable even by special leave. In my opinion, this contention is unsound. The provisions of clause (2) of section 15 leave no discretion in the Government either to affirm, modify or reject the award. It is bound to declare it binding. It has no option in the matter. In such a situation it is the determination by the Tribunal that matters. Without that determination Government cannot function. It does not possess the power either to adjudicate the dispute or to alter it in any manner whatsoever. That power vests in the Tribunal alone. The rights of the parties are really affected by the adjudication contained in the award, not by the Government's declaration which is automatic. It is no doubt true that announcement of the award by the Government gives it binding force but that does not affect the question of the appealability of the determination under Art. 136 of the Constitution. The apposite answer to this contention may be given in the language of the decision in Rex v. Electricity Commissioners. The relevant passage runs thus :-

"It is necessary, however, to deal with what I think was the main objection of the Attorney-General. Is this case he said the Commissioners come to no decision at all. They act merely as advisers. They recommend an order embodying a scheme to the Minister of Transport who may confirm it with or without modifications. Similarly the Minister of Transport comes to no decision. He submits the order to the Houses of Parliament, who may approve it with or without modifications. The Houses of Parliament may put anything into the order they please, whether consistent with the Act of 1919, or not. Until they have approved, nothing is decided, and in truth the whole procedure, draft scheme, inquiry, order, confirmation, approval, is only part of a process by which Parliament is expressing its will, and at no stage is subject to any control by the Courts. It is unnecessary to emphasize the constitutional importance of the contention. Given its full effect, it means that the checks and safeguards which have been imposed by Act of Parliament, including the freedom from compulsory taking, can be removed, and new and onerous and inconsistent obligations imposed without an Act of Parliament, and by simple resolution of both Houses of Parliament. I do not find it necessary to determine whether, on the proper construction of the statute, resolutions of the two Houses of Parliament could have the effect claimed. In the provisions that the final decision of the Commissioners is not to be operative until it has been approved by the two Houses of Parliament. I find nothing inconsistent with the view that they act judicially and within the limits prescribed by the Act of Parliament, and that the Courts have power to keep them within those limits. It is to be noted that it is the order of the Commissioner that eventually takes effect, neither the Minister of Transport who confirms, nor the Houses of Parliament who approve, can under the statute make an order which in respect of the matters in question has any operation. I know of no authority which compels me to hold that a proceeding cannot be a judicial proceeding subject to confirmation or approval, even where the approval has to be that of the Houses of Parliament. The authorities are to the contrary."
36. The observations, though they relate to a case which concerns the issue of a writ of prohibition and certiorari have application to the present case. Here no discretion whatsoever has been left in the Government in ordinary cases to either modify or to reject the determination of the tribunal. The fact that the Government has to make a declaration after the final decision of the tribunal is not in any was inconsistent with the view that the tribunal acts judicially. It may also be pointed out that within the statute itself a clue has been provided which shows that the circumstances that the award has to be declared by an order of Government to be binding does not affect the question of its applicability. In Art. 136, clause (2) express provision has been made for excepting from the ambit of Art. 136 the decisions of military Courts and Tribunals. It follows that but for the exception it was considered that these would be within Art. 136 clause (1). It is quite clear from the various provisions of the Army Act that the decisions of military tribunals or Courts are subject to confirmation either by the Commander-in-Chief or various other military authorities. It is only after such confirmation that can operate. It has never been considered that that fact in any way affects the question of their applicability.

37. Rex v. Minister of Health, also supports this view. There by the Housing Act, 1925, by section 40, a local authority which had prepared an improvement scheme was required to present a petition to the Minister praying that an order should be made confirming such scheme. Sub-section (3) provided that the Minister after considering the petition may cause a local inquiry to be made and may by order confirm the scheme with or without conditions or modifications. In sub-section (5), it was stated that the order of the Minister when made shall have effect as if enacted in this Act. It was held by the Court of Appeal that as the order made by the Minister was made without the statutory conditions having been complied with, it was ultra vires and, therefore, a writ of certiorari should issue for the purpose of quashing it. Reliance was placed by Scrutton, L. J., on Rex v. Electricity Commissioners. The same view was expressed in Minister of Health v. The King. It was observed that judicial review by prohibition or a writ of certiorari was permissible if the Minister of Health in confirming the order exceeded his statutory powers. It is clear, therefore, that simply because an order has to be confirmed by a Minister or by the Government it in any way affects the power of judicial review. Reference may also be made to the observations in Smith v. The Queen. At page 623 is was observed that it is a common principle in every case which was in itself the character of a judicial proceeding that the party against whom a judgment is to operate shall have an opportunity of being heard. In this sense it can hardly be disputed that the proceeding before an industrial tribunal is a judicial proceedings. In my judgment, therefore, the contention raised by Mr. Alladi that this Court cannot exercise its powers under Art. 136 because the decision of the tribunal has no force till a declaration is made by the Government cannot be sustained.

38. As regards section 19, it was contended that an award declared by the appropriate Government under section 15 to be binding can only come into operation on such date as may be specified by the appropriate Government and can only remain in operation for such period not exceeding one year, as may be fixed by that Government and it was said that herein the Government had the power to state the period from which the award was to commence and the time for which it was to remain in force. This section does not, in my opinion, affect the question of the appealability of the determination of the tribunal. Government has certain functions to perform in its own sphere after the award is made. In certain cases it is itself a party to the dispute, it has certain overriding powers and these overriding powers are that if it considers that the award is not in public interests it may refer it to the legislature. The legislature, however, has the power to modify, accept or reject the award. These overriding powers presuppose the existence of a valid determination by a tribunal. If that determination is in excess of jurisdiction or otherwise proceeds in a manner that offends against the rules of natural justice and is set aside by exercise of power under Art. 136, then no occasion arises for exercise of governmental power under the Act. Given a valid award, it could not be denied that the Government could exercise its powers in any manner it considered best and the exercise of that power is outside the Constitution of this Court. In this connection reference was made to Moses v. Parker. The passage on which emphasis was laid reads as follows :-

"The Court has been substituted for the commissioners to report to the governor. The difference is that their report is to be binding on him. Probably it was thought that the status and training of the judges made them the most proper depositaries of that power. But that does not make their action a judicial action in the sense that it can be tested and altered by appeal. It is no more judicial than was the action of the commissioners and the governor. The Court is to be guided by equity and good conscience and the best evidence. So were the commissioners. So every public officer ought to be. But they are expressly exonerated from all rules of law and equity, and all legal forms. How then can the propriety of their decision be tested on appeal ? What are the cannons by which this Board is to be guided in advising. Her Majesty whether the Supreme Court is right or wrong ? It seems almost impossible that decisions can be varied except by reference to some rule; whereas the Court making them is free from rules. If appeals were allowed, the certain result would be to establish some system of rules, and that is the very thing from which the Tesmanian Legislature has desired to leave the Supreme Court free and unfettered in each case. If it were clear that appeals ought to be allowed such difficulties would doubtless be met somehow. But there are strong arguments to show that the matter is not of an appreciable nature."
39. One would have expected that after this opinion the decision would have been that the Judicial Committee had no jurisdiction to entertain the appeal but their Lordships proceeded to base their decision not on this ground but on the ground that this was not a fit case for the exercise of the prerogative of the King. In my opinion, the observations made in that case have no apposite application to the provisions of the statute with which we are concerned. I do not see any difficulty in this case in testing the propriety of the determination of the tribunal. This Court is not to substitute its decision for the determination of the tribunal when granting relief under Art. 136. When it chooses to interfere in the exercise of these extraordinary powers, it does so because the tribunal has either exceeded its jurisdiction or has approached the questions referred to it in a manner which is likely to result in injustice or has adopted a procedure which runs counter to the well-established rules of natural justice. In other words, if it has denied a hearing to a party or has refused to record his evidence or has acted in any other manner, in an arbitrary or despotic fashion - in such circumstances no question arises of this Court constituting itself into a tribunal and assuming powers of settling a dispute. All that the Court when it entertains an appeal would do is to quash the award and direct the tribunal to proceed within the powers conferred on it and to approach the adjudication of the dispute according to principles of natural justice. This Court under Art. 136 would not constitute itself into a mere Court of error. Extraordinary powers have to be exercised in rare and exceptional cases and on well-known principles. Considered in the light of these principles, there is no insuperable difficulty in the present case of the nature pointed out in the passage cited above. It was conceded that the High Court could exercise powers under section 226 and could quash an award but it was said that under Art. 136 this power should not be exercised in an appeal. I do not see why, particularly when after the High Court has passed any decision on an application made to it in exercise of the powers under section 226, that decision could be brought to this Court in appeal. In the matter of an industrial dispute where expedition is the crux of the matter, it is essential that any abuse of powers by such tribunals is corrected as soon as possible and with expedition.

40. It may be mentioned that it is no novel practice for a Court empowered to grant special leave to exercise its powers even though there may be intermediate rights of appeal or other remedies available, if it is considered to do so in extraordinary situations. Vide Bentwick's Privy Council Practice, 3rd Edition, page 125. Therein it is stated as follows :-

"In several cases from Jamaica, the Privy Council granted leave to appeal to the Queen in Council directly from the Supreme Court, without an intermediate appeal (which would have been attended with much expense and delay) to the Court of Error in the island, there being in each of those cases manifestly some point of law raised which deserved discussion."
41. The cases were In re Barnett, Harrison v. Scott and Attorney-General of Jamaica v. Manderson. The phraseology employed in Art. 136 itself justifies this course. The article empowers this Court to grant special leave against sentences or orders made by this Court. In all other articles of the Constitution right of appeal is conferred against final decisions of the highest Court of Appeal in the country but under this article prayer is given to this Court to circumvent that procedure if it is considered necessary to do. I am, therefore, of the opinion that the mere circumstances that a remedy in the nature of a writ of certiorari is open to the petitioners does not necessarily lead to the conclusion that the power of this Court under Art. 136 is circumscribed by that circumstances. Whenever judicial review is permissible in one form or another, this Court as the highest Court in the land can exercise its special powers and circumvent ordinary procedure by granting special leave. What it has to ultimately decide, it can decide earlier.

42. I now proceed to examine some of the cases to which reference was made by Mr. Alladi.

43. Three Australian cases were cited which concern the construction of secs. 51, 71 and 72 of the Australian Constitution (63 and 64 Vict. c. 12), Section 72 requires that every justice of the High Court and every justice of any other Court created by the Parliament of the Commonwealth shall, subject to the power of removal contained in the section, be appointed for life. Section 71 confers the whole judicial power of the Commonwealth upon the Courts therein mentioned and no other tribunal or body can exercise that power. Every Court referred to in section 71 has to be constituted in the manner provided by section 72. The question in these cases was as to the meaning of the phrase "judicial power of the Commonwealth." Similar phraseology has not been used in any part of the Constitution of India and these circumstances it is difficult to derive any assistance from these decisions in solving the problem before us. The Constitution of India is not modelled on the Constitution of Australia and that being so, any observation made in decisions given under that Constitution cannot be held to be a safe guide in the interpretation of languages employed in a Constitution differently drafted.

44. The first of these cases is Waterside Worker's Federation of Australia v. J. W. Alexander Ltd. Therein it was held that the power conferred by the Commonwealth Conciliation and Arbitration Act, 1904-1915, upon the Commonwealth Court of Conciliation and Arbitration to enforce awards made by it is part of "the judicial power of the Commonwealth" within the meaning of section 71 of the Constitution, and can only be vested in the Courts mentioned in that section. Mr. Alladi placed reliance on a passage at page 467 in the judgment of Isaacs and Rich, JJ., which reads as follows :-

"The arbitral part of the Act, therefore, is quite within the power of pl. xxxv., and it not intended by the Act to be exercised by an ordinary Court of Justice, which, it is suggested, Parliament by some strange perversity proceeded to destroy at birth. It is true that enforcement provisions are found......... But all this was in imitation of the State Acts of Arbitration, and not in reliance on the Judicature Chapter of the Federal Constitution. The arbitral portion of the Act is, in our opinion, perfectly good, subject to its severability from any other portion which may be bad."
45. It was argued that the Industrial Tribunal here was an arbitration of the same kind as in Australia and exercises similar functions. It is, however, pertinent to observe that the phraseology employed in section 15 of the Indian Act is different from that used in the Australian statute. The Indian statute has constituted different bodies for different purposes. An Industrial Tribunal has been constituted only to discharge one function of adjudication. It is not described as an arbitral Tribunals. The Act has avoided the use of the word "arbitration" either in is preamble or in any of its relevant provisions though the determination has been named as an award. In these circumstances it is unsafe to seek any guidance from observations made in this case.

46. The next case to which reference was made is Rola Co. (Australia) Proprietary, Ltd. v. The Commonwealth. The question here was whether the Women's Employment Board constituted under the Women's Employment Act, 1942, did not exercise the judicial power of the Commonwealth. It was held that the Board exercised functions which were arbitral in character. Emphasis was laid on a passage occurring in page 198 of the report which reads as follows :-

"An industrial award lays down rules of conduct for the future. It does not purport to ascertain and enforce existing rights, it is directed to the creation of new rights. It is urged on behalf of the Plaintiff that a determination of the Committee does not create a rule of conduct binding the parties for the future, but that it authoritatively determines a possibly controverted question of fact and that the making of such an authoritative determination is necessarily an exercise of judicial power. Reference is made to the frequently quoted statement of Griffith, C.J., in Huddart Parker & Co. Pty., Ltd. v. Moorehead, approved by the Privy Council in Shell Co. of Australia, Ltd. v. Federal Commissioner of Taxation -
"I am of opinion that the words 'judicial power' as used in section 71 of the Constitution mean the power which every sovereign authority must of necessity have to decide controversies between its subjects, or between itself and its subjects, whether the rights relate to life, liberty or property. The exercise of this power does not begin until some tribunal which has power to give a binding and authoritative decision (whether subject to appeal or not) is called upon to take action."
47. Reg. 5C gives Committees power to decide controversies between subjects relating to their rights and the regulation purports to make those decisions binding and authoritative.

48. I am not satisfied that the words of Griffith, C.J., are properly interpreted when it is said that they mean that a power to make binding and authoritative decisions as to facts is necessarily judicial power. I direct attention to the concluding words - "is called upon to take action." In any opinion these words are directed to action to be taken by a Tribunal which has power to give a binding and authoritative decision. The mere giving of the decision is not the action to which the learned Chief Justice referred. If a body which has power to give a binding and authoritative decision is able to take action so as to enforce that decision, then, but only then, according to the definition quoted, all the attributes of judicial power are plainly present. I refer to what I say more in detail hereafter, that the Privy Council, in the Shell case in which approval was given to the definition quoted, expressly held that a Tribunal was not necessarily a Court because it gave decisions (even final decisions) between contending parties which affected their rights.

49. In Huddart Parker's case, Isaacs, J., referred to the statement of Palles, C. B., in R. v. Local Government Board for Ireland "to erect a Tribunal into a "Court" or "jurisdiction," so as to make its determinations judicial, the essential element is that it should have powers by its determination within jurisdiction, to impose liability or affects right."By this," said the learned Chief Baron, "I mean that the liability is imposed, or the right affected by the determination only, and not by the fact determined, and so that the liability will exist, or the right will be affected, although the determination be wrong in law or in fact. It is otherwise of a ministerial power. If the existence of such a power depends upon a contingency, although it may be necessary for the officer to determine whether the contingency has happened, in order to know whether he shall exercise the power, his determination does not bind. The happening of the contingency may be questioned in an action brought to try the legality of the act done under the alleged exercise of the power. But where the determination binds, although it is based on an erroneous view of facts or law, then the power authorizing it is judicial. There we get a modern use of the term "judicial power." This statement of the characteristics of judicial power looks to what, in Waterside Worker's Federation of Australia v. Gilchrist, Watt & Sanderson, Ltd., Isaacs and Rich, JJ., referred to as the creation of instant liability in specified persons as distinct from laying down a rule or standard of conduct for the future.

50. The decision of an ordinary Court that B is bound to pay money to A applies a pre-existing standard of rights and duties not created by the Court itself, with the result that there is an immediately enforceable liability of B to pay to A the sum of money in question. The decision of the Women's Employment Board does not create any such liability, nor does the determination of a Committee of Reference create any such liability, for example, to pay wages to a particular female, it would be necessary for the female or some person on her behalf (see Reg. 9A) to sue in a Court of competent jurisdiction. If such a proceeding succeeded there would then be a liability created by the determination of the Court. In such a proceeding the determination of the Committee of Reference would be evidence of the facts to which it related, but that determination would not in itself create liability." The concluding words of the passage quoted above at once distinguish the present case from the Australian case. The award given by the Industrial Tribunal in respect either of bonus or higher wages, etc. is enforceable by its own force and by the coercive machinery of the Act and it is not merely a declaration of a character that furnishes a cause of action to the employee to bring a suit on its foot to recover the wages. An arbitral Tribunal's decision cannot be enforced unless it has the sanction of the Court of justice behind it but the award of the tribunal is enforceable under the Act itself by the coercive machinery provided therein. It is the terms of the award that are enforceable and not the terms of the order made by the Government. It is the breach of the terms of the award that is punishable and not any breach of Government's order. The Government itself is bound to declare the award binding and it has no option whatsoever in the matter. It is no doubt true that the tribunal has not only to decide the existing rights and liabilities of the parties and it can lay down rules of conduct for the future but it does so because by law it is authorised to do so. Its decision carries the sanction with it. The Government is bound to give effect to it and the statute enforces it by coercive machinery. In may view, therefore, this decision again has no relevancy to the present case.

51. The third case to which reference was made is Shell Co. of Australia v. Federal Commissioner of Taxation. That was an Income-tax matter and the decision has been considered in an earlier part of this judgment. Reference was also made to Mohammad Ahmed v. Governal-General in Council in which it was held that an improvement trust was not a Civil Court subordinate to the High Court under section 115 of the Code of Civil Procedure. That has no bearing on the matter in issue here. Similar point was discussed in Hari v. Secretary of State for India. Labour Relations Board v. John East Iron Works Ltd. is a Canadian case and the decision proceeded on the same lines as in the Australian cases.

52. Mr. Sen appearing for the respondents placed reliance on O'Connor v. Waldron. The relevant passage occurs at page 81 which runs thus :-

53. Mr. Sen appearing for the respondents placed reliance on O'Conloped. Originally it was intended for the protection of Judges sitting in recognised Courts of justice established as such. The object no doubt was that Judge might exercise their functions free from any danger that they might be called to account for any words spoken as Judges. The doctrine has been extended to Tribunals exercising functions equivalent to those of an established Court of justice. In their Lordships' opinion the law on the subject was accurately stated by Lord Esher in Royal Aquarium, etc., Ltd. v. Parkinson, where he says that the privilege 'applies wherever there is an authorised inquiry which, though not before a Court of Justice, is before a Tribunal which has similar attributes...... This doctrine has never been extended further than to Courts of Justice and Tribunals acting in a manner similar to that in which such Courts act.'"

54. The learned Counsel contended that the word "Tribunal" in Art. 136 could only have reference to those Tribunals which exercise functions equivalent to that of a Court of Justice. I have no hesitation in holding that the Industrial Tribunal has similar attributes as that of a Court of Justice in view of the various provisions to which I have made reference. Reference was also made to certain passage occurring in pages 422 and 428 of Toronto Corporation v. York Corporation. That was a case of the Municipal Board of Ontario. It was held there that the Board was merely an administrative Tribunal. Next reliance was placed on R. v. National Arbitration Tribunal, Ex parte Horatio Crowther & Co., Ltd. That dealt with the powers of the National Arbitration Tribunal. In my opinion this citation also is not of much assistance.

55. It was again urged by Mr. Alladi that the word "Tribunal" was introduced in the article to provide for cases of Tribunals like the Board of Revenue. The suggestion does not appear to be sound because a Revenue Board has all the attributes of a Court of Justice and falls within the definition of the ward "Court" in matters where it adjudicates on rights of parties.

56. The word "Tribunal" has been used in previous legislation in a number of statutes and it is difficult to think that the Constitution when it introduced this word in Art. 136 intended to limit its meaning to only those Tribunals which though not described as Courts strictly speaking, were discharging the same or analogous functions as were being discharged by Courts.

57. For the reasons given above I am of the opinion that the word "tribunal" in Art. 136 has to be construed liberally and not in any narrow sense and an industrial inasmuch as it discharges functions of a judicial nature in accordance with law comes within the ambit of the article and from its determination an application for special leave is competent.

58. The question now to determine is whether the exercise of overriding powers of this Court can be justified on any ground whatsoever in the present. As I have already said, exceptional and extra-ordinary powers of this character can only be justifiably used where there has been a grave miscarriage of justice or where the procedure adopted by the Tribunal is such that it offends against all notions of legal procedure.

59. Dr. Bakshi Tek Chand for the petitioner-bank urged four grounds justifying exercise of true special jurisdiction of this Court. Firstly, he contended that the word "victimization" used in clause of the reference has been interpreted in such a manner by the Tribunal that it has usurped jurisdiction to decide disputes which were never referred to it. In my view this is not a matter which can justify the exercise of the powers under Art. 136. This Court is not a mere Court of error. The word "victimization" has not been defined in the statute and is not in any sense a term of law or a term of art. It is an ordinary English word which means that a certain person has become a victim, in other words, that he has been unjustly dealt with. It was argued that the word has acquired a special meaning in regard to industrial disputes and connotes a person who becomes a victim of the employer's wrath by reason of his trade union activities and that the word cannot relate to a person who has been merely unjustly dismissed. Be that as it may. The determination of the Tribunal has not been materially affected by this interpretation of the word to any large extent and that being so, it does not call for the exercise of the special power.

60. The second ground urged was that the tribunal has erred in ordering reinstatement of persons who were guilty of an illegal strike. It was contended that section 23(b) of the Act has been wrongly construed by it and as a result of this misconstruction persons who were guilty of a wrong and who could not have been reinstated have been reinstated. In brief, the argument was that under section 23(b) when a matter has been referred to a tribunal in respect of an earlier strike, any strike during the pendency of that dispute is an illegal strike and that was the situation here. The employees of the bank had struck work in December, 1948. That dispute had been referred to an industrial tribunal. It has during the pendency of that dispute that another strike took place which led to the dismissal of the employees who have now been reinstated by the present award. The Calcutta High Court has held that a strike during the pendency of the period of truce and during the pendency of an earlier dispute before a tribunal is illegal even if it is brought about as a result of fresh and new demands which are not covered by the earlier dispute. One of the members of the tribunal thought that the decision laid down the law correctly on the point, but the other member thought that the decision was erroneous. Both of them, however, agreed that whether the strike was legal or illegal that point did not in any way affect the question that they had to decide under issue 18. The consequences of an illegal strike are laid down in the Act and certain penalties are provided therein. The Act nowhere states that persons guilty of illegal strike cannot be reinstated. Be that as it may. The reference to the tribunal was made by the Government in respect of an illegal strike and the Tribunal was bound to give its decision on the reference. Item 18 of Schedule 2 clearly empowers the tribunal to deal with cases of victimization as a result of the third strike which the petitioner described as illegal. The tribunal may be right or may be wrong in the view they have taken but it seems to me this is again not a question of that vital character which would justify the grant of special leave under Article 136.

61. The next question raised by the learned Counsel was that the award of the tribunal is based on no evidence whatsoever. This contention requires serious consideration. I have examined the proceedings of the tribunal and it appears that all it did was that as required by rule 17 at the first sitting it called upon the parties to state their cases. Mr. Parwana on behalf of the employees stated their respective cases and Mr. Ved Vyas who represented the bank stated the bank's case and after the cases have been stated the proceedings terminated and both parties addressed arguments and the tribunal proceeded to give its award. Whether the charge of victimization in individual cases was proved or not depended on proof of certain facts which has to be established by evidence. The onus or proving victimization clearly rested on the employees. No evidence whatsoever was led on their behalf. The statement of the case by Mr. Parwana was not on oath. There was no examination or cross-examination of Mr. Parwana. No affidavit supporting the facts stated by Mr. Parwana was filed by him or by any employee. Mr. Parwana produced an abstract of the correspondence but the original correspondence was not produced. The bank disputed the facts stated by Mr. Parwana by means of a lengthy affidavit. It seems no reference was made ever to this affidavit by the tribunal. No counter affidavit was filed in reply to the facts stated in this affidavit. The bank wanted to call some evidence. Particular reference was made in respect of a scurrilous letter issued by one Bhattacharya on behalf of the employees and distributed by them, which, it is alleged, considerably shook the credit of the bank. This opportunity was denied to it. It was contended before us that the bank wanted to lead evidence on certain matters and that the opportunity to lead it was denied. There is nothing on the record to support this contention. The result, therefore, is that the facts on the basis of which allegations of victimization have been made are neither supported by an affidavit nor by any evidence and the award is based on no evidence whatsoever. The Act as well as the rules framed under it contemplate a proper hearing, discovery and inspection of documents and production of evidence, etc. None of this procedure was followed by the tribunal. It is difficult to set on what material the tribunal has given its award as there is none existing on the present record and the Respondents' Counsel could not point out to any such material. At one time during the argument I was inclined to think that possibly both parties by agreement consented to treat the statement of case as evidence in the case and did not wish to produce any other evidence, but the affidavit filed on behalf of the bank disputes all the facts stated by Mr. Parwana. The only evidence on the record is the bank's affidavit and if the facts contained in the affidavit are accepted, then the determination made by the tribunal cannot stand. It seems to me, therefore, that the procedure adopted by the tribunal was against all principles of natural justice and the award is thereby vitiated and should be set aside. It happens that when the safeguard of an appeal is not provided by law the tendency sometimes is to act in an arbitrary manner like a benevolent despot. Benevolent despotism, however, is foreign to a democratic Constitution. The members of the tribunal seems to have thought that having heard the statement of the cases of the parties they could proceed to a judgment on their own view of its right or wrong unaided by any material. That kind of procedure to my mind is unwarranted by the statute and is foreign to a democratic Constitution. In these circumstances it is the compelling duty of this Court to exercise its extraordinary powers and to quash such an award.

62. The last contention raised by Bakshi Tek Chand was that though a tribunal consisting of three persons was appointed to adjudicate on the dispute, the award has only been signed by two of them. Reference in this connection was made to section 16 of the Act which says that the award of a tribunal shall be in writing and shall be signed by all the members of the tribunal and that nothing in the section shall be deemed to prevent any member of the tribunal from recording a minute of dissent. The provisions of the section are mandatory and have not been complied with. It is common ground that the case was stated by the parties at a sitting when all the members of the tribunal were present and the arguments were heard by all of them. No sitting took place subsequent to this which would have necessitated the carrying on of proceedings by two members of the tribunal by a quorum. When the matter has been heard by all the three members, the award should have been given by all of them. Therefore the award given by two of them is not the award of the tribunal constituted by the Government. It is, therefore, vitiated and has to be quashed. Reference in this connection was made to section 8 of the Act which reads as follows :-

"If the services of the Chairman of the Board or of the Chairman or other member of a Court or Tribunal cease to be available at any time, the appropriate Government shall, in the case of a Chairman, and may in the case of any other member, appoint another independent person to fill the vacancy, and the proceedings shall be continued before the Board, Court or Tribunal so reconstituted."
63. The tribunal was never reconstituted by the Government by any notification. Under section 7 a tribunal has to be constituted in accordance with the provisions of the Act by the Government. The Government having constituted a tribunal of three persons it had power under section 8 to reconstitute it but it did not exercise that power. The result, therefore, is that the tribunal as originally constituted was not the tribunal which gave the award in this reference. Only two members have given the award. It was said the one of the members ceased to be available and the Government was not bound to fill up that vacancy. There is no material on the record to prove whether any member became unavailable and if so, when. But even if a member becomes unavailable and the Government does not choose to fill up the vacancy, still the Government has to reconstitute the tribunal by saying that two members will now constitute the tribunal. An affidavit with two telegrams annexed was filed before us on behalf of the Respondents which disclosed that Mr. Chandrasekhara Aiyar who was one of the members of the tribunal, in November, 1949, was appointed a member of the Boundary commission in Bengal and that the other two members sent a telegram to the Labour Ministry asking it to fill up the vacancy or to reconstitute the tribunal. The advice given by the Ministry was that they could proceed as they were and the Government would later on, if necessary, fill up the vacancy. We are not concerned whether the advice given was right or wrong. But the fact remains that the tribunal was never reconstituted and it was not denied that Mr. Chandrasekhara Aiyar is now sitting in the same tribunal without being again nominated to it and the tribunal is hearing the same reference under the other issues referred to it. Moreover, I do not see why after having heard the reference he could not give the award even if he was in Calcutta on sign the award given by the other two members. The idea of three persons hearing a case and two of them deciding it is repugnant to all notions of fairness. It may well have been that the opinion of the third may have influenced the other two or the decision arrived at may have been quite different. It so happened in this case that two members of the tribunal differed on an important question of law but somehow adjusted their difference and gave a unanimous award. The presence of the third is such a situation may have very vitally affected the result. After a good deal of thought I feel that it would be most dangerous for this Court to condone proceedings of this character. If exceptional powers are not exercised even when a body legally constituted under the statute does not function according to the statute, then they defeat the very purpose of the Constitution.

64. Reference in this connection may be made to the decision of their Lordships of the Privy Council in Fakira v. King-Emperor. In that case section 377 of the Code of Criminal Procedure as modified and as applicable to Hyderabad stood as follows :-

"In every case so submitted, the confirmation of the sentence or order passed by the Court of the Resident at Hyderabad shall, when such court consists of two or more Judges, be made, passed and signed by at least two of them."
65. In Fakira's case the order of confirmation was only made, passed and signed by one of them, though the Court of the Resident consisted of two Judges. Their Lordships held that the peremptory provisions of section 377 had not been complied with and that the sentence passed had not been validly confirmed. The appeal was allowed and the case was remitted to the Court of the Resident. The provisions of section 18 of the Industrial Disputes Act are also of the peremptory nature. Reference may also be made to a case arising under the Bar Councils Act reported in In re : an Advocate, Madras, where one member of the tribunal under that Act had died and had not signed the report. It was held that the tribunal ceased to be properly constituted and that the report could not be considered.

66. For the reasons given above I would quash this award and direct that the tribunal which is still functioning should readjudge item 18 of the reference and then submit its award on this point to Government. The employees cannot be held responsible for the method of procedure adopted by two members of the tribunal. Each party will have to bear their own costs in this Court. The appeal is allowed to the extent indicated above.

Mukherjea, J.

67. This appeal, which has come up before us on special leave, is directed against an award made by the All-India Industrial Tribunal, dated the 19th of January, 1950. The Tribunal was constituted by the Central Government under section 7 of the Industrial Dispute Act and a large number of dispute between several Banking companies and their employees were referred to it for adjudication. amongst these Banking companies were the Bharat Bank Limited, the Appellants before us, and the disputed between them and their employees, who are Respondents in this appeal, related inter alia to a number of cases of retrenchment and victimization which the latter alleged against the former. The tribunal held its enquiry in Delhi in respect to the cases which were connected with the Delhi Branch of the Appellants and as a result of the same, made their award on 19th January, 1950, holding that 26 persons, who were employees under the Appellants, were improperly dismissed by the latter and should be reinstated. Further directions were given in the award regarding the salaries and allowances that were to be paid to the dismissed employees. This award was declared to be binding in terms of the provisions of secs. 15 and 19 of the Industrial Disputes Act by the Central Government on 30th of January, 1950, and it was directed to remain in operation for a period of one year. It is against this award that the present appeal has been preferred.

68. On behalf of the Indian Union which appeared as an intervener in this appeal, as also on behalf of the Respondents, a preliminary objection was taken challenging the competency of the appeal. The contention put forward by Sir Alladi Krishnaswami Aiyar, who appeared for the intervener, in substance, is that Art. 136 of the Indian Constitution, under which special leave was prayed for and obtained by the Appellants in this case, does not contemplate or include within its scope an appeal against an award of an Industrial Tribunal which is not vested with, and cannot exercise, judicial powers, and the decision of which cannot, therefore, rank as a judicial determination. The Industrial Tribunal, it is said, is an administrative body exercising quasi-judicial functions and this Court cannot be called upon to exercise the powers of an Appellate Court in respect to the decision of a Tribunal which is really a part of the administrative machinery of the Government.

69. In reply to this objection, it has been urged by Bakshi Sir Tek Chand that the Tribunal constituted under the Industrial Disputes Act is really and in substance, a Court or judicial tribunal which is invested with the power and authority to exercise judicial functions; and in any event, the language of Art. 136 of the Constitution is wide enough to include an appeal from the award or determination of any Tribunal, be it judicial or not.

70. There are two questions which require consideration on this preliminary point. The first is, whether the award or decision of an Industrial Tribunal constituted under the Industrial Disputes Act is a judicial decision in the proper sense of the expression or is the pronouncement of an administrative or quasi judicial body which may exercise some of the functions of a Court of law but is really not so ? The other question turns upon the construction to be put upon Art. 136 of the Constitution particularly on the meaning to be given to the words "tribunal" and "determination" occurring therein; and the question is whether the language is wide enough to include an adjudication or award of an Industrial Tribunal.

71. As regards the first question, it is to be noticed that owing to the intricate and complex system of Government that exists in a modern State and the vast expansion of social legislation of all sorts that have taken place in England and in other countries including our own, within the last few decades, the so-called administrative and quasi judicial Tribunals have come to be a permanent feature of out social and political system. They function as adjudicating bodies in disputed concerning a large number of economic and social affairs. In a sense they are governmental bodies appertaining to the executive and not to the judicial branch of the State, though in various matters they are armed with judicial powers analogous to those normally carried out by Courts of law. The question is, what are the tests or distinguishing features, if any, which distinguish an administrative Tribunal from a Court of law. Once we are able to formulate these tests, we would be in a position to determine whether a Tribunal functioning under the Industrial Disputes Act is or is not a judicial Tribunal properly so called.

72. Whether a particular function or activity is judicial or not is often a difficult question to decide. The point was elaborately dealt with by Lord Sankey who delivered the judgment of the Privy Council in Shell Co. of Australia v. Federal Commissioner of Taxation. The question raised in that case was whether the Board of Review, which was set up in 1925 under the Commonwealth Income Tax legislation, was a Court exercising Judicial powers by a majority that it was an administrative and not a judicial Tribunal and this majority judgment was affirmed in course of this judgment that "the decided cases show that there are Tribunals which possess many of the trappings of a court but which, nevertheless, are not courts in the strict sense of exercising judicial power. Mere externals do not make a direction by an ad hoc tribunal to an administrative officer, an exercise by a Court of judicial power."

73. The actual decision in the case rested on the ground that the Board of Review could not be a judicial tribunal, as its orders were not conclusive for any purpose whatsoever. The decision, is seems, has only a negative value. The Lord Chancellor enumerated a series of negative propositions which stated inter alia that a tribunal is not necessarily a Court because two or more contending parties appear before it, nor because it hears witness, or gives a final decision which affects the right of the parties. What the real or positive test is, the Privy Council did not care to formulate, though the judgment quoted, with approval, certain observations of Griffith, C. J., given in another Australian case, namely, Huddart Parker and Co. v. Moorehead which to some extent neutralised the effect of the negative tests enumerated in the judgment. The observations of Griffith, C.J., are as follows :-

"I am of opinion that the words 'judicial power'.... mean to power which every sovereign authority must have to necessity to decide controversies between its subjects, or between itself and its subjects, whether the rights relate to life, liberty or property. The exercise of this power does not begin until some tribunal which has power to give a binding and authoritative decision (whether subject to appeal or not) is called upon to take action."
74. It may be stated that the authority to hear and decide on evidence between a proposal and an opposition though it is one of the most essential of judicial powers, may be present in an administrative tribunal also. In the majority of cases, administrative bodies are also armed with the powers of a Court of justice in summoning witnesses, administering oaths and punishing disobedience to its order made for the purpose of effecting its enquiries. Vide W. F. O'Conner v. Waldron. As a matter of fact, it is usual to find that those features which were at one time attached exclusively to activities carries on in a Court of law are being extended to committees, commissions or boards conducting enquiries under directions or supervision of the Government. The presence or absence of these features, therefore, does not furnish any conclusive test to determine whether a particular body is a judicial body or not. In the observations of Griffith, C.J., quoted above, the learned Chief Justice laid stress on the power to make a binding and authoritative decision as the essential element in the exercise of judicial power. The exact meaning and implication of these expressions were the subject-matter of discussion in later Australian cases and it was held by the majority of the Judges in Rola Co. (Australia) Pty. Limited v. The Commonwealth that they do not simply mean that if an authority is given power to decide controverted questions of fact and its determination is made binding on the parties to the controversy, if would be sufficient to show that judicial power was entrusted to such authority. A determination, it was pointed out, may be binding on the parties in the same sense as contract is binding on them. What is necessary is that the determination by its own force and without the aid or instrumentality of any other authority or power must affect the rights and obligations of the parties; or in other words, the decision itself irrespective of the facts decided, must create rights and impose obligations; and it should be enforceable as such under the ordinary law of the land. This undoubtedly is one of the fundamental tests which distinguishes a judicial body from one which exercises administrative or quasi judicial functions. Sometimes the decision or report of the administrative tribunal becomes operative after it is accepted by the head of the department under which the tribunal conducted its enquiries and it is then enforced by some sort of administrative process; or it might create rights between the parties which have to be sued upon in the ordinary way in a Court of Law and it is only on the basis of a judgment or decree that is obtained in such action that relief could be had by the party. The essence of judicial determination is that nothing further remains to be done except the enforcement of the judgment, a step which is compelled automatically by the law of the land.

75. The other fundamental test which distinguishes a judicial from a quasi judicial or administrative body is that the former decides controversies according to law, while the latter is not bound strictly to follow the law for its decision. The investigation of facts on evidence adduced by the parties may be a common feature in both judicial and quasi judicial tribunals, but the difference between the two lies in the fact that in a judicial proceeding the judge has got to apply to the facts found, the law of the land which is fixed and uniform. The quasi judicial tribunal on the other hand gives its decision on the differences between the parties not in accordance with fixed rules of law but on principles or administrative policy or convenience or what appears to be just and proper in the circumstances of a particular case. In other words, the process employed by an administrative tribunal in coming to its decision is not what is known as "judicial process." (See Robson's Justice and Administrative Law, p. 74.) Sir Maurice Gwyer in his deposition before the Committee on Minister's Powers appointed by the English Parliament in 1929 stated that "a clear distinction is to be drawn between judicial and quasi judicial powers." The "judicial power" was defined by the witness as a power to decide a question of legal right in a dispute between parties involving either a finding of fact or the application of a fixed rule or principle of law or involving both."The quasi judicial power," he defined as meaning "the power of giving decisions on questions of differences of an administrative and not justiciable character which cannot be determined by reference to any fixed law or principle of law but are matters of administrative discretion and judgment." (Vide Committee of Minister's Powers, Minutes of Evd. Vol. II, pages 15-16 and also Robson's Justice and Administrative Law, p. 319). In Cooper v. Wilson, Scott, L. J., quoted with approval and adopted as the basis of his judgment the following passage from the report of the above committee :

"A true judicial decision presupposes an existing dispute between two or more parties, and then involves four requisites :- (1) The presentation (not necessarily orally) of their case by the parties to the dispute; (2) if the dispute between them is a question of fact, the ascertainment of the fact by means of evidence of adduced by the parties to the dispute and often with the assistance of argument by or on behalf of the parties on the evidence; (3) if the dispute between them is a question of law, the submission of legal argument by the parties to the dispute and often with the assistance of argument by a finding upon the facts in dispute and an application of the law of the land to the facts so found, including where required a ruling upon any disputed question of law. A quasi judicial decision equally presupposes an existing dispute between two or more parties and involves and but does not necessarily involve and never involves. The place of is in fact taken by administrative action, the character of which is determined by the Minister's free choice."
76. In our opinion these statement correctly bring out the distinction between a judicial tribunal and an administrative body which exercises quasi judicial functions. These being the essential features which distinguish the two classes of tribunals, we would have to ascertain with reference to the provisions of the Industrial Disputes Act, which class or category of tribunals an Industrial Tribunal comes under.

77. The object of the Industrial Disputes Act, as set out in the preamble, is "to make provisions for investigation and settlement of industrial disputes and for certain other purposes hereinafter appearing." The word "settlement" suggests the idea of establishing compromise between the interest of disputing parties.

78. There are three classes of authorities provided for by the Act who are entrusted with the powers and duties of investigation and settlement of industrial disputes. First of all, there are conciliation officer of Boards of Conciliation, whose duties mainly are to induce parties to come to a fair and amicable settlement amongst themselves. Secondly, there are Courts of Enquiry and though they are described as Courts, their duties end with investigation into the matters referred to them and submitting reports thereupon to the appropriate Government. Lastly, there are Industrial Tribunals, composed of independent persons who either are or had been Judges of a High Court or District Judges or are qualified for appointment as High Court Judges.

79. It will be seen from the description given above that the Board of Conciliation or Court of Enquiry Disputes Act could, on no account, be regarded as judicial tribunals. To enable them to investigate facts they are, however, armed with certain powers of compelling attendance of witnesses and production of documents, etc. These provisions are to be found in section 11 of the Act. The significant thing to note is, that there is no distinction made in this respect between Conciliation Boards and Courts of Enquiry on the one hand and Industrial Tribunals on the other. The came power are conferred on the three classes of authorities without any distinction whatsoever and sub-section (3) of the Section 11 further lays down that any enquiry or investigation by a Board, Court of Enquiry or Tribunal shall be deemed to be judicial proceeding within the meaning of secs. 193 and 228 of the Indian Penal Code. This means that proceedings before an Industrial Tribunal or for the matter of that before the other two bodies could be deemed to be judicial proceedings only for certain specified purposes. The express provision making the proceedings judicial proceedings for those purposes only emphasises that they are not judicial proceedings otherwise.

80. Under section 15(1), the Industrial Tribunal has got to submit its award to the appropriate Government and sub-section (2) lays down that on receipt of such an award, the appropriate Government shall by order in writing declare the award to be binding. A different provision has been made in regard to cases where the Government itself figures as a party to the dispute. In such case, if the Government considers it inexpedient on public grounds to give effect to the award either in whose or in part, it may, at the earliest opportunity, lay the award for consideration before the Provincial or Central Legislative Assembly as the case may be and the Legislative Assembly may by its resolution confirm, modify or reject the award. After the resolution is passed, the Government is to declare the award so confirmed or modified to be binding [see sub-section (3)]. Sub-section (4) of section 15 expressly lays down that an award declared to be binding under any two of the previous sub-sections shall not be called into question in any manner whatsoever. The Government is not merely to declare the award binding but under section 19(3), if has got to specify the date when the award would come into force and also to fix the period during which it would remain binding, and this period shall not exceed one year.

81. It will be seen, therefore, that there is nothing in the Industrial Disputes Act from which it could be inferred that the Industrial Tribunal really functions as Court exercising judicial functions. Regarding the trappings or the external indicia of a Court, its position is almost the same as that of the Board of Conciliation or court of Enquiry and Bakshi Sir Tek Chand concedes that the latter are not Judicial Tribunals at all. The powers of an Industrial Tribunal are certainly wider than those of the other bodies, but it has no power to make a final pronouncement which would proprio vigore be binding on, and create rights and obligations between the parties. It is for the appropriate Government to declare the award to be binding and the part which the Government plays in such matters is not a mechanical part merely, for the award can really become operative only when the date of its commencement and the period of its duration are fixed, and it is for the Government and Government alone to fix the same. With regard to other class of cases, where the Government itself is one of the parties to the dispute, the position is still worse. An award in such cases is always subject to the contingency of being rejected or modified by the legislature before whom it could be placed for consideration at the option of the Government. Where a contingency like this is attached to an award, it can never be regarded as a final or binding decision which is of the essence of a judicial proceeding. The fact that in cases of disputes between private employers and their workmen, the Government has to accept the award as it is, make no difference in principle. Possibly, this rule was made in consideration of the status and training of the people who constitute the Tribunal, but nevertheless the determination cannot acquire any authority or force, so long as the appropriate Government does not make the declaration and fix the time of its operation as mentioned above. In regard to the other class of awards, where the Government is one of the disputing parties, the award on the face of it is neither the final nor the authoritative pronouncement on the matter in dispute, and it is always in the powers of one of the disputing parties to subject it to further scrutiny at the hands of the legislature who can reject the whole award or effect such change in it as to considers proper. This shows the real nature of the Tribunal and it is not and could not be suggested that the Industrial Tribunal which exercises judicial functions when the dispute is only between private employers and their workmen, and it ceases to be such when the employer is the Government itself.

82. We would now examine the process by which an Industrial Tribunal comes to its decisions and I have no hesitation in holding that the process employed is not judicial process at all. In setting the disputes between the employers and the workmen, the function of the Tribunal is not confined to administration of justice in accordance with law. It can confer rights and privileges on either party which it considers reasonable and proper, through they may not be within the terms of any existing agreement. It is not merely to interpret or give effect to the contractual rights and obligations of the parties. It can create new rights and obligations between them which it considers essential for keeping industrial peace. An industrial dispute as has been said on many occasions is nothing but a trial of strength between the employers on the one hand and the workmen's organisation on the other and the Industrial Tribunal has got to arrive at some equitable arrangement for averting strikes and lock outs which impede production of goods and the industrial development of the country. The Tribunal is not bound by the rigid rules of law. The process it employs is rather an extended form of the process of collective bargaining and is more akin to administrative than to judicial function.

83. In describing the true position of an Industrial Tribunal in dealing with Labour disputes, this Court in Western India Automobile Association v. Industrial Tribunal, Bombay and others quoted with approval, a passage from Ludwig Teller's well known work on the subject, where the learned author observes that "industrial arbitration may involve the extension of an existing agreement or the making of new one or in general the creation of new obligation or modification of old ones, while commercial arbitration generally concerns itself with interpretation of existing obligations and disputes relating to existing agreement." The views expressed in these observations were adopted in its entirety by this Court. Out conclusion, therefore, is that an Industrial Tribunal formed under the Industrial Disputes Act is not a judicial tribunal and its determination is not a judicial determination in the proper sense of these expressions.

84. We now come to the other question as to whether an appeal could be taken to this Court against an award of an Industrial Tribunal by special leave under Article 136 of the Constitution. Article 136 is a part of Chapter IV of the Constitution which deals with the Union Judiciary. The different jurisdictions of the Supreme Court have been prescribed in a series of articles commencing from Article 131. Art. 131 defines the original jurisdiction of the Supreme Court. Art. 132 deals with its appellate powers in cases where substantial question of law as to the interpretation of the Constitution are involved. Article 133 contains the provision relating to appeals in civil cases from judgments, decrees and orders of the High Courts; and Article 134 makes provisions relating to criminal appeals. Article 135 lays down that the Supreme Court shall have jurisdiction and powers with respect to any matter not covered by Articles 133 and 134, if such jurisdiction and power could have been exercised by the Federal Court prior to the coming into force of the present Constitution. Then comes Article 136 which runs as follows :-

"(1) Notwithstanding anything in this chapter the Supreme Court may, in its discretion, grant special leave to appeal from any judgment, decree, determination, sentence or order in any cause or matter passed or made by any Court or tribunal in the territory of India."
85. The article is worded in the widest terms possible. It vests in the Supreme Court a plenary jurisdiction in the matter of entertaining and hearing appeals by granting of special leave against any kind of judgment, decree or order made by any Court or tribunal in any cause or matter and the powers could be exercised in spite of and overriding the specific provisions for appeal contained in the previous Articles. The controversy so far as the present case is concerned mainly centers round the interpretation to be put upon two words, namely, "determination" and "tribunal" used in the Article. Does the word "tribunal" mean a judicial tribunal only and is the expression "determination" restricted to what is known as "judicial determination"?

86. Sir Alladi's contention is that in interpreting these words we should follows the principle of ejusdem generis."Determination", he says, "must be taken to be judicial determination which is of the same nature as decree, judgment, order or sentence; and "tribunal" associated with the word "Court" could not but mean "judicial tribunal".

87. Bakshi Sir Tek Chand on the other hand lays stress on the fact that the word "determination" was not in the original draft Constitution, and it was subsequently added presumably with a view to widen the scope of Article 136 and include within it, the decisions of administrative and quasi judicial tribunals also. He points out that according to the definition given section 2(b) of the Industrial Disputes Act, "award" means a determination, either interim or final, of an industrial dispute by an Industrial Tribunal.

88. There is undoubtedly something to be said in favour of both these views. The difficulty, in our opinion, arises from the fact that neither of these terms "determination" or "tribunal" has a fixed or definite connotation in ordinary language. The word "determination" means and signifies the ending of a controversy or litigation by the decision of a Judge or Arbitrator. It cannot be said that it is restricted exclusively to proceedings in Court. Likewise, the dictionary meaning of the word "tribunal" is "Court of justice" or "seat of a Judge". By "Judge" we mean some authority by which contested matters are decided between rival parties. Here again, it is not possible to say that the expression is applicable only to a regular Court of law. If the tribunal is a full-fledged judicial tribunal, it is not disputed that its decisions would be proper subject-matter or appeal under Article 136 of the Constitution. The question is whether this Article includes within its scope the determinations of quasi judicial tribunal as well.

89. Our view is that ordinarily we should not put any restricted interpretation upon the plain words of an Article in the Constitution and thereby limit out powers of granting special leave for appeals, which the Constitution for best of reason did not choose to fetter or circumscribe in any way. At the same time, we must admit that some sort of restricted interpretation may be unavoidable in view of the context in which particular words appear; and certain restrictions may be implicit in the very purpose for which Article 136 has been framed. Article 136 empowers us in our discretion to hear appeals from pronouncements of all inferior Courts and tribunals. With regard to law Courts, no difficulty arises. As regards tribunals which are not Courts in the proper sense of the expression, it may not be proper, in our opinion, to lay down a hard and fast rule that no appeals could, on any account, be allowed against determinations of such tribunals. There are numerous varieties of these adjudicating bodies, whose structures vary greatly in character and composition and so do the powers and functions which they exercise. The best thing to do would be to examine each type of cases as it arises and if we find that with regard to determinations emanating from certain tribunals it is not possible for us to exercise fully and effectively the powers of an Appellate Court, such determinations must be held to lie outside the purview of Article 136 of the Constitution.

90. This disability in the matter of exercising our powers as an Appellate Court might arise from the fact that the rules and principles by which we ordinarily judge the soundness or otherwise of judicial decisions are not capable of being applied to the determinations of certain administrative tribunals. It might also arise from the fact that the law under which the tribunal functions prevents us from making any effective order which would be binding and operative of its own force without the intervention of some other power or authority; or there may be some kind of contingency attached to it. In our opinion, these difficulties do confront us in the entertaining or hearing of an appeal against the decision of an Industrial Tribunal. In the first place, as we have said above the determination of an Industrial Tribunal does not become complete and binding unless and until it is declared to be so by the appropriate Government. Till the Government makes such declaration, neither of the parties to the dispute can have any real reason for filing an appeal. An, appeal, if it lies, could be filed after the determination has been declared binding. But in such cases, is it the determination of the tribunal merely which is challenged by way of appeal or is it the determination by the tribunal to which has been super-added a declaration by the Government. The decision in the appeal would undoubtedly affect not merely the decision of the tribunal merely which is challenged by way of appeal of is it the a tribunal within the meaning of Article 136. Assuming again that the award is set aside and we substitute out own determination in place of the award given by the tribunal, will our award be enforceable by itself or will it require a declaration by the Government to make it binding ? If Government is itself a party to the dispute, will it be open to Government to place out decision for consideration by the Legislative Assembly ? And will the Legislative Assembly be competent to reject or modify our award ? These problems arise because under section 15 the award under the Act becomes binding only when the Government declares it to be so and if our judgment takes the place of the award of the tribunal, all the infirmities that attach to the award must necessarily attach to out judgment also.

91. The other difficulty is no less formidable. As said above, the tribunal is not bound to decide the disputes by application of the ordinary law of the land. A good deal depends upon questions of policy and public convenience. It is not possible for us to judge the propriety of the decision by the reference to some standard or fixed rules and we think that the very policy of the law prevents us from interfering with the discretion exercised by the tribunal.

92. Where the discretion is committed to any body or a tribunal exercising quasi judicial functions which are not fettered by ordinary rules of law, the tribunal should in the absence of any provision to the contrary be deemed to have the final authority in the exercise of that discretion. We cannot sit in appeal over their decision and substitute our own discretion for theirs. Questions, however, may and so arise where such quasi judicial body attempts to usurp jurisdiction which it does not possess. It may assume jurisdiction under a mistaken view of law or refuse to exercise jurisdiction properly by adoption of extraneous or irrelevant considerations; or there may be cases where in its proceedings the tribunal violates the principles of natural justice. In all such cases the most proper and adequate remedy would be by writs of certiorari or prohibition and the Court having authority may direct that the decision of the body or tribunal might be brought up to the quashed for lack of jurisdiction or for mistake apparent on the face of it; and if the proceedings had not terminated at that time, a writ prohibition may also be issued for preventing the tribunal from exceeding its jurisdiction. The issuing of such writs would not be an exercise of appellant powers which means the rehearing of the case and passing of such judgment which of the opinion of the Appellate Court the original tribunal should have made. The object of these writs is simply to keep the exercise of powers by these quasi judicial tribunal within the limits of jurisdiction assigned to them by law and to restrain them from acting in excess of their authority. These principles are well settled and require no elucidation, Rex. v. Electricity Commissioner and Board of Education v. Rice. Our conclusion, therefore, is that Article 136 of the Constitution does not contemplate a determination given by the Industrial Tribunal.

93. Even assuming for argument's sake that we have got jurisdiction under Article 136, the exercise of which would depend upon the circumstances of each case, in view of the reasons which we have set out above, this is not an appeal which, in our opinion, should be admitted even if we have the power to do so.

94. The result is that the preliminary objection succeeds and the appeal fails and is dismissed with costs.

Patanjali Sastri, J.

95. I entirely agree and have nothing to add.

96. Agents : Ganpat Rai for the Appellant; R. R. Biswas for the Respondents; P. A. Mehta for the Intervener (The Union of India)."""
# Run summarization
summary = summarize_text(report)
print("Summary:", summary)


Using device: cuda
Initializing TextGraphSummarizer on device: cuda
Summary size set to: 0.8
Running preprocess_text...
Preprocessed words (count): 11571
Running get_embeddings...
Running build_graph...
Total nodes created: 10072
Adding edges to graph...
Total edges created: 94920
Input feature shape: torch.Size([23734, 782])
Edge index shape: torch.Size([2, 94920])
Scores: [-0.06641731  0.08594928  0.01269344  0.09233467  0.00190105  0.1372303
 -0.05227126 -0.08785494  0.06373131  0.19259158 -0.16525862  0.14410943
  0.06336015 -0.06137311  0.0195686   0.09201378  0.10926066  0.03112465
  0.0906647  -0.1085085   0.06101545 -0.10201169 -0.04625694 -0.09518578
  0.19986477 -0.03385781 -0.00096864 -0.06652845 -0.14118478  0.17400202
 -0.04487089 -0.00418433]
Summarization complete.
Summary: supreme court india 
 bharat bank ltd . , delhi vs employees bharat bank ltd . , ... 1 march , 1950 
 equivalent citations : [ 1950]suppscr317 
 bench : h.j


In [50]:
!pip install -U spacy
!python -m spacy download en_core_web_md

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached numpy-2.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached numpy-2.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.5 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.8.3 requires cubinlinker, which is not installed.
cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.3 requires ptxcompiler, which is not installed.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
ucxx 0.39.1 requires libucx>=1.15.0, which is not installed.
apache-beam 2.46.0 requires cloudpickle~=2.2.1, but you have cloudpickle 3.0.0 which is incomp

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 48.6 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [51]:
# Install the required libraries if not already installed
!pip install sentence-transformers

# Import necessary classes
from sentence_transformers import SentenceTransformer

# Verify installation
try:
    model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
    print("SentenceTransformer model loaded successfully.")
except Exception as e:
    print("Error loading SentenceTransformer:", e)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


SentenceTransformer model loaded successfully.


In [52]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, AttentionalAggregation
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import spacy
from transformers import BertModel, BertTokenizer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

class TextGraphSummarizer:
    def __init__(self, summary_size=0.1):
        print(f"Initializing TextGraphSummarizer on device: {device}")
        self.nlp = spacy.load("en_core_web_md")
        self.summary_size = summary_size
        self.important_legal_terms = {"judgment": 10.0, "plaintiff": 8.0, "defendant": 8.0, "petition": 6.0}
        self.model = BertModel.from_pretrained("bert-base-uncased").to(device)
        self.tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
        self.pos_tags = ["ADJ", "NOUN", "VERB"]

    def preprocess_text(self, text):
        print("Running preprocess_text...")
        doc = self.nlp(text.lower())
        words = [token.text for token in doc if not token.is_stop]
        print(f"Preprocessed words (count): {len(words)}")
        return words, doc

    def get_embeddings(self, words):
        print("Running get_embeddings...")
        inputs = self.tokenizer(words, padding=True, truncation=True, return_tensors="pt").to(device)
        with torch.no_grad():
            embeddings = self.model(**inputs).last_hidden_state.mean(dim=1)
        return embeddings

    def build_graph(self, words, doc, embeddings):
        print("Running build_graph...")
        G = nx.Graph()

        # Step 1: Initialize TF-IDF dictionary for words
        tfidf_vectorizer = TfidfVectorizer()
        tfidf_scores = tfidf_vectorizer.fit_transform([' '.join(words)])
        feature_names = tfidf_vectorizer.get_feature_names_out()
        tfidf_dict = {word: tfidf_scores[0, idx] for word, idx in zip(feature_names, range(tfidf_scores.shape[1]))}
        tfidf_dict_gpu = {word: torch.tensor(tfidf_score, dtype=torch.float32).to(device) for word, tfidf_score in tfidf_dict.items()}

        feature_size = embeddings.shape[1] + 10 + 1  # Embedding size + entity type + combined weight
        expected_feature_size = 782

        # Add nodes to the graph
        for i, token in enumerate(doc):
            if not token.is_alpha or i >= len(embeddings):
                continue

            # Entity type embedding
            entity_type = torch.zeros(10, dtype=torch.float32).to(device)
            if token.ent_type_:
                entity_type[hash(token.ent_type_) % 10] = 1.0

            # Get tf-idf and legal weights
            tfidf_weight = tfidf_dict_gpu.get(token.text.lower(), torch.tensor(1.0, dtype=torch.float32).to(device))
            law_weight = torch.tensor(self.important_legal_terms.get(token.text.lower(), 1.0), dtype=torch.float32).to(device)
            combined_weight = tfidf_weight * law_weight

            # Create node feature vector
            node_features = torch.cat([embeddings[i], entity_type, combined_weight.unsqueeze(0)])

            # Adjust to expected feature size by padding/truncating
            if node_features.size(0) < expected_feature_size:
                padding = torch.zeros(expected_feature_size - node_features.size(0), dtype=torch.float32).to(device)
                node_features = torch.cat([node_features, padding])
            elif node_features.size(0) > expected_feature_size:
                node_features = node_features[:expected_feature_size]

            # Add node to graph
            assert node_features.size(0) == expected_feature_size, f"Node feature size mismatch: {node_features.size(0)}"
            G.add_node(i, x=node_features)

        # Edge creation with validation
        edge_attributes = {}
        sentences = list(doc.sents)

        for i, sent1 in enumerate(sentences):
            for j, sent2 in enumerate(sentences):
                if i < j:
                    similarity = sent1.similarity(sent2)
                    if similarity > 0.3:
                        edge_key = (sent1.start, sent2.start)
                        edge_attributes.setdefault(edge_key, {})
                        edge_attributes[edge_key]['similarity'] = similarity

        # Position proximity edges
        for i in range(len(sentences) - 1):
            edge_key = (sentences[i].start, sentences[i + 1].start)
            edge_attributes.setdefault(edge_key, {})
            edge_attributes[edge_key]['position_proximity'] = 0.3

        valid_edges = 0
        for (u, v), attrs in edge_attributes.items():
            if isinstance(attrs.get('similarity', 0.0), float) and \
               isinstance(attrs.get('position_proximity', 0.0), float):
                G.add_edge(u, v, **attrs)
                valid_edges += 1

        print(f"Total nodes created: {len(G.nodes)}")
        print(f"Valid edges added: {valid_edges}")

        return G

    def build_pyg_data(self, G):
        max_index = max(G.nodes)
        num_nodes = max_index + 1
        node_features = torch.zeros(num_nodes, 782).to(device)

        for node in G.nodes:
            if 'x' in G.nodes[node]:
                feature = G.nodes[node]['x']
                if feature.size(0) < 782:
                    feature = torch.cat([feature, torch.zeros(782 - feature.size(0)).to(device)])
                elif feature.size(0) > 782:
                    feature = feature[:782]
                node_features[node] = feature
            else:
                node_features[node] = torch.zeros(782).to(device)

        edge_index = torch.tensor(list(G.edges)).t().contiguous()
        edge_attr = torch.ones(edge_index.size(1)).to(device)
    
        return Data(x=node_features, edge_index=edge_index, edge_attr=edge_attr)

class GNNTextSummarizer(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNNTextSummarizer, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.attention = AttentionalAggregation(nn.Linear(hidden_dim, 1))
        self.output_layer = nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.attention(x)
        x = self.output_layer(x)
        
        return x

def summarize_text(report):
    summarizer = TextGraphSummarizer(summary_size=0.3)
    print(f"Summary size set to: {summarizer.summary_size}")

    words, doc = summarizer.preprocess_text(report)
    sentences = list(doc.sents)
    embeddings = summarizer.get_embeddings(words)
    G = summarizer.build_graph(words, doc, embeddings)
    data = summarizer.build_pyg_data(G).to(device)

    input_dim = 782
    model = GNNTextSummarizer(input_dim=input_dim, hidden_dim=64, output_dim=1).to(device)
    
    model.eval()
    with torch.no_grad():
        scores = model(data).cpu().numpy().flatten()

    sentence_scores = []
    for i, sentence in enumerate(sentences):
        if sentence:
            score = np.mean([scores[token.i] for token in sentence if token.i < len(scores)])
            sentence_scores.append((i, score, sentence.text))

    num_selected = max(1, int(len(sentence_scores) * summarizer.summary_size))
    selected_sentences = sorted(sentence_scores, key=lambda x: x[1], reverse=True)[:num_selected]
    selected_sentences = sorted(selected_sentences, key=lambda x: x[0])
    summary = ' '.join([text for _, _, text in selected_sentences])

    print("\nSummarization complete.")
    return summary, selected_sentences



report = """COMMISSIONER, TRADE TAX, U.P., LUCKNOW v. M/S. SUMAN TRADING CO. [2015] INSC 4 (6 January 2015)
'REPORTABLE' IN THE SUPREME COURT OF INDIA CIVIL APPELLATE JURISDICTION CIVIL APPEAL NO. 28 OF 2015 [Arising out of Special Leave Petition (C) No.32616 of 2013] M/s. Pepsico India Holding Pvt. Ltd. ... Appellant(s) versus Krishna Kant Pandey ...Respondent(s)
M.Y.EQBAL,J.
1\. Leave granted.

2\. This appeal by special leave is directed against judgment and order dated 23.5.2013 of the High Court of Allahabad at Lucknow Bench whereby learned Single Judge classifying the respondent as 'workman' allowed the writ petition preferred by him, quashed the order dated August 24, 2007 passed by the Industrial Tribunal II, State of Uttar Pradesh, Lucknow (in short, 'the Tribunal') and directed the Tribunal to decide respondent's Case No.84/2004 on merit.

3\. The factual matrix of the case is that the respondent was appointed on the post of Operator/Technician Grade III for six months on probation basis w.e.f. 13th of March, 1995 against the salary of Rs. 2600/- per month.
Having been found his services satisfactory, he was confirmed w.e.f. 13th September, 1995 and was also awarded one increment w.e.f. 1st of February, 1996. Earlier, he was appointed in the Plant of Jainpur (Kanpur Dehat) from where he was transferred to Sathariya Plant, District Jaunpur, U.P. on 30th of August, 1996 on the revised pay scale i.e. Rs. 5450/-. Pursuant to the subsequent transfer order, he was posted at Lucknow in the month of June, 1997 and till 2000 he was awarded annual increments at the rate of Rs.490/-. Subsequently, he was promoted to the post of Line Supervisor in the pay scale of Rs.7716/- and thereafter to the post of Fleet Executive.

4\. It is the case of the respondent that being posted as a Fleet Executive, he was to discharge the mechanical work and that being so, he was called as skilled workman. It is stated that no other staff was posted in his subordination. The respondent also pointed out the conduct of the employer transferring him from one place to another and also compelling him to resign from the post or to be on long leave. On being asked to proceed on leave, respondent remained on leave w.e.f. 9th October, 2003 to 17th October, 2003. When he turned up, he was not permitted to join for want of instructions of the superior authorities. Thereafter, respondent wrote a letter on 8th November, 2003 to the Vice President seeking guidance for further action, upon which the employer became unhappy and terminated his services on 14th of November, 2003 by giving one month's salary in lieu of notice prior to termination.

5\. Aggrieved by the said termination, respondent preferred a reference before the Conciliation Officer, Lucknow alleging that he is a 'workman' within the meaning of the Uttar Pradesh Industrial Disputes Act, 1947 (hereinafter referred to as the 'Act') and termination of his services by the Company is contrary to Section 6 of the Act. The appellant Company pleaded that the respondent did not satisfy the criteria of a workman as defined under Section 2(z) of the Act. The Industrial Tribunal dismissed the reference stating that the respondent is not a workman under Section 2(z) of the Act and, therefore, no challenge to the termination is maintainable before the Tribunal.

6. Respondent, being aggrieved, moved the High Court by way of a writ petition challenging the order of the Tribunal and also for his re- instatement to the post of Fleet Executive with continuity of service and for payment of full back wages. In reply, the appellant pleaded that the order of termination is in accordance with the provisions of the Act.
After hearing learned counsel on either side, learned Single Judge of the High Court allowed writ petition of the respondent, quashed order of the Tribunal and directed it to proceed with the adjudication of the respondent's case on merit. Hence, the present appeal by special leave by the appellant-Company.

7\. Mr. C.U. Singh, learned counsel appearing for the appellant, assailed the order passed by the High Court on various grounds inter alia, the High Court has exceeded its jurisdiction conferred upon it under Article 226 of the Constitution of India by reversing the finding recorded by the Tribunal. Learned counsel submitted that the High Court has committed grave error in issuing suo motu directions to the executive to amend a relevant provision of Section 2(z) of the U.P. Industrial Disputes Act (in short, 'State Act'). According to the learned counsel, issuing such direction by the High Court amounts to issue a direction to the legislature to enact a law in a particular manner. Learned counsel submitted that the High Court cannot mandate the executive to introduce or enact a legislation, howsoever necessary or desirable. Learned counsel drew our attention to the provisions of Section 2(z) of the said Act which defines 'workman' and submitted that the High Court has failed to appreciate that the nature of duties and responsibilities entrusted upon the respondent are not manual, skilled or unskilled or technical services, but manual, managerial and supervisory. In the capacity of Fleet Executive, respondent was required to monitor each and every vehicle of the Fleet and ensure that the necessary repair proceedings were carried out. Learned counsel further submitted that the High Court has misdirected itself in considering the relevant facts and pleadings which were not even placed before the Industrial Tribunal. Lastly, it was contended that the High Court in exercise of its jurisdiction under Article 226 of the Constitution ought not re-appreciate or re-weigh evidence and disturb the finding of facts recorded by the Tribunal based on appreciation of evidence. Learned counsel relied upon the judgment of this Court in the case of Chandavarkar Sita Ratna Rao vs. Ashalata S. Guram[1986] INSC 197; , (1986) 4 SCC 447, Birla Corpn. Ltd..
vs. Rajeshwar Mahato and Others, (2001) 10 SCC 611 and S.K. Mani vs.
M/s. Carona Sahu Company Limited and Ors., [1994] INSC 165; (1994) 3 SCC 510.

8\. Mr. Kavin Gulati, learned senior counsel appearing for the respondent firstly contended that before conciliation, the appellant raised an objection with regard to the jurisdiction of the Tribunal and the matter was finally referred to the Labour Court for adjudication on a limited question as to whether the termination of services of the respondent was justified. According to the learned counsel, the appellant-management cannot raise the question of jurisdiction of the Labour Court. Learned counsel referred to Rule 12 of the Industrial Disputes Rules in support of his contention and relied upon the decision of this Court in the case of M/s. U.P. Electric Supply Co., Ltd. vs. The Workmen of M/s. S.N. Choudhary Contractors and Anr., (1960) 3 SCR page 189. Mr. Gulati, learned senior counsel also relied upon the decision of this Court in the case of TISCO Ltd. v. State of Jharkhand, (2014) 1 SCC 536, and Bhogpur Coop Sugar Mills Ltd. vs. Harmesh Kumar, (2006) 13 SCC 28, for the proposition that the Labour Court has limited jurisdiction to adjudicate the disputes referred to it and not to enter into any other question that may be raised in the reference.

9\. We have heard learned counsel for both the parties and considered the relevant facts and the law applicable thereto. The admitted facts are that at the relevant time, the respondent was working as a Fleet Supervisor and drawing a salary of Rs. 7716/-. Initially, he was appointed as a technician Grade-III in 1995 on the salary of Rs. 2600/- per month and after getting increment his salary was increased. By giving one month's salary, in lieu of the notice, his services were terminated.

10\. It appears that the appellant raised a preliminary objection before the Labour Court that the respondent was not a Labour as defined under Section 2(z) of the said Act and the Deputy Labour Commissioner, Lucknow, who had referred the present case, was not competent for this purpose. The Labour Court recorded the evidence adduced by both the appellant and the respondent and discussed the evidence, and elaborately considered the case of the parties. The Labour Court finally recorded finding that the respondent is not a workman within the meaning of Section 2(z) of the Act.
Paras 13 to 15 of the order are reproduced hereinbelow:- "13. The statement of the Applicant is that although he was given the post in the name of Fleet Executive and he was posted at the warehouse at Lucknow, however practically he was doing the work of skilled manual and as such he fall within the definition as given in Section 2(z) of the Industrial Disputes Act, 1947. According to him his main work was to remove the technical defects at 100% from the fleet. His other works which have been mentioned by the management in their written statement, were secondary. It has also been stated that the written statement of the management is not on affidavit, therefore the same cannot be relied upon.
He was executing all his work in accordance with the directions of the higher officials. He was not having any right of doing the work as per his own wishes. He has stated in his statement that no staff was working under him. He used to do the work himself for keeping the vehicles 100% free/available from technical deformities and removed the difficulties of the vehicles. It has also been stated that there is a difference in the statement of witnesses of the management EW-1 and EW-2 regarding the repair of the work of the company and the same cannot be relied upon. It has been stated while relying upon the legal arrangement given by the Hon'ble Management Staff Association and others [1970] INSC 235; (1970) 3 SCC 378 that any personnel fall within the definition of labour or not, it depends upon the fact that what is the nature of the main works being done by him. The same cannot be assessed with the name of the post. If the concerned person is doing the work of manual skilled unskilled work, then he is in the definition of labour, as the main work of the Applicant was to repair the vehicles, which he used to do from his own hands. No other person was working under him and he was not having the right to take decision by himself. Therefore, he falls within the definition of Labour. It has been stated while referring (supra) that the work of the Applicant was similar to fupelling superintendent, which has been considered by the Hon'ble Apex Court as labour, because his work was manual and not supervisory. Therefore he falls within the definition of labour and the termination of service made by the management is retrenchment, which has been done in violation of the provisions of Section 60N of the U.P. Industrial Disputes Act, 1947.
Therefore his termination of service is improper and illegal. On this basis he may be reinstated in service alongwith salary for leave period.

14\. It has been argued on behalf of Management that out of the works executed by the Applicant on the post of Fleet Executive, the details of the paid work are mentioned in their written statement. The same has been admitted by the Applicant in his arguments. In this manner, the details regarding the main work out of the works on the post of Fleet Executive being done by him is proved. All these works are supervisory in nature.
The major work of repair of the vehicle used to be done from outside. The work of washing and cleaning of the vehicles was done by driver. The Applicant has never done any type of repairing work and neither anybody has seen the Applicant while doing such work. In this manner mainly the Applicant was doing the work of supervisory nature. Therefore does not fall within the definition of labour. His services have been terminated under the terms. In this manner the order of termination of his service is proper and legal. He is not entitled to get any relief.

15\. The main question in this industrial dispute is whether the Applicant K.K. Pandey is a labour, as claimed by him, as this claim has been made by him and as such onus to prove the same lies on him. According to para 11 and 12 of his written statement he was having only one responsibility on the post of fleet executive that he was to ensure the availability of the vehicles free from technical deformities. According to the written statement for this work nobody was working under him and he used to do the work of repair with his own hands. He has reiterated this fact in his statement also. In this regard except his statement has not produced any evidence to confirm the same. On the other hand he has admitted in his arguments as regards the details of different works mentioned by the management in para No.1 of their written statement. According to it out of his works, there is a detail of 15 main works. In this manner the statement made by the Applicant regarding his main work remained rebutted.
The statement of the Applicant regarding the post of Fleet Executive on the basis of which he is claiming himself as labour is not liable to be believed."

11\. On the basis of the findings based on elaborate discussions and analyzing the evidence, the Labour Court came to the conclusion that at the relevant time the respondent was working as a Fleet Executive which is supervisory in nature and does not fall within the definition of 'labour' as defined under Section 2(z) of the Act. Hence, he is not entitled to any relief. The respondent challenged the aforesaid award passed by the Labour Court in a writ petition before the High Court. After considering the definition contained in Section 2(z) of the Act and the nature of work assigned to the respondent, the High Court arrived at a conclusion that the nature of work prevalent on the date of termination was as that of a workman. Curiously enough, though the respondent did not come under the definition of workman under Section 2(z) of the Act, the High Court proceeded on the basis that the U.P. Industrial Disputes Act was enacted in 1947 and although the respondent cannot be held to be a workman under the said definition, held that he shall have to be classified as a workman and directed the Government to make amendment in Section 2(z) of the Act excluding some of the clauses. For better appreciation, relevant portion of the order is quoted hereinbelow:- "There is one more exclusion clause in section 2(z) of the Act i.e. Clause (iv) which excludes the employee who being employed in supervisory capacity draws wages exceeding Rs. 500/- per mensem or exercise, either by the nature of the duties attached to the office or by reason of the powers vested in him, functions mainly of a managerial in nature. It is very much obvious that by nature of duties assigned to the petitioner, it cannot be said that he was attached to the office or mainly managerial function was vested with him.
So far as another condition for exclusion from the definition of "workman" viz drawing wages exceeding Rs. 500/- per mensem is concerned, it is not in dispute that the petitioner on the date of retrenchment had been getting more Rs. 500/- mensem. This clause is a part of original form of the definition of "workman" as is provided under section 2 (z) of the Act.
The U. P. Industrial Disputes Act was enacted in 1947. The petitioner was retrenched from service in 2003. The date of his initial appointment is on 13th of March, 1995. By passage of time the GDP growth had been increased in number of times from 1947 to 2003. Therefore, the enhancement in income was a natural corollary, in the light of which, I am of the view that this clause has become unworkable and redundant. Now every workman working in the Industry definitely would have been getting more than Rs. 500/- per mensem and if this clause is permitted to be prevailed no workman shall be governed under the definition of "workman". Therefore, I am of the view that this clause has lost its significance and if the employee is covered under the definition of "workman" as is defined under Section 2 (z) of the U. P. Industrial Disputes Act and further is not covered under the exclusion clause except clause (iv), he shall be classified as "workman".
The clause (iv) of section 2 (z) shall not come in the way of his categorization as "workman".
It is advisable to the State Government to consider to make an amendment in section 2 (z) of the U. P. Industrial Disputes Act, 1947 in general and to exclude the class (iv) from the exclusion in particular.
Since the present petitioner has been classified by this court, as above, under the definition of "workman" the order impugned dated 24th of August, 2007 passed by the Industrial Tribunal II, State of U. P., Lucknow is hereby quashed with the direction to the Tribunal to proceed with the adjudication case No. 82/2004 to adjudicate upon it on merit."

12\. Considering the entire facts of the case and the findings recorded by the Labour Court, prima facie we are of the view that the High Court has exceeded in exercise of its jurisdiction under Articles 226 and 227 of the Constitution of India in interfering with the finding of facts recorded by the Labour Court. It is well settled that the High Court in the guise of exercising its jurisdiction normally should not interfere under Article 227 of the Constitution and convert itself into a court of appeal.

13\. While discussing the power of the High Court under Articles 226 and 227 of the Constitution interfering with the facts recorded by the courts or the tribunal, this Court in the case of Chandavarkar S.R. Rao vs.
Ashalata S. Guram, (supra) held as under :- "17. In case of finding of facts, the court should not interfere in exercise of its jurisdiction under Article 227 of the Constitution.
Reference may be made to the observations of this Court in Bathutmal Raichand Oswal v. Laxmibai R. Tarta where this Court observed that the High Court could not in the guise of exercising its jurisdiction under Article 227 convert itself into a court of appeal when the legislature has not conferred a right of appeal. The High Court [pic]was not competent to correct errors of facts by examining the evidence and reappreciating.
Speaking for the Court, Bhagwati, J. as the learned Chief Justice then was, observed at p. 1301 of the report as follows: (SCC p. 864, para 7) "The special civil application preferred by the appellant was admittedly an application under Article 227 and it is, therefore, material only to consider the scope and ambit of the jurisdiction of the High Court under that article. Did the High Court have jurisdiction in an application under Article 227 to disturb the findings of fact reached by the District Court? It is well settled by the decision of this Court in Waryam Singh v.
Amarnath that the ... power of superintendence conferred by Article 227 is, as pointed out by Harries, C.J., in Dalmia Jain Airways v. Sukumar Mukherjee to be exercised most sparingly and only in appropriate cases in order to keep the subordinate courts within the bounds of their authority and not for correcting mere errors.
This statement of law was quoted with approval in the subsequent decision of this Court in Nagendra Nath Bose v. Commr. of Hills Division and it was pointed out by Sinha, J., as he then was, speaking on behalf of the court in that case:
It is thus, clear that the powers of judicial interference under Article 227 of the Constitution with orders of judicial or quasi-judicial nature, are not greater than the power under Article 226 of the Constitution. Under Article 226 the power of interference may extend to quashing an impugned order on the ground of a mistake apparent on the face of the record. But under Article 227 of the Constitution, the power of interference is limited to seeing that the tribunal functions within the limits of its authority."

14\. In the case of Birla Corpn. Ltd.. vs. Rajeshwar Mahato and Others, (2001) 10 SCC, the question of validity of termination of services of the respondent by the appellant-Corporation was referred to the Industrial Tribunal. On evidence, the Industrial Tribunal found that the duties of the respondent were mainly managerial or administrative. The Tribunal held that the respondent was not a workman and the reference was therefore not maintainable against the decision of the Tribunal. The Tribunal relying on Section 2(s)(iv) (as amended in West Bengal W.B.) held that as the respondent was drawing salary less than Rs.1600/- per month, he had to be regarded as a workman. The Corporation moved this Court against the order of the High Court. This Court while setting aside the decision of the High Court held as under :- "4. It was not in dispute that at the time of the termination of services of Respondent 1, he was receiving Rs 1185 per month by way of salary. The Tribunal recorded the evidence as well as took into consideration documentary evidence which was produced by the parties. On the basis of the evidence which was adduced before it, the Tribunal observed that:
"The main duties of Shri Rajeshwar Mahato were both supervisory and administrative in nature.
In the instant case, Shri Mahato's functions were mainly of a managerial nature. He had control as well as supervision over the work of the jute mill workers working under him."

11\. As we have also noticed hereinabove, the Tribunal had given a categorical finding to the effect that Respondent 1's function was mainly of managerial nature. His duties were both supervisory and administrative and therefore he was regarded as not being a workman. Though the Tribunal did not specifically state so, it is evident that it is because of Section 2(s)(iii) that Respondent 1 was regarded as not being a workman.

12\. Neither the Single Judge nor the Division Bench of the High Court, as we have already noticed, referred to this aspect of the matter. Even assuming that the West Bengal amendment was applicable, that would still not help to hold Respondent 1 as a workman if the finding of the Tribunal with regard to the nature of the duties performed by him, as arrived at by the Tribunal, is not set aside as being frivolous or without any evidence.
As long as the finding of the Tribunal stands, namely, that the respondent was an employee mainly in a managerial or administrative capacity, the award of the Tribunal could not have been set aside. As we have already observed the Single Judge or even the Division Bench could have come to the conclusion that the finding so arrived at by the Tribunal was either frivolous or not based on any evidence. But this aspect of the case was completely overlooked by the High Court. The emphasis of the Single Judge as well as the Division [pic]Bench was only with regard to applicability of the amendment of the State of West Bengal to Section 2(s) of the Industrial Disputes Act. In our opinion, therefore, the High Court erred in allowing on this ground the writ petition filed by Respondent 1. The decision of the High Court is set aside and the writ petition filed therein by the respondent stands dismissed."

15\. In the case of Indian Overseas Bank vs. I.O.B. Staff Canteen Workers' Union and Another, (2000) 4 SCC 245, this Court considered a similar question with regard to the power of the High Court under Article 226 against the findings recorded by the Industrial Tribunal. Reversing the decision of the Single Judge and restoring the fact finding decision of the Tribunal this, Court held :- "17. The learned Single Judge seems to have undertaken an exercise, impermissible for him in exercising writ jurisdiction, by liberally reappreciating the evidence and drawing conclusions of his own on pure questions of fact, unmindful, though aware fully, that he is not exercising any appellate jurisdiction over the awards passed by a tribunal, presided over by a judicial officer. The findings of fact recorded by a fact-finding authority duly [pic]constituted for the purpose and which ordinarily should be considered to have become final, cannot be disturbed for the mere reason of having been based on materials or evidence not sufficient or credible in the opinion of the writ court to warrant those findings, at any rate, as long as they are based upon some material which are relevant for the purpose or even on the ground that there is yet another view which can reasonably and possibly be taken. The Division Bench was not only justified but well merited in its criticism of the order of the learned Single Judge and in ordering restoration of the award of the Tribunal. On being taken through the findings of the Industrial Tribunal as well as the order of the learned Single Judge and the judgment of the Division Bench, we are of the view that the Industrial Tribunal had overwhelming materials which constituted ample and sufficient basis for recording its findings, as it did, and the manner of consideration undertaken, the objectivity of approach adopted and reasonableness of findings recorded seem to be unexceptionable. The only course, therefore, open to the writ Judge was to find out the satisfaction or otherwise of the relevant criteria laid down by this Court, before sustaining the claim of the canteen workmen, on the facts found and recorded by the fact-finding authority and not embark upon an exercise of reassessing the evidence and arriving at findings of one's own, altogether giving a complete go-by even to the facts specifically found by the Tribunal below."

16\. The order of the Tribunal would show that the respondent-workman accepted different works assigned to him which were purely of supervisory and managerial nature. The details of 15 managerial/supervisory works assigned to the respondent have been analyzed by the Tribunal which finally came to the conclusion that the respondent is not a workman within the meaning of Section 2(z) of the Act.

17\. In exercise of its writ jurisdiction, the High Court proceeded initially on the basis that the appellant had entered into service on the post of Operator/Technician Grade-III, which is a technical post and from there he was promoted to different posts including Fleet Executive. The High Court committed grave error in holding that although he is not covered under the definition of workman as defined under Section 2(z) of the Act he shall be classified as a workman. The High Court further exceeded its jurisdiction in advising the Government to make an amendment in Section 2(z) of the Act and to exclude some clauses. The order passed by the High Court cannot be sustained in law.

18\. We, therefore, allow this appeal and set aside the order of the High Court and restore the order passed by the Tribunal. However, we give liberty to the respondent to move the appropriate forum to challenge, in accordance with law, the order of termination passed by the appellant.
"""
summary, selected_sentences = summarize_text(report)
print("Summary:", summary)


Using device: cuda
Initializing TextGraphSummarizer on device: cuda
Summary size set to: 0.3
Running preprocess_text...
Preprocessed words (count): 2838
Running get_embeddings...
Running build_graph...
Total nodes created: 2433
Valid edges added: 14610

Summarization complete.
Summary: commissioner, trade tax, u.p., lucknow v. m/s. suman trading co. [2015] insc 4 (6 january 2015)
'reportable' in the supreme court of india civil appellate jurisdiction civil appeal no. 28 of 2015 [arising out of special leave petition (c) no.32616 of 2013] m/s. pepsico india holding pvt. ltd. ... appellant(s) versus krishna kant pandey ...respondent(s)
m.y.eqbal,j.
1\. leave granted.

 2\. this appeal by special leave is directed against judgment and order dated 23.5.2013 of the high court of allahabad at lucknow bench whereby learned single judge classifying the respondent as 'workman' allowed the writ petition preferred by him, quashed the order dated august 24, 2007 passed by the industrial tribunal i

In [53]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, AttentionalAggregation
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import spacy
from transformers import BertModel, BertTokenizer, T5ForConditionalGeneration, T5Tokenizer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

class TextGraphSummarizer:
    def __init__(self, summary_size=0.1):
        print(f"Initializing TextGraphSummarizer on device: {device}")
        self.nlp = spacy.load("en_core_web_md")
        self.summary_size = summary_size
        self.important_legal_terms = {"judgment": 10.0, "plaintiff": 8.0, "defendant": 8.0, "petition": 6.0}

        # Load Sentence-BERT and Legal-BERT
        self.sentence_model = BertModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2").to(device)
        self.legal_model = BertModel.from_pretrained("nlpaueb/legal-bert-base-uncased").to(device)
        self.sentence_tokenizer = BertTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
        self.legal_tokenizer = BertTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")

    def preprocess_text(self, text):
        print("Running preprocess_text...")
        doc = self.nlp(text.lower())
        words = [token.text for token in doc if not token.is_stop]
        return words, doc

    def get_embeddings(self, words):
    # Truncate or pad words to match model input requirements
        max_length = self.sentence_tokenizer.model_max_length
        words = words[:max_length]

    # Tokenize and get embeddings
        sentence_inputs = self.sentence_tokenizer(words, padding=True, truncation=True, return_tensors="pt").to(device)
        legal_inputs = self.legal_tokenizer(words, padding=True, truncation=True, return_tensors="pt").to(device)

        with torch.no_grad():
            sentence_outputs = self.sentence_model(**sentence_inputs)
            legal_outputs = self.legal_model(**legal_inputs)

        # Use mean pooling
            sentence_embeddings = sentence_outputs.last_hidden_state.mean(dim=1)
            legal_embeddings = legal_outputs.last_hidden_state.mean(dim=1)

    # Concatenate embeddings
        combined_embeddings = torch.cat((sentence_embeddings, legal_embeddings), dim=1)
    
        return combined_embeddings

    def build_graph(self, words, doc, embeddings):
        print("Running build_graph...")
        G = nx.Graph()
        tfidf_vectorizer = TfidfVectorizer()
        tfidf_scores = tfidf_vectorizer.fit_transform([' '.join(words)])
        feature_names = tfidf_vectorizer.get_feature_names_out()
        tfidf_dict = {word: tfidf_scores[0, idx] for word, idx in zip(feature_names, range(tfidf_scores.shape[1]))}
        tfidf_dict_gpu = {word: torch.tensor(tfidf_score, dtype=torch.float32).to(device) for word, tfidf_score in tfidf_dict.items()}
        feature_size = embeddings.shape[1] + 10 + 1  
        expected_feature_size = 782

        for i, token in enumerate(doc):
            if not token.is_alpha or i >= len(embeddings):
                continue
            entity_type = torch.zeros(10, dtype=torch.float32).to(device)
            if token.ent_type_:
                entity_type[hash(token.ent_type_) % 10] = 1.0
            tfidf_weight = tfidf_dict_gpu.get(token.text.lower(), torch.tensor(1.0, dtype=torch.float32).to(device))
            law_weight = torch.tensor(self.important_legal_terms.get(token.text.lower(), 1.0), dtype=torch.float32).to(device)
            combined_weight = tfidf_weight * law_weight
            node_features = torch.cat([embeddings[i], entity_type, combined_weight.unsqueeze(0)])
            if node_features.size(0) < expected_feature_size:
                padding = torch.zeros(expected_feature_size - node_features.size(0), dtype=torch.float32).to(device)
                node_features = torch.cat([node_features, padding])
            elif node_features.size(0) > expected_feature_size:
                node_features = node_features[:expected_feature_size]
            G.add_node(i, x=node_features)

        edge_attributes = {}
        sentences = list(doc.sents)

        for i, sent1 in enumerate(sentences):
            for j, sent2 in enumerate(sentences):
                if i < j:
                    similarity = sent1.similarity(sent2)
                    if similarity > 0.3:
                        edge_key = (sent1.start, sent2.start)
                        edge_attributes.setdefault(edge_key, {})
                        edge_attributes[edge_key]['similarity'] = similarity

        for i in range(len(sentences) - 1):
            edge_key = (sentences[i].start, sentences[i + 1].start)
            edge_attributes.setdefault(edge_key, {})
            edge_attributes[edge_key]['position_proximity'] = 0.3

        valid_edges = 0
        for (u, v), attrs in edge_attributes.items():
            if isinstance(attrs.get('similarity', 0.0), float) and \
               isinstance(attrs.get('position_proximity', 0.0), float):
                G.add_edge(u, v, **attrs)
                valid_edges += 1

        return G

    def build_pyg_data(self, G):
        max_index = max(G.nodes)
        num_nodes = max_index + 1
        node_features = torch.zeros(num_nodes, 782).to(device)

        for node in G.nodes:
            if 'x' in G.nodes[node]:
                feature = G.nodes[node]['x']
                if feature.size(0) < 782:
                    feature = torch.cat([feature, torch.zeros(782 - feature.size(0)).to(device)])
                elif feature.size(0) > 782:
                    feature = feature[:782]
                node_features[node] = feature
            else:
                node_features[node] = torch.zeros(782).to(device)

        edge_index = torch.tensor(list(G.edges)).t().contiguous()
        edge_attr = torch.ones(edge_index.size(1)).to(device)
    
        return Data(x=node_features, edge_index=edge_index, edge_attr=edge_attr)

class GNNTextSummarizer(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNNTextSummarizer, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        
        # Add batch normalization and dropout
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        
        self.attention = AttentionalAggregation(nn.Linear(hidden_dim, 1))
        
        # Add dropout
        self.dropout = nn.Dropout(0.3)
        
        self.output_layer = nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        
        # First convolution layer
        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        # Second convolution layer
        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        # Attention aggregation
        x = self.attention(x)
        
        # Output layer
        x = self.output_layer(x)
        
        return x

class HybridTextGraphSummarizer(TextGraphSummarizer):
    def __init__(self, summary_size=0.3):
        super().__init__(summary_size=summary_size)
        self.abstractive_model = T5ForConditionalGeneration.from_pretrained("t5-small").to(device)
        self.abstractive_tokenizer = T5Tokenizer.from_pretrained("t5-small")

    def abstractive_summarization(self, text_block, min_length=50, max_length=150):
        inputs = self.abstractive_tokenizer(
            "summarize: " + text_block, return_tensors="pt", max_length=512, truncation=True
        ).to(device)
        summary_ids = self.abstractive_model.generate(
            inputs["input_ids"], max_length=max_length, min_length=min_length, length_penalty=2.0, num_beams=4, early_stopping=True
        )
        return self.abstractive_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    def chunk_text(self, text, chunk_size=512, overlap_size=50):
        words = text.split()  # Simple whitespace-based split
        chunks = []
        for i in range(0, len(words), chunk_size - overlap_size):
            chunks.append(' '.join(words[i:i + chunk_size]))
            if i + chunk_size >= len(words):
                break  # Stop if we have reached the end of the text
        return chunks
    def hybrid_summarize(self, report):
        words, doc = self.preprocess_text(report)
        sentences = list(doc.sents)
        embeddings = self.get_embeddings(words)
        G = self.build_graph(words, doc, embeddings)
        data = self.build_pyg_data(G).to(device)

        input_dim = 782
        model = GNNTextSummarizer(input_dim=input_dim, hidden_dim=64, output_dim=1).to(device)

        model.eval()
        with torch.no_grad():
            scores = model(data)
    
    # Ensure scores is a 1D numpy array
        if scores.dim() > 1:
            scores = scores.squeeze()
    
    # Convert to numpy, ensuring it's a 1D array matching number of nodes
        scores_np = scores.cpu().numpy()
    
    # If scores_np is a single value, create an array of that value repeated for all nodes
        if scores_np.ndim == 0:
            scores_np = np.full(len(data.x), scores_np)
    
        sentence_scores = []
        for i, sentence in enumerate(sentences):
            if sentence:
            # Use token indices within the range of scores_np
                token_indices = [
                    token.i for token in sentence 
                    if 0 <= token.i < len(scores_np)
                ]
            
                if token_indices:
                    score = np.mean([scores_np[idx] for idx in token_indices])
                    sentence_scores.append((i, score, sentence.text))

    # Sort sentences by score and select top sentences
        sentence_scores.sort(key=lambda x: x[1], reverse=True)
        print("\nSentence Scores:")
        for idx, score, sentence in sentence_scores:
            print(f"Sentence {idx} (Score: {score:.4f}): {sentence}")
        num_sentences = max(1, int(len(sentences) * self.summary_size))
        selected_sentences = sentence_scores[:num_sentences]
        selected_text = ' '.join([sent[2] for sent in selected_sentences])

    # Rest of the method remains the same...
        chunks = self.chunk_text(selected_text, chunk_size=512, overlap_size=50)
        summaries = []
        for chunk in chunks:
            original_length = len(chunk.split())
            target_length = max(1, int(original_length * self.summary_size))
            min_length = max(1, target_length // 2)
            max_length = target_length

            refined_summary = self.abstractive_summarization(chunk, min_length, max_length)
            summaries.append(refined_summary)

    # Combine summaries and truncate to about 50% of the original report length
        final_summary = ' '.join(summaries)
        target_final_length = max(1, int(len(report.split()) * 0.5))
        final_summary_words = final_summary.split()
        final_summary = ' '.join(final_summary_words[:target_final_length])

        print("\nFinal Summary:", final_summary)
        return final_summary, selected_sentences
if __name__ == "__main__":
    report = """
   COMMISSIONER, TRADE TAX, U.P., LUCKNOW v. M/S. SUMAN TRADING CO. [2015] INSC 4 (6 January 2015)
'REPORTABLE' IN THE SUPREME COURT OF INDIA CIVIL APPELLATE JURISDICTION CIVIL APPEAL NO. 28 OF 2015 [Arising out of Special Leave Petition (C) No.32616 of 2013] M/s. Pepsico India Holding Pvt. Ltd. ... Appellant(s) versus Krishna Kant Pandey ...Respondent(s)
M.Y.EQBAL,J.
1\. Leave granted.

2\. This appeal by special leave is directed against judgment and order dated 23.5.2013 of the High Court of Allahabad at Lucknow Bench whereby learned Single Judge classifying the respondent as 'workman' allowed the writ petition preferred by him, quashed the order dated August 24, 2007 passed by the Industrial Tribunal II, State of Uttar Pradesh, Lucknow (in short, 'the Tribunal') and directed the Tribunal to decide respondent's Case No.84/2004 on merit.

3\. The factual matrix of the case is that the respondent was appointed on the post of Operator/Technician Grade III for six months on probation basis w.e.f. 13th of March, 1995 against the salary of Rs. 2600/- per month.
Having been found his services satisfactory, he was confirmed w.e.f. 13th September, 1995 and was also awarded one increment w.e.f. 1st of February, 1996. Earlier, he was appointed in the Plant of Jainpur (Kanpur Dehat) from where he was transferred to Sathariya Plant, District Jaunpur, U.P. on 30th of August, 1996 on the revised pay scale i.e. Rs. 5450/-. Pursuant to the subsequent transfer order, he was posted at Lucknow in the month of June, 1997 and till 2000 he was awarded annual increments at the rate of Rs.490/-. Subsequently, he was promoted to the post of Line Supervisor in the pay scale of Rs.7716/- and thereafter to the post of Fleet Executive.

4\. It is the case of the respondent that being posted as a Fleet Executive, he was to discharge the mechanical work and that being so, he was called as skilled workman. It is stated that no other staff was posted in his subordination. The respondent also pointed out the conduct of the employer transferring him from one place to another and also compelling him to resign from the post or to be on long leave. On being asked to proceed on leave, respondent remained on leave w.e.f. 9th October, 2003 to 17th October, 2003. When he turned up, he was not permitted to join for want of instructions of the superior authorities. Thereafter, respondent wrote a letter on 8th November, 2003 to the Vice President seeking guidance for further action, upon which the employer became unhappy and terminated his services on 14th of November, 2003 by giving one month's salary in lieu of notice prior to termination.

5\. Aggrieved by the said termination, respondent preferred a reference before the Conciliation Officer, Lucknow alleging that he is a 'workman' within the meaning of the Uttar Pradesh Industrial Disputes Act, 1947 (hereinafter referred to as the 'Act') and termination of his services by the Company is contrary to Section 6 of the Act. The appellant Company pleaded that the respondent did not satisfy the criteria of a workman as defined under Section 2(z) of the Act. The Industrial Tribunal dismissed the reference stating that the respondent is not a workman under Section 2(z) of the Act and, therefore, no challenge to the termination is maintainable before the Tribunal.

6. Respondent, being aggrieved, moved the High Court by way of a writ petition challenging the order of the Tribunal and also for his re- instatement to the post of Fleet Executive with continuity of service and for payment of full back wages. In reply, the appellant pleaded that the order of termination is in accordance with the provisions of the Act.
After hearing learned counsel on either side, learned Single Judge of the High Court allowed writ petition of the respondent, quashed order of the Tribunal and directed it to proceed with the adjudication of the respondent's case on merit. Hence, the present appeal by special leave by the appellant-Company.

7\. Mr. C.U. Singh, learned counsel appearing for the appellant, assailed the order passed by the High Court on various grounds inter alia, the High Court has exceeded its jurisdiction conferred upon it under Article 226 of the Constitution of India by reversing the finding recorded by the Tribunal. Learned counsel submitted that the High Court has committed grave error in issuing suo motu directions to the executive to amend a relevant provision of Section 2(z) of the U.P. Industrial Disputes Act (in short, 'State Act'). According to the learned counsel, issuing such direction by the High Court amounts to issue a direction to the legislature to enact a law in a particular manner. Learned counsel submitted that the High Court cannot mandate the executive to introduce or enact a legislation, howsoever necessary or desirable. Learned counsel drew our attention to the provisions of Section 2(z) of the said Act which defines 'workman' and submitted that the High Court has failed to appreciate that the nature of duties and responsibilities entrusted upon the respondent are not manual, skilled or unskilled or technical services, but manual, managerial and supervisory. In the capacity of Fleet Executive, respondent was required to monitor each and every vehicle of the Fleet and ensure that the necessary repair proceedings were carried out. Learned counsel further submitted that the High Court has misdirected itself in considering the relevant facts and pleadings which were not even placed before the Industrial Tribunal. Lastly, it was contended that the High Court in exercise of its jurisdiction under Article 226 of the Constitution ought not re-appreciate or re-weigh evidence and disturb the finding of facts recorded by the Tribunal based on appreciation of evidence. Learned counsel relied upon the judgment of this Court in the case of Chandavarkar Sita Ratna Rao vs. Ashalata S. Guram[1986] INSC 197; , (1986) 4 SCC 447, Birla Corpn. Ltd..
vs. Rajeshwar Mahato and Others, (2001) 10 SCC 611 and S.K. Mani vs.
M/s. Carona Sahu Company Limited and Ors., [1994] INSC 165; (1994) 3 SCC 510.

8\. Mr. Kavin Gulati, learned senior counsel appearing for the respondent firstly contended that before conciliation, the appellant raised an objection with regard to the jurisdiction of the Tribunal and the matter was finally referred to the Labour Court for adjudication on a limited question as to whether the termination of services of the respondent was justified. According to the learned counsel, the appellant-management cannot raise the question of jurisdiction of the Labour Court. Learned counsel referred to Rule 12 of the Industrial Disputes Rules in support of his contention and relied upon the decision of this Court in the case of M/s. U.P. Electric Supply Co., Ltd. vs. The Workmen of M/s. S.N. Choudhary Contractors and Anr., (1960) 3 SCR page 189. Mr. Gulati, learned senior counsel also relied upon the decision of this Court in the case of TISCO Ltd. v. State of Jharkhand, (2014) 1 SCC 536, and Bhogpur Coop Sugar Mills Ltd. vs. Harmesh Kumar, (2006) 13 SCC 28, for the proposition that the Labour Court has limited jurisdiction to adjudicate the disputes referred to it and not to enter into any other question that may be raised in the reference.

9\. We have heard learned counsel for both the parties and considered the relevant facts and the law applicable thereto. The admitted facts are that at the relevant time, the respondent was working as a Fleet Supervisor and drawing a salary of Rs. 7716/-. Initially, he was appointed as a technician Grade-III in 1995 on the salary of Rs. 2600/- per month and after getting increment his salary was increased. By giving one month's salary, in lieu of the notice, his services were terminated.

10\. It appears that the appellant raised a preliminary objection before the Labour Court that the respondent was not a Labour as defined under Section 2(z) of the said Act and the Deputy Labour Commissioner, Lucknow, who had referred the present case, was not competent for this purpose. The Labour Court recorded the evidence adduced by both the appellant and the respondent and discussed the evidence, and elaborately considered the case of the parties. The Labour Court finally recorded finding that the respondent is not a workman within the meaning of Section 2(z) of the Act.
Paras 13 to 15 of the order are reproduced hereinbelow:- "13. The statement of the Applicant is that although he was given the post in the name of Fleet Executive and he was posted at the warehouse at Lucknow, however practically he was doing the work of skilled manual and as such he fall within the definition as given in Section 2(z) of the Industrial Disputes Act, 1947. According to him his main work was to remove the technical defects at 100% from the fleet. His other works which have been mentioned by the management in their written statement, were secondary. It has also been stated that the written statement of the management is not on affidavit, therefore the same cannot be relied upon.
He was executing all his work in accordance with the directions of the higher officials. He was not having any right of doing the work as per his own wishes. He has stated in his statement that no staff was working under him. He used to do the work himself for keeping the vehicles 100% free/available from technical deformities and removed the difficulties of the vehicles. It has also been stated that there is a difference in the statement of witnesses of the management EW-1 and EW-2 regarding the repair of the work of the company and the same cannot be relied upon. It has been stated while relying upon the legal arrangement given by the Hon'ble Management Staff Association and others [1970] INSC 235; (1970) 3 SCC 378 that any personnel fall within the definition of labour or not, it depends upon the fact that what is the nature of the main works being done by him. The same cannot be assessed with the name of the post. If the concerned person is doing the work of manual skilled unskilled work, then he is in the definition of labour, as the main work of the Applicant was to repair the vehicles, which he used to do from his own hands. No other person was working under him and he was not having the right to take decision by himself. Therefore, he falls within the definition of Labour. It has been stated while referring (supra) that the work of the Applicant was similar to fupelling superintendent, which has been considered by the Hon'ble Apex Court as labour, because his work was manual and not supervisory. Therefore he falls within the definition of labour and the termination of service made by the management is retrenchment, which has been done in violation of the provisions of Section 60N of the U.P. Industrial Disputes Act, 1947.
Therefore his termination of service is improper and illegal. On this basis he may be reinstated in service alongwith salary for leave period.

14\. It has been argued on behalf of Management that out of the works executed by the Applicant on the post of Fleet Executive, the details of the paid work are mentioned in their written statement. The same has been admitted by the Applicant in his arguments. In this manner, the details regarding the main work out of the works on the post of Fleet Executive being done by him is proved. All these works are supervisory in nature.
The major work of repair of the vehicle used to be done from outside. The work of washing and cleaning of the vehicles was done by driver. The Applicant has never done any type of repairing work and neither anybody has seen the Applicant while doing such work. In this manner mainly the Applicant was doing the work of supervisory nature. Therefore does not fall within the definition of labour. His services have been terminated under the terms. In this manner the order of termination of his service is proper and legal. He is not entitled to get any relief.

15\. The main question in this industrial dispute is whether the Applicant K.K. Pandey is a labour, as claimed by him, as this claim has been made by him and as such onus to prove the same lies on him. According to para 11 and 12 of his written statement he was having only one responsibility on the post of fleet executive that he was to ensure the availability of the vehicles free from technical deformities. According to the written statement for this work nobody was working under him and he used to do the work of repair with his own hands. He has reiterated this fact in his statement also. In this regard except his statement has not produced any evidence to confirm the same. On the other hand he has admitted in his arguments as regards the details of different works mentioned by the management in para No.1 of their written statement. According to it out of his works, there is a detail of 15 main works. In this manner the statement made by the Applicant regarding his main work remained rebutted.
The statement of the Applicant regarding the post of Fleet Executive on the basis of which he is claiming himself as labour is not liable to be believed."

11\. On the basis of the findings based on elaborate discussions and analyzing the evidence, the Labour Court came to the conclusion that at the relevant time the respondent was working as a Fleet Executive which is supervisory in nature and does not fall within the definition of 'labour' as defined under Section 2(z) of the Act. Hence, he is not entitled to any relief. The respondent challenged the aforesaid award passed by the Labour Court in a writ petition before the High Court. After considering the definition contained in Section 2(z) of the Act and the nature of work assigned to the respondent, the High Court arrived at a conclusion that the nature of work prevalent on the date of termination was as that of a workman. Curiously enough, though the respondent did not come under the definition of workman under Section 2(z) of the Act, the High Court proceeded on the basis that the U.P. Industrial Disputes Act was enacted in 1947 and although the respondent cannot be held to be a workman under the said definition, held that he shall have to be classified as a workman and directed the Government to make amendment in Section 2(z) of the Act excluding some of the clauses. For better appreciation, relevant portion of the order is quoted hereinbelow:- "There is one more exclusion clause in section 2(z) of the Act i.e. Clause (iv) which excludes the employee who being employed in supervisory capacity draws wages exceeding Rs. 500/- per mensem or exercise, either by the nature of the duties attached to the office or by reason of the powers vested in him, functions mainly of a managerial in nature. It is very much obvious that by nature of duties assigned to the petitioner, it cannot be said that he was attached to the office or mainly managerial function was vested with him.
So far as another condition for exclusion from the definition of "workman" viz drawing wages exceeding Rs. 500/- per mensem is concerned, it is not in dispute that the petitioner on the date of retrenchment had been getting more Rs. 500/- mensem. This clause is a part of original form of the definition of "workman" as is provided under section 2 (z) of the Act.
The U. P. Industrial Disputes Act was enacted in 1947. The petitioner was retrenched from service in 2003. The date of his initial appointment is on 13th of March, 1995. By passage of time the GDP growth had been increased in number of times from 1947 to 2003. Therefore, the enhancement in income was a natural corollary, in the light of which, I am of the view that this clause has become unworkable and redundant. Now every workman working in the Industry definitely would have been getting more than Rs. 500/- per mensem and if this clause is permitted to be prevailed no workman shall be governed under the definition of "workman". Therefore, I am of the view that this clause has lost its significance and if the employee is covered under the definition of "workman" as is defined under Section 2 (z) of the U. P. Industrial Disputes Act and further is not covered under the exclusion clause except clause (iv), he shall be classified as "workman".
The clause (iv) of section 2 (z) shall not come in the way of his categorization as "workman".
It is advisable to the State Government to consider to make an amendment in section 2 (z) of the U. P. Industrial Disputes Act, 1947 in general and to exclude the class (iv) from the exclusion in particular.
Since the present petitioner has been classified by this court, as above, under the definition of "workman" the order impugned dated 24th of August, 2007 passed by the Industrial Tribunal II, State of U. P., Lucknow is hereby quashed with the direction to the Tribunal to proceed with the adjudication case No. 82/2004 to adjudicate upon it on merit."

12\. Considering the entire facts of the case and the findings recorded by the Labour Court, prima facie we are of the view that the High Court has exceeded in exercise of its jurisdiction under Articles 226 and 227 of the Constitution of India in interfering with the finding of facts recorded by the Labour Court. It is well settled that the High Court in the guise of exercising its jurisdiction normally should not interfere under Article 227 of the Constitution and convert itself into a court of appeal.

13\. While discussing the power of the High Court under Articles 226 and 227 of the Constitution interfering with the facts recorded by the courts or the tribunal, this Court in the case of Chandavarkar S.R. Rao vs.
Ashalata S. Guram, (supra) held as under :- "17. In case of finding of facts, the court should not interfere in exercise of its jurisdiction under Article 227 of the Constitution.
Reference may be made to the observations of this Court in Bathutmal Raichand Oswal v. Laxmibai R. Tarta where this Court observed that the High Court could not in the guise of exercising its jurisdiction under Article 227 convert itself into a court of appeal when the legislature has not conferred a right of appeal. The High Court [pic]was not competent to correct errors of facts by examining the evidence and reappreciating.
Speaking for the Court, Bhagwati, J. as the learned Chief Justice then was, observed at p. 1301 of the report as follows: (SCC p. 864, para 7) "The special civil application preferred by the appellant was admittedly an application under Article 227 and it is, therefore, material only to consider the scope and ambit of the jurisdiction of the High Court under that article. Did the High Court have jurisdiction in an application under Article 227 to disturb the findings of fact reached by the District Court? It is well settled by the decision of this Court in Waryam Singh v.
Amarnath that the ... power of superintendence conferred by Article 227 is, as pointed out by Harries, C.J., in Dalmia Jain Airways v. Sukumar Mukherjee to be exercised most sparingly and only in appropriate cases in order to keep the subordinate courts within the bounds of their authority and not for correcting mere errors.
This statement of law was quoted with approval in the subsequent decision of this Court in Nagendra Nath Bose v. Commr. of Hills Division and it was pointed out by Sinha, J., as he then was, speaking on behalf of the court in that case:
It is thus, clear that the powers of judicial interference under Article 227 of the Constitution with orders of judicial or quasi-judicial nature, are not greater than the power under Article 226 of the Constitution. Under Article 226 the power of interference may extend to quashing an impugned order on the ground of a mistake apparent on the face of the record. But under Article 227 of the Constitution, the power of interference is limited to seeing that the tribunal functions within the limits of its authority."

14\. In the case of Birla Corpn. Ltd.. vs. Rajeshwar Mahato and Others, (2001) 10 SCC, the question of validity of termination of services of the respondent by the appellant-Corporation was referred to the Industrial Tribunal. On evidence, the Industrial Tribunal found that the duties of the respondent were mainly managerial or administrative. The Tribunal held that the respondent was not a workman and the reference was therefore not maintainable against the decision of the Tribunal. The Tribunal relying on Section 2(s)(iv) (as amended in West Bengal W.B.) held that as the respondent was drawing salary less than Rs.1600/- per month, he had to be regarded as a workman. The Corporation moved this Court against the order of the High Court. This Court while setting aside the decision of the High Court held as under :- "4. It was not in dispute that at the time of the termination of services of Respondent 1, he was receiving Rs 1185 per month by way of salary. The Tribunal recorded the evidence as well as took into consideration documentary evidence which was produced by the parties. On the basis of the evidence which was adduced before it, the Tribunal observed that:
"The main duties of Shri Rajeshwar Mahato were both supervisory and administrative in nature.
In the instant case, Shri Mahato's functions were mainly of a managerial nature. He had control as well as supervision over the work of the jute mill workers working under him."

11\. As we have also noticed hereinabove, the Tribunal had given a categorical finding to the effect that Respondent 1's function was mainly of managerial nature. His duties were both supervisory and administrative and therefore he was regarded as not being a workman. Though the Tribunal did not specifically state so, it is evident that it is because of Section 2(s)(iii) that Respondent 1 was regarded as not being a workman.

12\. Neither the Single Judge nor the Division Bench of the High Court, as we have already noticed, referred to this aspect of the matter. Even assuming that the West Bengal amendment was applicable, that would still not help to hold Respondent 1 as a workman if the finding of the Tribunal with regard to the nature of the duties performed by him, as arrived at by the Tribunal, is not set aside as being frivolous or without any evidence.
As long as the finding of the Tribunal stands, namely, that the respondent was an employee mainly in a managerial or administrative capacity, the award of the Tribunal could not have been set aside. As we have already observed the Single Judge or even the Division Bench could have come to the conclusion that the finding so arrived at by the Tribunal was either frivolous or not based on any evidence. But this aspect of the case was completely overlooked by the High Court. The emphasis of the Single Judge as well as the Division [pic]Bench was only with regard to applicability of the amendment of the State of West Bengal to Section 2(s) of the Industrial Disputes Act. In our opinion, therefore, the High Court erred in allowing on this ground the writ petition filed by Respondent 1. The decision of the High Court is set aside and the writ petition filed therein by the respondent stands dismissed."

15\. In the case of Indian Overseas Bank vs. I.O.B. Staff Canteen Workers' Union and Another, (2000) 4 SCC 245, this Court considered a similar question with regard to the power of the High Court under Article 226 against the findings recorded by the Industrial Tribunal. Reversing the decision of the Single Judge and restoring the fact finding decision of the Tribunal this, Court held :- "17. The learned Single Judge seems to have undertaken an exercise, impermissible for him in exercising writ jurisdiction, by liberally reappreciating the evidence and drawing conclusions of his own on pure questions of fact, unmindful, though aware fully, that he is not exercising any appellate jurisdiction over the awards passed by a tribunal, presided over by a judicial officer. The findings of fact recorded by a fact-finding authority duly [pic]constituted for the purpose and which ordinarily should be considered to have become final, cannot be disturbed for the mere reason of having been based on materials or evidence not sufficient or credible in the opinion of the writ court to warrant those findings, at any rate, as long as they are based upon some material which are relevant for the purpose or even on the ground that there is yet another view which can reasonably and possibly be taken. The Division Bench was not only justified but well merited in its criticism of the order of the learned Single Judge and in ordering restoration of the award of the Tribunal. On being taken through the findings of the Industrial Tribunal as well as the order of the learned Single Judge and the judgment of the Division Bench, we are of the view that the Industrial Tribunal had overwhelming materials which constituted ample and sufficient basis for recording its findings, as it did, and the manner of consideration undertaken, the objectivity of approach adopted and reasonableness of findings recorded seem to be unexceptionable. The only course, therefore, open to the writ Judge was to find out the satisfaction or otherwise of the relevant criteria laid down by this Court, before sustaining the claim of the canteen workmen, on the facts found and recorded by the fact-finding authority and not embark upon an exercise of reassessing the evidence and arriving at findings of one's own, altogether giving a complete go-by even to the facts specifically found by the Tribunal below."

16\. The order of the Tribunal would show that the respondent-workman accepted different works assigned to him which were purely of supervisory and managerial nature. The details of 15 managerial/supervisory works assigned to the respondent have been analyzed by the Tribunal which finally came to the conclusion that the respondent is not a workman within the meaning of Section 2(z) of the Act.

17\. In exercise of its writ jurisdiction, the High Court proceeded initially on the basis that the appellant had entered into service on the post of Operator/Technician Grade-III, which is a technical post and from there he was promoted to different posts including Fleet Executive. The High Court committed grave error in holding that although he is not covered under the definition of workman as defined under Section 2(z) of the Act he shall be classified as a workman. The High Court further exceeded its jurisdiction in advising the Government to make an amendment in Section 2(z) of the Act and to exclude some clauses. The order passed by the High Court cannot be sustained in law.

18\. We, therefore, allow this appeal and set aside the order of the High Court and restore the order passed by the Tribunal. However, we give liberty to the respondent to move the appropriate forum to challenge, in accordance with law, the order of termination passed by the appellant.
"""
    summarizer = HybridTextGraphSummarizer(summary_size=0.5)  # Adjust summary size as needed
    summary, sentence_details = summarizer.hybrid_summarize(report)

   


Using device: cuda
Initializing TextGraphSummarizer on device: cuda
Running preprocess_text...
Running build_graph...

Sentence Scores:
Sentence 1 (Score: -0.1040): [2015] insc 4 (6 january 2015)
'reportable' in the supreme court of india civil appellate jurisdiction civil appeal no. 28 of 2015 [arising out of special leave petition (c) no.32616 of 2013] m/s. pepsico india holding pvt. ltd. ...
Sentence 3 (Score: -0.1040): 2\. this appeal by special leave is directed against judgment and order dated 23.5.2013 of the high court of allahabad at lucknow bench whereby learned single judge classifying the respondent as 'workman' allowed the writ petition preferred by him, quashed the order dated august 24, 2007 passed by the industrial tribunal ii, state of uttar pradesh, lucknow (in short, 'the tribunal') and directed the tribunal to decide respondent's case no.84/2004 on merit.


Sentence 5 (Score: -0.1040): 13th of march, 1995 against the salary of rs.
Sentence 7 (Score: -0.1040): having

In [69]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, AttentionalAggregation
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import spacy
from transformers import BertModel, BertTokenizer, T5ForConditionalGeneration, T5Tokenizer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

class TextGraphSummarizer:
    def __init__(self, summary_size=0.1):
        print(f"Initializing TextGraphSummarizer on device: {device}")
        self.nlp = spacy.load("en_core_web_md")
        self.summary_size = summary_size
        self.important_legal_terms = self.important_legal_terms = [
    "abandonment", "abatement", "abduction", "abscond", "absolute discharge", "absolute owner", 
    "absolute privilege", "abstract title", "abuse process", "acceptance service", "accessory", 
    "accomplice", "accumulation", "accused", "acquit", "acquittal", "active trust", "actual bodily harm", 
    "actual loss", "actus reus", "ademption", "adjournment", "adjudicate", "adjudication order", 
    "administration order", "administrator evidence", "admission", "adoption", "adoptive child", 
    "adoptive parent", "adverse possession", "adverse witness", "affidavit", "affirmation", "affray", 
    "agency", "aggravated assault", "aggravated burglary", "aggravated damages", "agreement", 
    "aiding and abetting", "alias", "alibi", "alienation", "allegation", "alternate director", 
    "alternative verdict", "amalgamation", "ambiguity", "ambulatory will", "amnesty", "annual accounts", 
    "annual return", "annuitant", "annuity", "annul", "antenuptial agreement", "anton piller order", 
    "appeal", "appellant", "appellate jurisdiction", "appointee", "appointor", "arbitrage", "arbitration", 
    "arbitrator", "arraignment", "arrest", "arrestable offence", "arson", "articles association", "assault", 
    "asset", "assignment", "assurance", "assured tenancy", "attachment earnings", "attorney general", 
    "autopsy", "bail", "bail hostel", "bailee", "bailiff", "bailiwick", "bailment", "bailor", "bankruptcy", 
    "bankruptcy order", "bankruptcy search", "bar", "bare trust", "barrister", "barter", "battery", 
    "bench warrant", "beneficial interest", "beneficiary", "bequeath", "bigamy", "bill costs", "bill exchange", 
    "bill lading", "binding precedent", "blackmail", "bona fide", "bona vacantia", "bond", "bonded goods", 
    "breach contract", "breach duty", "burglary", "bylaw", "capital gains tax", "capital punishment", 
    "care order", "case law", "causation", "caveat emptor", "central criminal court", "certiorari", 
    "challenge jury", "chancery division", "charge certificate", "charge sheet", "charging clause", 
    "charity commission", "chattel", "child abuse order", "child support agency", "chose action", 
    "chose possession", "circuit judge", "citizen's arrest", "civil court", "claim", "claimant", "coercion", 
    "collateral", "committal proceedings", "common assault", "companies house", "company secretary", 
    "compensation order", "compulsory purchase", "concealment", "conditional agreement", "conditional sale", 
    "confiscation order", "consecutive sentence", "consent", "consideration", "consignee", "consignor", 
    "consistory court", "conspiracy", "constructive dismissal", "constructive notice", "constructive trust", 
    "consumer protection", "contempt of court", "contingency fee", "contingent legacy", "contract for services", 
    "contract law", "contract of exchange", "contract of service", "contributory negligence", "conversion", 
    "convey", "conveyance", "conveyancing", "conviction", "copyright", "coroner", "corporation tax", "corpus", 
    "counsel", "counterclaim", "counterfeit", "counterpart", "county court", "county court judge", "coupon", 
    "court of appeal", "court of protection", "covenant", "creditor", "criminal damage", "criminal responsibility", 
    "cross-examine", "crown court", "culpa", "curfew", "customs duties", "damages", "debenture", "debt", 
    "debt securities", "debtor", "deceit", "decree absolute", "decree nisi", "deed of arrangement", "defamation", 
    "default", "defence", "defendant", "dependant", "deponent", "deposition", "depreciation", "derogation", 
    "determination", "devise", "devisee", "diocese", "diplomatic immunity", "direction", "director", 
    "disbursement", "discharge", "disclaimer", "discovery", "discretionary trust", "disposal", "distrain/distress", 
    "dividend", "dividend shares", "divorce petition", "domicile", "drawee", "drawer", "duress", "duty", "easement", 
    "enabling legislation", "endorsement", "endowment policy", "engrossment", "equitable mortgage", "escrow", 
    "estate", "estimate", "estoppel", "euthanasia", "ex gratia", "ex parte", "excess jurisdiction", "excise duty", 
    "exclusive licence", "execute", "executed", "executive director", "executor", "executory", "executrix", 
    "exemplary damages", "expert witness", "extradition", "false imprisonment", "false pretence", "false representation", 
    "family division", "felony", "feme covert", "feme sole", "feu duty", "fiduciary", "final judgement", "fixed charge", 
    "floating charge", "forbearance", "force majeure", "foreclosure", "forfeiture", "fostering", "fraud", 
    "fraudulent conveyance", "fraudulent preference", "fraudulent trading", "free from encumbrances", "freehold", 
    "frustration", "futures contract", "garnishee order", "general damages", "general meeting", "grant of probate", 
    "grievous bodily harm", "guarantee", "guarantee company", "guarantor", "guardian", "guilty", "habeas corpus", 
    "harassment", "hearsay evidence", "hereditament", "high court of justice", "hire purchase", "holding company", 
    "hostile witness", "house of lords", "housing associations", "hypothecation", "indict", "indictable offence", 
    "indictment", "initio", "injunction", "intangible property", "interest", "interrogatories", "intestacy/intestate", 
    "intimidation", "issued share capital", "joint lives policy", "joint tenancy", "joint will", "joyriding", 
    "judge advocate general", "judgement creditor", "judgement debtor", "judgement proceedings", "judicial discretion", 
    "judicial immunity", "judicial precedent", "judicial separation", "junior barrister", "jure", "juror", 
    "jury service", "justice", "justification", "juvenile offender", "kerb crawling", "kidnap", "know-how", "land", 
    "land registry", "lawsuit", "leading question", "lease", "leasehold", "legacy", "legal scheme", "legatee", "lessee", 
    "lessor", "letter of credit", "lex", "liabilities", "liability", "libel", "licence", "licensed conveyancer", 
    "licensee", "lien policy", "life assured", "life imprisonment", "life interest", "life tenant", "limited company", 
    "liquidated damages", "liquidation", "liquidator", "litigant", "litigation", "loan capital", "loan creditor", 
    "magistrate", "magistrates' court", "maintenance", "majority", "male issue", "malfeasance", "malice aforethought", 
    "malicious falsehood", "malicious prosecution", "mandate", "manslaughter", "market overt", "martial law", 
    "master of the rolls", "material facts", "matricide", "matrimonial causes", "matrimonial home", "mediation", 
    "mens rea", "mercantile law", "merchantable quality", "mesne profits", "minor", "minority", "minutes", "misadventure", 
    "miscarriage of justice", "misconduct", "misdirection", "misfeasance", "misrepresentation", "misuse", "mitigating circumstances", 
    "money laundering", "mortgage", "mortgage deed", "mortgagee", "mortgagor", "motion", "murder", "mutual will", 
    "natural justice", "negligence", "negligent misstatement", "negligent misrepresentation", "nervous shock", 
    "no fault divorce", "nolo contendere", "notice", "notary", "novation", "novation agreement", "obligation", 
    "obscenity", "offence", "offender", "offence by company", "officer", "official receiver", "offer", "offer and acceptance", 
    "official secrets act", "oppression", "option contract", "oral evidence", "out of court settlement", "overruling", 
    "ownership", "packing", "parol evidence", "partnership", "party", "patent", "patent rights", "personal injury", 
    "personal representatives", "plaintiff", "plea", "plea bargaining", "pleading", "possession", "postal vote", 
    "power of attorney", "pre-action disclosure", "precedent", "prejudice", "preliminary inquiry", "prescription", 
    "presumption", "private nuisance", "probate", "proceedings", "proclamation", "promissory note", "proof of debt", 
    "proprietary estoppel", "prosecute", "prosecution", "prospective juror", "protective order", "protection order", 
    "public nuisance", "public prosecutor", "punitive damages", "pure economic loss", "quasi contract", "quash", 
    "questionnaire", "racketeering", "rebuttal", "recovery", "referral fee", "refusal", "regulation", "reimbursement", 
    "remedy", "remand", "remand hearing", "remedy", "rescind", "residual estate", "respondent", "restraint of trade", 
    "restitution", "restitutionary claim", "retrial", "revocation", "right of redemption", "right of way", 
    "rising costs", "rule of law", "salvage", "savings bond", "secured debt", "security", "security interest", 
    "settlement", "settlement agreement", "set-off", "shareholder", "shareholders' agreement", "short term lease", 
    "signatory", "solicitor", "solicitor-client privilege", "solicitor's lien", "special damages", "special verdict", 
    "specialist tribunal", "specific performance", "statute", "statutory authority", "statutory corporation", 
    "statutory instrument", "stay of execution", "steward", "stipulation", "strict liability", "subcontractor", 
    "subrogation", "sue", "subpoena", "substantial performance", "succession", "sufficiency", "summary conviction", 
    "summary offence", "summary proceedings", "supersede", "surcharge", "survivorship", "suspension", "suspension clause", 
    "suspension order", "taciturn", "tax avoidance", "tax evasion", "taxable income", "taxation", "tenant", 
    "tenant's agreement", "tender", "testamentary capacity", "testamentary disposition", "testator", "testimony", 
    "tort", "total loss", "trade secret", "trade union", "trademark", "transfer", "transfer of title", "trustee", 
    "trust deed", "trustee in bankruptcy", "unacceptable risk", "unlawful detainer", "unliquidated damages", 
    "unsound mind", "validity", "vicarious liability", "victim impact statement", "voluntary arrangement", 
    "waiver", "warranty", "warranty of merchantability", "waste", "will", "witness", "wrongful act", 
    "wrongful dismissal", "wrongful imprisonment"
]

        
        # Stop words as a list (you can specify your own list of stop words)
        self.stop_words = ["a", "about", "above", "acceptance", "accordingly", "accounts", "action", "actual", "after", "against", "all", "also", "although", "an", "and", "another", "any", "are", "around", "as", "at", "be", "because", "been", "before", "being", "between", "both", "but", "by", "can", "could", "did", "do", "does", "done", "during", "each", "either", "every", "few", "for", "from", "had", "has", "have", "hoc", "how", "idem", "if", "in", "infinitum", "inside", "into", "is", "it", "its", "less", "like", "many", "may", "might", "more", "most", "much", "must", "neither", "no", "none", "not", "of", "off", "on", "one", "or", "other", "out", "outside", "over", "per", "pro", "quod", "same", "several", "shall", "should", "so", "some", "such", "than", "that", "the", "their", "them", "then", "there", "these", "these are general words that are common in everyday language or are too generic:", "this", "those", "though", "through", "to", "under", "until", "up", "very", "was", "were", "what", "when", "where", "which", "while", "who", "whom", "whose", "why", "will", "with", "without", "would"]

        # Load Sentence-BERT and Legal-BERT
        self.sentence_model = BertModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2").to(device)
        self.legal_model = BertModel.from_pretrained("nlpaueb/legal-bert-base-uncased").to(device)
        self.sentence_tokenizer = BertTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
        self.legal_tokenizer = BertTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")

    def preprocess_text(self, text):
        print("Running preprocess_text...")
        doc = self.nlp(text.lower())
        words = [token.text for token in doc if not token.is_stop]
        print(f"Extracted {len(words)} words after preprocessing.")
        return words, doc

    def get_embeddings(self, words):
        print("Running get_embeddings...")
        max_length = self.sentence_tokenizer.model_max_length
        words = words[:max_length]
        print(f"Using first {max_length} words for embedding.")

        # Tokenize and get embeddings
        sentence_inputs = self.sentence_tokenizer(words, padding=True, truncation=True, return_tensors="pt").to(device)
        legal_inputs = self.legal_tokenizer(words, padding=True, truncation=True, return_tensors="pt").to(device)

        with torch.no_grad():
            sentence_outputs = self.sentence_model(**sentence_inputs)
            legal_outputs = self.legal_model(**legal_inputs)

            # Use mean pooling
            sentence_embeddings = sentence_outputs.last_hidden_state.mean(dim=1)
            legal_embeddings = legal_outputs.last_hidden_state.mean(dim=1)

            print(f"Sentence embeddings shape: {sentence_embeddings.shape}")
            print(f"Legal embeddings shape: {legal_embeddings.shape}")

        # Concatenate embeddings
        combined_embeddings = torch.cat((sentence_embeddings, legal_embeddings), dim=1)
        print(f"Combined embeddings shape: {combined_embeddings.shape}")
        return combined_embeddings
    def adjust_embeddings_for_legal_terms(self, embeddings, words):
        print("Running adjust_embeddings_for_legal_terms...")
    
    # Ensure the words list does not exceed the embedding size
        max_tokens = embeddings.size(0)
        words = words[:max_tokens]  # Slice words list to match embedding size

        adjusted_embeddings = embeddings.clone()  # Create a copy of embeddings to adjust

        for i, word in enumerate(words):
            if word in self.important_legal_terms:
            # Increase the embedding value for important legal terms
                adjusted_embeddings[i] *= 1.2  # Increase by a factor of 1.2 (can be adjusted)
            elif word in self.stop_words:
            # Decrease the embedding value for stop words
                adjusted_embeddings[i] *= 0.8  # Decrease by a factor of 0.8 (can be adjusted)
    
    # Normalize adjusted embeddings (optional, depending on your needs)
        adjusted_embeddings = F.normalize(adjusted_embeddings, p=2, dim=-1)
    
        print(f"Adjusted embeddings shape: {adjusted_embeddings.shape}")
        return adjusted_embeddings

    def build_graph(self, words, doc, embeddings):
        print("Running build_graph...")
        G = nx.Graph()

    # TF-IDF vectorizer
        tfidf_vectorizer = TfidfVectorizer()
        tfidf_scores = tfidf_vectorizer.fit_transform([' '.join(words)])
        feature_names = tfidf_vectorizer.get_feature_names_out()
        tfidf_dict = {word: tfidf_scores[0, idx] for word, idx in zip(feature_names, range(tfidf_scores.shape[1]))}
        print(f"TF-IDF dictionary size: {len(tfidf_dict)}")

    # Add nodes
        for i, token in enumerate(doc):
            if not token.is_alpha or i >= len(embeddings):
                continue
            G.add_node(i, x=embeddings[i])

    # Add edges based on similarity
        sentences = list(doc.sents)
        print(f"Number of sentences identified: {len(sentences)}")
        edge_attributes = {}

        for i, sent1 in enumerate(sentences):
            for j, sent2 in enumerate(sentences):
                if i < j:
                    similarity = sent1.similarity(sent2)
                    if similarity > 0.3:
                        edge_key = (sent1.start, sent2.start)
                        edge_attributes.setdefault(edge_key, {})
                        edge_attributes[edge_key]['similarity'] = similarity

    # Add sequential edges
        for i in range(len(sentences) - 1):
            edge_key = (sentences[i].start, sentences[i + 1].start)
            edge_attributes.setdefault(edge_key, {})
            edge_attributes[edge_key]['position_proximity'] = 0.3

    # Add edges to graph
        for (u, v), attrs in edge_attributes.items():
            G.add_edge(u, v, **attrs)

        print(f"Graph construction complete: {len(G.nodes)} nodes, {len(G.edges)} edges")
        return G

    def build_pyg_data(self, G):
        print("Running build_pyg_data...")
        num_nodes = max(G.nodes) + 1 if G.nodes else 0  # Handle case where G has no nodes
        feature_dim = 1152  # Update this to match the actual embedding size
        node_features = torch.zeros(num_nodes, feature_dim).to(device)

        for node in G.nodes:
            if 'x' in G.nodes[node]:
                feature = G.nodes[node]['x']
                node_features[node] = feature

    # Handle the case where there are no edges
        edge_index = torch.tensor(list(G.edges)).t().contiguous().to(device)
        if edge_index.numel() == 0:  # Check if edge_index is empty
            print("Warning: Graph has no edges!")
            edge_index = torch.empty(2, 0).to(device)  # Create an empty edge_index tensor
            edge_attr = torch.empty(0).to(device)  # No edge attributes
        else:
            print(f"Edge index shape: {edge_index.shape}")
            edge_attr = torch.ones(edge_index.size(1)).to(device)

        return Data(x=node_features, edge_index=edge_index, edge_attr=edge_attr)


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, AttentionalAggregation

class GNNTextSummarizer(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNNTextSummarizer, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim).to(device)
        self.bn1 = nn.BatchNorm1d(hidden_dim).to(device)
        self.conv2 = GCNConv(hidden_dim, hidden_dim).to(device)
        self.bn2 = nn.BatchNorm1d(hidden_dim).to(device)
        self.attention = AttentionalAggregation(nn.Linear(hidden_dim, 1)).to(device)
        self.dropout = nn.Dropout(0.3).to(device)
        self.output_layer = nn.Linear(hidden_dim, output_dim).to(device)
        self.sigmoid = nn.Sigmoid()

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        print("Running GNN forward pass...")

        # Apply convolution layers
        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.dropout(x)

        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.dropout(x)

        # Attention mechanism
        x = self.attention(x)

        # Output layer
        x = self.output_layer(x)
        x = self.sigmoid(x)

        # Normalize the output to ensure it's between 0 and 1
        x = (x - x.min()) / (x.max() - x.min())

        return x

class HybridTextGraphSummarizer(TextGraphSummarizer):
    def __init__(self, summary_size=0.3):
        super().__init__(summary_size=summary_size)
        print(f"Initializing HybridTextGraphSummarizer with T5 model on device: {device}")
        self.abstractive_model = T5ForConditionalGeneration.from_pretrained("t5-small").to(device)
        self.abstractive_tokenizer = T5Tokenizer.from_pretrained("t5-small")

    def abstractive_summarization(self, text_block, min_length=50, max_length=150):
        print("Running abstractive_summarization...")
        inputs = self.abstractive_tokenizer(
            "summarize: " + text_block, return_tensors="pt", max_length=512, truncation=True
        ).to(device)
        print(f"Input tokenized shape: {inputs['input_ids'].shape}")
        summary_ids = self.abstractive_model.generate(
            inputs["input_ids"], max_length=max_length, min_length=min_length, 
            length_penalty=2.0, num_beams=4, early_stopping=True
        )
        summary = self.abstractive_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        print(f"Generated summary length: {len(summary.split())} words")
        return summary

    def chunk_text(self, text, chunk_size=512, overlap_size=50):
        print("Chunking text for abstractive summarization...")
        words = text.split()
        chunks = []
        for i in range(0, len(words), chunk_size - overlap_size):
            chunk = ' '.join(words[i:i + chunk_size])
            print(f"Chunk {len(chunks)}: {len(chunk.split())} words")
            chunks.append(chunk)
            if i + chunk_size >= len(words):
                break
        return chunks

    def hybrid_summarize(self, report):
        # Preprocess text
        words, doc = self.preprocess_text(report)
        sentences = list(doc.sents)
        
        # Get embeddings
        embeddings = self.get_embeddings(words)
        print(f"Embeddings shape: {embeddings.shape}")

        # Adjust embeddings for legal terms and stop words
        adjusted_embeddings = self.adjust_embeddings_for_legal_terms(embeddings, words)

        # Build graph
        G = self.build_graph(words, doc, adjusted_embeddings)
        print(f"Graph has {len(G.nodes)} nodes and {len(G.edges)} edges")

        # Convert to PyG data
        data = self.build_pyg_data(G).to(device)
        print(f"PyG Data: {data.x.shape[0]} nodes, {data.edge_index.shape[1]} edges")

        # Initialize and evaluate GNN model
        input_dim = 1152  # Adjust as needed
        model = GNNTextSummarizer(input_dim=input_dim, hidden_dim=64, output_dim=1).to(device)
        model.eval()
        with torch.no_grad():
            scores = model(data)

        # Process GNN output scores
      # Process GNN output scores
        if scores.dim() > 1:
            scores = scores.squeeze()  # Remove extra dimensions
# Convert the tensor to a numpy array
        scores_np = scores.cpu().numpy()
        print(f"Scores shape: {scores_np.shape}")

# Now scores_np should be a 1D numpy array and you can use len() safely
        if scores_np.ndim == 0:
            scores_np = np.full(len(data.x), scores_np)

# Score sentences
        sentence_scores = []
        for i, sentence in enumerate(sentences):
            token_indices = [token.i for token in sentence if 0 <= token.i < len(scores_np)]
            if token_indices:
                score = np.mean([scores_np[idx] for idx in token_indices])
                sentence_scores.append((i, score, sentence.text))
            else:
                print(f"Sentence {i} has no valid token indices for scoring.")

# Sort and select top sentences for extractive summary
        sentence_scores.sort(key=lambda x: x[1], reverse=True)
        num_sentences = max(1, int(len(sentences) * self.summary_size))
        selected_sentences = sentence_scores[:num_sentences]
        print(f"Selected {num_sentences} sentences for extractive summary")
        extractive_summary = ' '.join([sent[2] for sent in selected_sentences])

        # Chunk and run abstractive summarization
        chunks = self.chunk_text(extractive_summary)
        summaries = []
        for chunk in chunks:
            refined_summary = self.abstractive_summarization(chunk)
            summaries.append(refined_summary)

        # Combine summaries
        final_summary = ' '.join(summaries)
        target_final_length = max(1, int(len(report.split()) * 0.5))
        final_summary_words = final_summary.split()
        final_summary = ' '.join(final_summary_words[:target_final_length])

        return final_summary, extractive_summary, selected_sentences


if __name__ == "__main__":
    report = """
DARGA RAM @ GUNGA v. STATE OF RAJASTHAN [2015] INSC 16 (8 January 2015)
REPORTABLE IN THE SUPREME COURT OF INDIA CRIMINAL APPELLATE JURISDICTION CRIMINAL APPEAL NO. 513 OF 2008 Darga Ram @ Gunga ...Appellant Versus State of Rajasthan ...Respondent
T.S. THAKUR, J.

1\. The appellant was tried and convicted for offences punishable under Sections 376 and 302 IPC. For the offence of rape punishable under Section 376, he was sentenced to undergo imprisonment for a period of 10 years besides a fine of Rs.1000/- and default sentence of one month with rigorous imprisonment. Similarly, for the offence of murder punishable under Section 302 IPC, he was sentenced to undergo life imprisonment besides a fine of Rs.3,000/- and default sentence of three months' rigorous imprisonment. Both the sentences were directed to run concurrently.
Criminal Appeal No.604 of 2004 filed by him was heard and dismissed by a Division Bench of the High Court of Judicature for Rajasthan at Jodhpur.
The present appeal assails the impugned judgment and order.

2\. A first Information Report was registered at Police Station Rani in the State of Rajasthan on 11th April, 1998, inter alia, stating that the complainant on 9th April, 1998 had organised a "Jaagran" (night long prayer meet) near a well belonging to one Magga Ram. The complainant and other relatives, in all around 50 persons assembled for the "Jaagran" that continued till late night. This included his seven year old daughter-Kamala who went to sleep along with other children close to the place where the "Jaagran" was held. When he returned to his house he noticed that Kamala was missing. Assuming that she may have gone away with one of the relatives, a search was made at their houses but Kamala remained untraceable. The search was then extended to neighbouring areas where the dead body of Kamala was discovered by Magga Ram (PW-5) and Pura Ram. On receipt of this information he and Naina Ram (PW-2) went to the place and found that baby Kamala had been raped and killed by crushing her head with a stone. The dead body of Kamala was, according to the report, lying on the spot.

3\. A case under Sections 302 and 376 of the IPC was registered on the basis of the above information and investigation started which led to the arrest of the appellant and eventually a charge sheet against him before the jurisdictional magistrate who committed the case to Additional Sessions Judge, (Fast Track), Bali.

4\. Before the Sessions Court, the appellant pleaded not guilty and claimed a trial. At the trial the prosecution produced 19 witnesses apart from placing reliance upon several documents. No evidence in defence was, however, led by the appellant. By its judgment and order dated 27th January, 2004 the trial Court eventually held the appellant guilty and accordingly convicted and sentenced him as indicated above. Aggrieved by the judgment and order passed by the trial Court, the appellant preferred Criminal Appeal No.604 of 2004 which was, upon reappraisal of the evidence adduced before the trial Court, dismissed by the High Court affirming the conviction recorded against the appellant and the sentence awarded to him for both the offences.
5. We have heard learned counsel for the parties at considerable length.
Prosecution case is based entirely on circumstantial evidence as no ocular account of the incident has been presented to the Court. Both the Courts below have, however, found the circumstantial evidence adduced by the prosecution to be sufficient to record a finding of guilt against the appellant for the offences with which he was charged. We may briefly refer to the circumstance as also the evidence supporting the same.

6\. The first and foremost is the deposition of Ota Ram (PW-4) which clearly establishes that the appellant was also one of those who had participated in the "Jaagran" along with other villagers. To the same effect is the statement of Maga Ram (PW-5) who too had testified that the appellant was present in the "Jaagran". He had seen Kamala at around 10.00 in the night. The deposition of both these witnesses proves that apart from the appellant and several others, baby Kamala the deceased was also present at the "Jaagran" with other children and had gone off to sleep after taking dinner. That version is supported even by Naina (PW-1), who states that the appellant was also present in the "Jaagran" around mid night when the tea was served to those present including the appellant. The witness has further deposed that his son and daughter Kamala were sleeping around the place but Kamala was found missing in the morning. There is, in our opinion, no reason to disbelieve the version of these witnesses when they say that the "Jaagran" was held by the complainant in which Kamala his daughter was present and gone off to sleep nor is there any reason to disbelieve the story that even the appellant was present at the "Jaagran"
and had tea with other witnesses around mid night.

7\. That Kamala died a homicidal death was not seriously disputed either before the Courts below or before us and rightly so because the statement of doctor Omprakash Kuldeep (PW-18) who conducted the post-mortem and authored the report marked as Ex. P-34 has clearly opined that Kamala died a homicidal death on account of injury on her head. In the deposition, the doctor certified injuries even on her private parts. The post-mortem report certifies the following injuries on the person of the deceased:
"1. Face crushed.

2. Upper lip wad cut. Bleeding was from right ear, dried seminal stains on right and left thigh.
3. Nose bone was depressed and fractured.
4. Fracture was on left orbital margin.
5. Fracture was in left temporal bone.
6. Fracture was in maxilla bone of left side.

7. Fracture in parietal bone and occipital bone of right side which was upto the base of skull.
8. Incise teeth of lower and upper (jaw) were broken.
9. Achaimoisis was present in Genital organs labia.
10. Crushing wound was on forechet and perineum.
11. Hymn was congested."

8\. Rajendra Singh (PW-9), who investigated the case and who is a witness to the scene of occurrence, seized blood stained clothes of the deceased including two hair recovered from the private parts of the deceased. He is also witness to the seizure of blood stained clothes of the appellant on the basis of a disclosure statement made by him. Equally important is the circumstance that the FSL report found the trouser and the shirt of the appellant to be stained with human blood belonging to group 'A' which happened to be the blood group of the deceased also. The stone used for crushing the head of the deceased was also found to be smeared with human blood of group 'A'.

9\. What supports the prosecution case in a great measure is also the fact that the appellant had suffered multiple injuries on his private parts. The medical examination report dated 13th April, 1998 marked as Ex.
P-38 has noticed the following injuries on the person of the appellant:
"(i) Abrasion 1x0.5 cm. Size Dorsal Aspect of (Rt) Elbow joint.
(ii) Abrasion 3x2 cm. Size Medical Aspect of (Lt) Elbow joint.
(iii) Multiple Abrasion Varying in Size Dorsal Aspect of (Lt) Elbow joint.
(iv) Abrasion 7.5x1 cm. Size Ant. aspect of (Rt.) leg Just below (Rt.) knee joint (v) Abrasion 1.5x1 cm. Ant. aspect of (Lt.) knee joint (vi) Abrasion 1x0.5 cm. Medial side of Ant. Aspect (Lt.) knee joint (vii) Abrasion 1x1 cm. Lt. side of Ant. Aspect of (Lt.) knee joint (viii) Abrasion 1x0.5 cm. Dorsal Aspect of Retracted Prepuce.
(ix) Abrasion 2x0.25 cm. Lat. Aspect of (Rt.) side of Retracted prepuce.
(x) Abrasion 0.25x0.25 cm. Dorsal Aspect of glans penis (xi) Abrasion 2x0.25 cm. Lat. Aspect of (Rt.0 Thigh (xii) Abrasion 2x0.25 cm. (Rt.) gluteal Region (xiii) Abrasion 2x1 cm. (Lt.) Palm Duration of all injuries i.e. S.No. i to xiii is 3-5 days. "

10\. No explanation was, however, offered by the appellant for the injuries sustained by him one of which was found even at his penis. To summarise, the prosecution has clearly established:
(1) That a "Jaagran" was arranged by the complainant on the offside of village near the well in which nearly 50 people participated including Kamala the deceased child.
(2) The deceased-Kamala had gone out to sleep after dinner around mid night.
(3) The appellant was also participating in the "Jaagran" and was seen sitting along with some of the prosecution witnesses.
(4) Kamala-deceased was found missing in the morning but upon search her dead body was noticed at some distance in the village in a naked condition with injuries on her private parts and her head smashed with a stone lying nearby.
(5) The appellant made a disclosure statement leading to the recovery of his blood stained clothes.
(6) The blood was found to be of human origin and belonging to group 'A"
which also was the blood group of the deceased-Kamala.
(7) The appellant on medical examination was found to have several injuries on his body including injury on his penis.
(8) The injuries found on the person of the appellant were said to be 3 to 5 days old.
(9) The appellant did not offer any explanation for the injuries on his body.

11\. The above circumstances, in our opinion, form a complete chain and lead to an irresistible conclusion that the appellant was responsible for the offence of rape and murder of the hapless baby-Kamala who appears to have been picked up from the place where she was sleeping with other children and taken at a distance only to be raped and eventually killed.
The trial Court, in the light of the evidence on record and careful analysis undertaken by it, correctly came to the conclusion that the appellant was guilty of murder of the deceased. There is no reason whatsoever for us to interfere with that finding.

12\. What remains to be addressed now is an application filed by the appellant in this Court seeking to raise a plea that the appellant was a juvenile on the date of the commission of offence hence entitled to the benefit of Juvenile Justice (Care and Protection of Children) Act, 2000.
Since the appellant did not have any documentary evidence like a school or other certificate referred to under the Act mentioned above, this Court had directed the Principal, Government Medical College, Jodhpur, to constitute a Board of Doctors for medical examination including radiological examination of the appellant to determine the age of the appellant as in April, 1998 when the offence in question was committed. The Superintendant of the Central Jail was directed to ensure production of the appellant for the purpose of determination of his age before the Medical Board for carrying out the tests and examination. In compliance with the said direction, the Principal constituted a Medical Board for determining the age of the appellant and submitted a report dated 4th February, 2014. The report records the following findings and conclusions:
"Age estimation of Darga ram @ Gunga s/o Heera on the basis of findings of X Ray of Elbow, Wrist, Pelvis, Sternum, Medial end of Clavicle, Skull and left shoulder joint (film no.10252 dated 04-02-2014, Eight Film and CT Scan of Skull and Mandible (film 56013, four films) dated 04-02-2014, is as below:-
1. All Epiphysis around elbow joint, lower end of Radius & Ulna, Ilias Crest & Ischial tuberosity & for medial end of Clavicle have appeared 7 fused, it suggests that his age is above 22 years.

2. All the body pieces of sternum have fused with each other but not fused with Xiphoid process & manubrium sternum, it suggests his age is above 25 years but below 40 years.

3. Posterior 1/3 of sagital suture have fused, it suggests his age is above 30 years & below 40 years.

4. Ventral 7 Dorsal margins of pubic symphysis are completely defined 7 there are no granular appearance on it, it suggests his age is below 36 years.
Opinion:- Concluding all the above radiological findings, dental & Clinical appearance, the age of Darga Ram @ Gunga S/o Heera is in between 30 years to 36 years and the average age of Darga Ram @ Gunga S/o Heera is about 33 years on the date of examination.
Enclosure:- X Ray (8 plates) & CT Scan 4 Plates) as above.
Sd/- Sd/- Sd/- (Dr. L. Raichandani) (Dr. A.L.Chauhan) (Dr. P.C.
Vyas) Professor, Anatomy PHOD, Radiodiagnosis PHOD, forensic Medicine Dr. S.N. Medical College Dr. S.N. Medical College Dr. S.N. Medical College Jodhpur Jodhpur Jodhpur"

13\. It is evident from the opinion tendered by the Board that the appellant's age has been placed in the range of 30 to 36 years. The Board appears to have taken the average of two extremitees and concluded that the appellant's age on the date of the examination was about 33 years. It was on the basis of this estimate that Mr. Panjwani contended that the appellant should have been around 14 years, 2 months and 7 days old if his age was 30 years on the date of medical examination. He should have been 17 years, 2 months and 7 days old on the date of the occurrence if his age is taken as 33 years and 20 years, 2 months and 7 days if his age is taken as 36 years on the date of the medical examination. It was argued that even if one were to accept the average of the two estimates in the range of 30- 36 years, mentioned by the Medical Board, he was a juvenile on the date of the occurrence being only 17 years, 2 months hence entitled to the benefit of the provisions of the Juvenile Justice (Care and Protection of Children) Act, 2000.

14\. The appellant is reported to be a deaf and dumb. He was never admitted to any school. There is, therefore, no officially maintained record regarding his date of birth. Determination of his age on the date of the commission of the offence is, therefore, possible only by reference to the medical opinion obtained from the duly constituted Medical Board in terms of Rule 12(3) (b) of the Juvenile Justice (Care and Protection of Children) Rules, 2007. Rule 12(3)(b) reads as under:
"12. Procedure to be followed in determination of Age.- (1) xxxxxxxxxxxxxxx (2) xxxxxxxxxxxxxxx (3) (b) and only in the absence of either (i), (ii) or (iii) of clause (a) above, the medical opinion will be sought from a duly constituted Medical Board, which will declare the age of the juvenile or child. In case exact assessment of the age cannot be done, the Court or the Board or, as the case may be, the Committee, for the reasons to be recorded by them, may, if considered necessary, give benefit to the child or juvenile by considering his/her age on lower side within the margin of one year.
and, while passing orders in such case shall, after taking into consideration such evidence as may be available, or the medical opinion, as the case may be, record a finding in respect of his age and either of the evidence specified in any of the clauses (a)(i), (ii), (iii) or in the absence whereof, clause (b) shall be the conclusive proof of the age as regards such child or the juvenile in conflict with law"

15\. The medical opinion given by the duly constituted Board comprising Professors of Anatomy, Radiodiagnosis and Forensic Medicine has determined his age to be "about" 33 years on the date of the examination. The Board has not been able to give the exact age of the appellant on medical examination no matter advances made in that field. That being so in terms of Rule 12 (3) (b) the appellant may even be entitled to benefit of fixing his age on the lower side within a margin of one year in case the Court considers it necessary to do so in the facts and circumstances of the case.
The need for any such statutory concession may not however arise because even if the estimated age as determined by the Medical Board is taken as the correct/true age of the appellant he was just about 17 years and 2 months old on the date of the occurrence and thus a juvenile within the meaning of that expression as used in the Act aforementioned. Having said that we cannot help observing that we have not felt very comfortable with the Medical Board estimating the age of the appellant in a range of 30 to 36 years as on the date of the medical examination. The general rule about age determination is that the age as determined can vary plus minus two years but the Board has in the case at hand spread over a period of six years and taken a mean to fix the age of the appellant at 33 years. We are not sure whether that is the correct way of estimating the age of the appellant. What reassures us about the estimate of age is the fact that the same is determined by a Medical Board comprising Professors of Anatomy, Radiodiagnosis and Forensic Medicine whose opinion must get the respect it deserves. That apart even if the age of the appellant was determined by the upper extremity limit i.e. 36 years the same would have been subject to variation of plus minus 2 years meaning thereby that he could as well be 34 years on the date of the examination. Taking his age as 34 years on the date of the examination he would have been 18 years, 2 months and 7 days on the date of the occurrence but such an estimate would be only an estimate and the appellant may be entitled to additional benefit of one year in terms of lowering his age by one year in terms of Rule 12 (3)(b) (supra) which would then bring him to be 17 years and 2 months old, therefore, a juvenile.

16\. In the totality of the circumstances, we have persuaded ourselves to go by the age estimate given by the Medical Board and to declare the appellant to be a juvenile as on the date of the occurrence no matter the offence committed by him is heinous and but for the protection available to him under the Act the appellant may have deserved the severest punishment permissible under law. The fact that the appellant has been in jail for nearly 14 years is the only cold comfort for us to let out of jail one who has been found guilty of rape and murder of an innocent young child.

17\. In the result, this appeal succeeds but only in part and to the extent that while the conviction of the appellant for offences under Section 302 and 376 of IPC is affirmed the sentence awarded to him shall stand set aside with a direction that the appellant shall be set free from prison unless required in connection with any other case.


"""
    summarizer = HybridTextGraphSummarizer(summary_size=0.55)  # Adjust summary size as needed
    final_summary, extractive_summary, selected_sentences = summarizer.hybrid_summarize(report)
    print("Extractive Summary:", extractive_summary)
    print("Abstractive Summary:", final_summary)
   


Using device: cuda
Initializing TextGraphSummarizer on device: cuda
Initializing HybridTextGraphSummarizer with T5 model on device: cuda
Running preprocess_text...
Extracted 2210 words after preprocessing.
Running get_embeddings...
Using first 512 words for embedding.
Sentence embeddings shape: torch.Size([512, 384])
Legal embeddings shape: torch.Size([512, 768])
Combined embeddings shape: torch.Size([512, 1152])
Embeddings shape: torch.Size([512, 1152])
Running adjust_embeddings_for_legal_terms...
Adjusted embeddings shape: torch.Size([512, 1152])
Running build_graph...
TF-IDF dictionary size: 682
Number of sentences identified: 128
Graph construction complete: 527 nodes, 8127 edges
Graph has 527 nodes and 8127 edges
Running build_pyg_data...
Edge index shape: torch.Size([2, 8127])
PyG Data: 3692 nodes, 8127 edges
Running GNN forward pass...
Scores shape: ()
Selected 70 sentences for extractive summary
Chunking text for abstractive summarization...
Chunk 0: 512 words
Chunk 1: 512 word

In [60]:
pip install rouge-score


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [61]:
!pip install --upgrade rouge-score






huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [62]:
!pip install transformers sentence-transformers scipy sklearn
!pip install gensim


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
    

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.8.3 requires cubinlinker, which is not installed.
cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.3 requires ptxcompiler, which is not installed.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
ucxx 0.39.1 requires libucx>=1.15.0, which is not installed.
apache-beam 2.46.0 requires cloudpickle~=2.2.1, but you have cloudpickle 3.0.0 which is incompa

In [63]:
pip install bert-score


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [64]:
!pip install rouge-score nltk scikit-learn


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [65]:
!pip install py-meteor


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: Could not find a version that satisfies the requirement py-meteor (from versions: none)
ERROR: No matching distribution found for py-meteor


In [71]:
import torch
from transformers import BertTokenizer, BertModel
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity

import nltk
from rouge_score import rouge_scorer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from nltk.translate.bleu_score import sentence_bleu

# Function to compute BERTScore
def compute_bertscore(reference_text, generated_text):
    from bert_score import score
    # Calculate BERTScore using the default model
    P, R, F1 = score([generated_text], [reference_text], lang="en")
    print(f"BERTScore Precision: {P.item()}, Recall: {R.item()}, F1: {F1.item()}")
    return P.item(), R.item(), F1.item()

# Function to compute Sentence-BERT Cosine Similarity
def compute_sbert_cosine(reference_text, generated_text):
    # Load pre-trained Sentence-BERT model
    model = SentenceTransformer('all-MiniLM-L6-v2')
    # Get embeddings for both texts
    reference_embedding = model.encode(reference_text)
    generated_embedding = model.encode(generated_text)
    # Compute cosine similarity between the embeddings
    similarity = cosine(reference_embedding, generated_embedding)
    print(f"Sentence-BERT Cosine Similarity: {1 - similarity}")  # Cosine similarity ranges from 0 to 1
    return 1 - similarity

# Function to compute ROUGE score
def compute_rouge(reference_text, generated_text):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference_text, generated_text)
    print("ROUGE Scores:")
    for metric, score in scores.items():
        print(f"{metric}: {score.fmeasure:.4f}")
    return scores

# Function to compute BLEU score
def compute_bleu(reference_text, generated_text):
    reference = [reference_text.split()]  # BLEU expects list of lists of words
    generated = generated_text.split()
    bleu_score = sentence_bleu(reference, generated)
    print(f"BLEU Score: {bleu_score:.4f}")
    return bleu_score

# Function to compute METEOR score

# Function to compute Cosine Similarity
def compute_cosine_similarity(reference_text, generated_text):
    # Create word embeddings using TF-IDF or Sentence Transformers (using sklearn for simplicity)
    from sklearn.feature_extraction.text import TfidfVectorizer
    
    vectorizer = TfidfVectorizer().fit([reference_text, generated_text])
    tfidf_matrix = vectorizer.transform([reference_text, generated_text])
    cosine_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
    print(f"Cosine Similarity: {cosine_sim[0][0]:.4f}")
    return cosine_sim[0][0]



# Function to compute Word Mover's Distance


# Example texts
reference_text = """A first Information Report was registered at Police Station Rani in the State of Rajasthan on 11th April, 1998, inter alia, stating that the complainant on 9th April, 1998 had organised a "Jaagran" (night long prayer meet) near a well belonging to one Magga Ram.
The complainant and other relatives, in all around 50 persons assembled for the "Jaagran" that continued till late night.
This included his seven year old daughter-Kamala who went to sleep along with other children close to the place where the "Jaagran" was held.
When he returned to his house he noticed that Kamala was missing.
The dead body of Kamala was, according to the report, lying on the spot.
A case under Sections 302 and 376 of the IPC was registered on the basis of the above information and investigation started which led to the arrest of the appellant and eventually a charge sheet against him before the jurisdictional magistrate who committed the case to Additional Sessions Judge, (Fast Track), Bali.
Prosecution case is based entirely on circumstantial evidence as no ocular account of the incident has been presented to the Court.
Both the Courts below have, however, found the circumstantial evidence adduced by the prosecution to be sufficient to record a finding of guilt against the appellant for the offences with which he was charged.
What remains to be addressed now is an application filed by the appellant in this Court seeking to raise a plea that the appellant was a juvenile on the date of the commission of offence hence entitled to the benefit of Juvenile Justice (Care and Protection of Children) Act, 2000.
Since the appellant did not have any documentary evidence like a school or other certificate referred to under the Act mentioned above, this Court had directed the Principal, Government Medical College, Jodhpur, to constitute a Board of Doctors for medical examination including radiological examination of the appellant to determine the age of the appellant as in April, 1998 when the offence in question was committed.
"Age estimation of Darga ram @ Gunga s/o Heera on the basis of findings of X Ray of Elbow, Wrist, Pelvis, Sternum, Medial end of Clavicle, Skull and left shoulder joint (film no.10252 dated 04-02-2014, Eight Film and CT Scan of Skull and Mandible (film 56013, four films) dated 04-02-2014, is as below:-
It is evident from the opinion tendered by the Board that the appellant's age has been placed in the range of 30 to 36 years.
The Board appears to have taken the average of two extremitees and concluded that the appellant's age on the date of the examination was about 33 years.
The appellant is reported to be a deaf and dumb.
The medical opinion given by the duly constituted Board comprising Professors of Anatomy, Radiodiagnosis and Forensic Medicine has determined his age to be "about" 33 years on the date of the examination.
Taking his age as 34 years on the date of the examination he would have been 18 years, 2 months and 7 days on the date of the occurrence but such an estimate would be only an estimate and the appellant may be entitled to additional benefit of one year in terms of lowering his age by one year in terms of Rule 12 (3)(b) (supra) which would then bring him to be 17 years and 2 months old, therefore, a juvenile."
"""
generated_text = """darga ram @ gunga v. state of rajasthan [2015] insc 16 (8 january 2015)
reportable in the supreme court of india criminal appellate jurisdiction criminal appeal no. 513 of 2008 darga ram @ gunga ...appellant versus state of rajasthan ... respondent
t.s. thakur, j.

1\. the appellant was tried and convicted for offences punishable under sections 376 and 302 ipc. for the offence of rape punishable under section 376, he was sentenced to undergo imprisonment for a period of 10 years besides a fine of rs.1000/- and default sentence of one month with rigorous imprisonment. similarly, for the offence of murder punishable under section 302 ipc, he was sentenced to undergo life imprisonment besides a fine of rs.3,000/- and default sentence of three months' rigorous imprisonment. both the sentences were directed to run concurrently.
criminal appeal no.604 of 2004 filed by him was heard and dismissed by a division bench of the high court of judicature for rajasthan at jodhpur.
 the present appeal assails the impugned judgment and order.

 2\. a first information report was registered at police station rani in the state of rajasthan on 11th april, 1998, inter alia, stating that the complainant on 9th april, 1998 had organised a "jaagran" (night long prayer meet) near a well belonging to one magga ram. the complainant and other relatives, in all around 50 persons assembled for the "jaagran" that continued till late night. this included his seven year old daughter-kamala who went to sleep along with other children close to the place where the "jaagran" was held. when he returned to his house he noticed that kamala was missing. assuming that she may have gone away with one of the relatives, a search was made at their houses but kamala remained untraceable. the search was then extended to neighbouring areas where the dead body of kamala was discovered by magga ram (pw-5) and pura ram. on receipt of this information he and naina ram (pw-2) went to the place and found that baby kamala had been raped and killed by crushing her head with a stone. the dead body of kamala was, according to the report, lying on the spot.

 3\. a case under sections 302 and 376 of the ipc was registered on the basis of the above information and investigation started which led to the arrest of the appellant and eventually a charge sheet against him before the jurisdictional magistrate who committed the case to additional sessions judge, (fast track), bali.

 4\. before the sessions court, the appellant pleaded not guilty and claimed a trial. at the trial the prosecution produced 19 witnesses apart from placing reliance upon several documents. no evidence in defence was, however, led by the appellant. by its judgment and order dated 27th january, 2004 the trial court eventually held the appellant guilty and accordingly convicted and sentenced him as indicated above. aggrieved by the judgment and order passed by the trial court, the appellant preferred criminal appeal no.604 of 2004 which was, upon reappraisal of the evidence adduced before the trial court, dismissed by the high court affirming the conviction recorded against the appellant and the sentence awarded to him for both the offences.
 5. we have heard learned counsel for the parties at considerable length.
 prosecution case is based entirely on circumstantial evidence as no ocular account of the incident has been presented to the court. both the courts below have, however, found the circumstantial evidence adduced by the prosecution to be sufficient to record a finding of guilt against the appellant for the offences with which he was charged. we may briefly refer to the circumstance as also the evidence supporting the same.

 6\. the first and foremost is the deposition of ota ram (pw-4) which clearly establishes that the appellant was also one of those who had participated in the "jaagran" along with other villagers. to the same effect is the statement of maga ram (pw-5) who too had testified that the appellant was present in the "jaagran". he had seen kamala at around 10.00 in the night. the deposition of both these witnesses proves that apart from the appellant and several others, baby kamala the deceased was also present at the "jaagran" with other children and had gone off to sleep after taking dinner. that version is supported even by naina (pw-1), who states that the appellant was also present in the "jaagran" around mid night when the tea was served to those present including the appellant. the witness has further deposed that his son and daughter kamala were sleeping around the place but kamala was found missing in the morning. there is, in our opinion, no reason to disbelieve the version of these witnesses when they say that the "jaagran" was held by the complainant in which kamala his daughter was present and gone off to sleep nor is there any reason to disbelieve the story that even the appellant was present at the "jaagran"
and had tea with other witnesses around mid night.

 7\. that kamala died a homicidal death was not seriously disputed either before the courts below or before us and rightly so because the statement of doctor omprakash kuldeep (pw-18) who conducted the post-mortem and authored the report marked as ex. p-34 has clearly opined that kamala died a homicidal death on account of injury on her head. in the deposition, the doctor certified injuries even on her private parts. the post-mortem report certifies the following injuries on the person of the deceased:
"1. face crushed.

 2. upper lip wad cut. bleeding was from right ear, dried seminal stains on right and left thigh.
 3. nose bone was depressed and fractured.
 4. fracture was on left orbital margin.
 5. fracture was in left temporal bone.
 6. fracture was in maxilla bone of left side.

 7. fracture in parietal bone and occipital bone of right side which was upto the base of skull.
 8. incise teeth of lower and upper (jaw) were broken.
 9. achaimoisis was present in genital organs labia.
 10. crushing wound was on forechet and perineum.
 11. hymn was congested. "

8\. rajendra singh (pw-9), who investigated the case and who is a witness to the scene of occurrence, seized blood stained clothes of the deceased including two hair recovered from the private parts of the deceased. he is also witness to the seizure of blood stained clothes of the appellant on the basis of a disclosure statement made by him. equally important is the circumstance that the fsl report found the trouser and the shirt of the appellant to be stained with human blood belonging to group 'a' which happened to be the blood group of the deceased also. the stone used for crushing the head of the deceased was also found to be smeared with human blood of group 'a'.

 9\. what supports the prosecution case in a great measure is also the fact that the appellant had suffered multiple injuries on his private parts. the medical examination report dated 13th april, 1998 marked as ex.
p-38 has noticed the following injuries on the person of the appellant:
"(i) abrasion 1x0.5 cm. size dorsal aspect of (rt) elbow joint.
 (ii) abrasion 3x2 cm. size medical aspect of (lt) elbow joint.
(iii) multiple abrasion varying in size dorsal aspect of (lt) elbow joint.
(iv) abrasion 7.5x1 cm . size ant. aspect of (rt.) leg just below (rt.) knee joint (v) abrasion 1.5x1 cm. ant. aspect of (lt.) knee joint (vi) abrasion 1x0.5 cm. medial side of ant. aspect (lt.) knee joint (vii) abrasion 1x1 cm. lt. side of ant. aspect of (lt.) knee joint (viii) abrasion 1x0.5 cm. dorsal aspect of retracted prepuce.
 (ix) abrasion 2x0.25 cm."""
compute_bertscore(reference_text, generated_text)

print("\nCalculating Sentence-BERT Cosine Similarity...")
compute_sbert_cosine(reference_text, generated_text)
compute_cosine_similarity(reference_text, generated_text)
# Compute different metrics
compute_rouge(reference_text, generated_text)
compute_bleu(reference_text, generated_text)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore Precision: 0.8433066606521606, Recall: 0.841445803642273, F1: 0.8423752188682556

Calculating Sentence-BERT Cosine Similarity...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Sentence-BERT Cosine Similarity: 0.6256222278621513
Cosine Similarity: 0.8797
ROUGE Scores:
rouge1: 0.4498
rouge2: 0.2853
rougeL: 0.3232
BLEU Score: 0.1508


0.150768956732956